In [ ]:
Queries Iceberg

In [3]:
%%configure -f
{
  "conf": {
    "spark.sql.extensions": "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions",
    "spark.sql.catalog.dev": "org.apache.iceberg.spark.SparkCatalog",
    "spark.sql.catalog.dev.warehouse": "s3://apacheiceberg/iceberg100",
    "spark.sql.catalog.dev.catalog-impl": "org.apache.iceberg.aws.glue.GlueCatalog",
    "spark.sql.catalog.dev.io-impl": "org.apache.iceberg.aws.s3.S3FileIO",
    "spark.sql.catalog.dev.lock-impl": "org.apache.iceberg.aws.dynamodb.DynamoDbLockManager",
    "spark.sql.catalog.dev.lock.table": "myGlueLockTable"
  }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1705790429480_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1705790429480_0002,pyspark,idle,Link,Link,None,✔


In [4]:
spark.sql("SHOW DATABASES").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+
|    namespace|
+-------------+
|      default|
|   deltalake1|
| deltalake100|
|deltalake1000|
|  deltalake11|
|   iceberg100|
|  iceberg1000|
|    iceberg11|
|  icebergtest|
+-------------+

In [5]:
spark.sql("use DATABASE iceberg100")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [3]:
## Select databases
db = 'dev.iceberg100'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
queries = {
    "q1": f"""
	with customer_total_return as
	(select sr_customer_sk as ctr_customer_sk
	,sr_store_sk as ctr_store_sk
	,sum(SR_FEE) as ctr_total_return
	from {db}.store_returns
	,{db}.date_dim
	where sr_returned_date_sk = d_date_sk
	and d_year =2000
	group by sr_customer_sk
	,sr_store_sk)
	 select  c_customer_id
	from customer_total_return ctr1
	,{db}.store
	,{db}.customer
	where ctr1.ctr_total_return > (select avg(ctr_total_return)*1.2
	from customer_total_return ctr2
	where ctr1.ctr_store_sk = ctr2.ctr_store_sk)
	and s_store_sk = ctr1.ctr_store_sk
	and s_state = 'TN'
	and ctr1.ctr_customer_sk = c_customer_sk
	order by c_customer_id
	limit 100""",
    "q2": f"""
	with wscs as
	 (select sold_date_sk
			,sales_price
	  from (select ws_sold_date_sk sold_date_sk
				  ,ws_ext_sales_price sales_price
			from {db}.web_sales
			union all
			select cs_sold_date_sk sold_date_sk
				  ,cs_ext_sales_price sales_price
			from {db}.catalog_sales)),
	 wswscs as
	 (select d_week_seq,
			sum(case when (d_day_name='Sunday') then sales_price else null end) sun_sales,
			sum(case when (d_day_name='Monday') then sales_price else null end) mon_sales,
			sum(case when (d_day_name='Tuesday') then sales_price else  null end) tue_sales,
			sum(case when (d_day_name='Wednesday') then sales_price else null end) wed_sales,
			sum(case when (d_day_name='Thursday') then sales_price else null end) thu_sales,
			sum(case when (d_day_name='Friday') then sales_price else null end) fri_sales,
			sum(case when (d_day_name='Saturday') then sales_price else null end) sat_sales
	 from wscs
		 ,{db}.date_dim
	 where d_date_sk = sold_date_sk
	 group by d_week_seq)
	 select d_week_seq1
		   ,round(sun_sales1/sun_sales2,2)
		   ,round(mon_sales1/mon_sales2,2)
		   ,round(tue_sales1/tue_sales2,2)
		   ,round(wed_sales1/wed_sales2,2)
		   ,round(thu_sales1/thu_sales2,2)
		   ,round(fri_sales1/fri_sales2,2)
		   ,round(sat_sales1/sat_sales2,2)
	 from
	 (select wswscs.d_week_seq d_week_seq1
			,sun_sales sun_sales1
			,mon_sales mon_sales1
			,tue_sales tue_sales1
			,wed_sales wed_sales1
			,thu_sales thu_sales1
			,fri_sales fri_sales1
			,sat_sales sat_sales1
	  from wswscs,{db}.date_dim
	  where {db}.date_dim.d_week_seq = wswscs.d_week_seq and
			d_year = 1998) y,
	 (select wswscs.d_week_seq d_week_seq2
			,sun_sales sun_sales2
			,mon_sales mon_sales2
			,tue_sales tue_sales2
			,wed_sales wed_sales2
			,thu_sales thu_sales2
			,fri_sales fri_sales2
			,sat_sales sat_sales2
	  from wswscs
		  ,{db}.date_dim
	  where {db}.date_dim.d_week_seq = wswscs.d_week_seq and
			d_year = 1998+1) z
	 where d_week_seq1=d_week_seq2-53
	 order by d_week_seq1""",
    "q3": f"""
	select  dt.d_year
		   ,{db}.item.i_brand_id brand_id
		   ,{db}.item.i_brand brand
		   ,sum(ss_sales_price) sum_agg
	 from  {db}.date_dim dt
		  ,{db}.store_sales
		  ,{db}.item
	 where dt.d_date_sk = {db}.store_sales.ss_sold_date_sk
	   and {db}.store_sales.ss_item_sk = {db}.item.i_item_sk
	   and {db}.item.i_manufact_id = 816
	   and dt.d_moy=11
	 group by dt.d_year
		  ,{db}.item.i_brand
		  ,{db}.item.i_brand_id
	 order by dt.d_year
			 ,sum_agg desc
			 ,brand_id
	 limit 100""",
    "q4": f"""
	with year_total as (
	 select c_customer_id customer_id
		   ,c_first_name customer_first_name
		   ,c_last_name customer_last_name
		   ,c_preferred_cust_flag customer_preferred_cust_flag
		   ,c_birth_country customer_birth_country
		   ,c_login customer_login
		   ,c_email_address customer_email_address
		   ,d_year dyear
		   ,sum(((ss_ext_list_price-ss_ext_wholesale_cost-ss_ext_discount_amt)+ss_ext_sales_price)/2) year_total
		   ,'s' sale_type
	 from {db}.customer
		 ,{db}.store_sales
		 ,{db}.date_dim
	 where c_customer_sk = ss_customer_sk
	   and ss_sold_date_sk = d_date_sk
	 group by c_customer_id
			 ,c_first_name
			 ,c_last_name
			 ,c_preferred_cust_flag
			 ,c_birth_country
			 ,c_login
			 ,c_email_address
			 ,d_year
	 union all
	 select c_customer_id customer_id
		   ,c_first_name customer_first_name
		   ,c_last_name customer_last_name
		   ,c_preferred_cust_flag customer_preferred_cust_flag
		   ,c_birth_country customer_birth_country
		   ,c_login customer_login
		   ,c_email_address customer_email_address
		   ,d_year dyear
		   ,sum((((cs_ext_list_price-cs_ext_wholesale_cost-cs_ext_discount_amt)+cs_ext_sales_price)/2) ) year_total
		   ,'c' sale_type
	 from {db}.customer
		 ,{db}.catalog_sales
		 ,{db}.date_dim
	 where c_customer_sk = cs_bill_customer_sk
	   and cs_sold_date_sk = d_date_sk
	 group by c_customer_id
			 ,c_first_name
			 ,c_last_name
			 ,c_preferred_cust_flag
			 ,c_birth_country
			 ,c_login
			 ,c_email_address
			 ,d_year
	union all
	 select c_customer_id customer_id
		   ,c_first_name customer_first_name
		   ,c_last_name customer_last_name
		   ,c_preferred_cust_flag customer_preferred_cust_flag
		   ,c_birth_country customer_birth_country
		   ,c_login customer_login
		   ,c_email_address customer_email_address
		   ,d_year dyear
		   ,sum((((ws_ext_list_price-ws_ext_wholesale_cost-ws_ext_discount_amt)+ws_ext_sales_price)/2) ) year_total
		   ,'w' sale_type
	 from {db}.customer
		 ,{db}.web_sales
		 ,{db}.date_dim
	 where c_customer_sk = ws_bill_customer_sk
	   and ws_sold_date_sk = d_date_sk
	 group by c_customer_id
			 ,c_first_name
			 ,c_last_name
			 ,c_preferred_cust_flag
			 ,c_birth_country
			 ,c_login
			 ,c_email_address
			 ,d_year
			 )
	  select
					  t_s_secyear.customer_id
					 ,t_s_secyear.customer_first_name
					 ,t_s_secyear.customer_last_name
					 ,t_s_secyear.customer_birth_country
	 from year_total t_s_firstyear
		 ,year_total t_s_secyear
		 ,year_total t_c_firstyear
		 ,year_total t_c_secyear
		 ,year_total t_w_firstyear
		 ,year_total t_w_secyear
	 where t_s_secyear.customer_id = t_s_firstyear.customer_id
	   and t_s_firstyear.customer_id = t_c_secyear.customer_id
	   and t_s_firstyear.customer_id = t_c_firstyear.customer_id
	   and t_s_firstyear.customer_id = t_w_firstyear.customer_id
	   and t_s_firstyear.customer_id = t_w_secyear.customer_id
	   and t_s_firstyear.sale_type = 's'
	   and t_c_firstyear.sale_type = 'c'
	   and t_w_firstyear.sale_type = 'w'
	   and t_s_secyear.sale_type = 's'
	   and t_c_secyear.sale_type = 'c'
	   and t_w_secyear.sale_type = 'w'
	   and t_s_firstyear.dyear =  1999
	   and t_s_secyear.dyear = 1999+1
	   and t_c_firstyear.dyear =  1999
	   and t_c_secyear.dyear =  1999+1
	   and t_w_firstyear.dyear = 1999
	   and t_w_secyear.dyear = 1999+1
	   and t_s_firstyear.year_total > 0
	   and t_c_firstyear.year_total > 0
	   and t_w_firstyear.year_total > 0
	   and case when t_c_firstyear.year_total > 0 then t_c_secyear.year_total / t_c_firstyear.year_total else null end
			   > case when t_s_firstyear.year_total > 0 then t_s_secyear.year_total / t_s_firstyear.year_total else null end
	   and case when t_c_firstyear.year_total > 0 then t_c_secyear.year_total / t_c_firstyear.year_total else null end
			   > case when t_w_firstyear.year_total > 0 then t_w_secyear.year_total / t_w_firstyear.year_total else null end
	 order by t_s_secyear.customer_id
			 ,t_s_secyear.customer_first_name
			 ,t_s_secyear.customer_last_name
			 ,t_s_secyear.customer_birth_country
	limit 100""",
    "q5": f"""
	with ssr as
	 (select s_store_id,
			sum(sales_price) as sales,
			sum(profit) as profit,
			sum(return_amt) as returns,
			sum(net_loss) as profit_loss
	 from
	  ( select  ss_store_sk as store_sk,
				ss_sold_date_sk  as date_sk,
				ss_ext_sales_price as sales_price,
				ss_net_profit as profit,
				cast(0 as decimal(7,2)) as return_amt,
				cast(0 as decimal(7,2)) as net_loss
		from {db}.store_sales
		union all
		select sr_store_sk as store_sk,
			   sr_returned_date_sk as date_sk,
			   cast(0 as decimal(7,2)) as sales_price,
			   cast(0 as decimal(7,2)) as profit,
			   sr_return_amt as return_amt,
			   sr_net_loss as net_loss
		from {db}.store_returns
	   ) salesreturns,
		 {db}.date_dim,
		 {db}.store
	 where date_sk = d_date_sk
		   and d_date between cast('2000-08-19' as date)
					  and (cast('2000-08-19' as date) +  INTERVAL 14 days)
		   and store_sk = s_store_sk
	 group by s_store_id)
	 ,
	 csr as
	 (select cp_catalog_page_id,
			sum(sales_price) as sales,
			sum(profit) as profit,
			sum(return_amt) as returns,
			sum(net_loss) as profit_loss
	 from
	  ( select  cs_catalog_page_sk as page_sk,
				cs_sold_date_sk  as date_sk,
				cs_ext_sales_price as sales_price,
				cs_net_profit as profit,
				cast(0 as decimal(7,2)) as return_amt,
				cast(0 as decimal(7,2)) as net_loss
		from {db}.catalog_sales
		union all
		select cr_catalog_page_sk as page_sk,
			   cr_returned_date_sk as date_sk,
			   cast(0 as decimal(7,2)) as sales_price,
			   cast(0 as decimal(7,2)) as profit,
			   cr_return_amount as return_amt,
			   cr_net_loss as net_loss
		from {db}.catalog_returns
	   ) salesreturns,
		 {db}.date_dim,
		 {db}.catalog_page
	 where date_sk = d_date_sk
		   and d_date between cast('2000-08-19' as date)
					  and (cast('2000-08-19' as date) +  INTERVAL 14 days)
		   and page_sk = cp_catalog_page_sk
	 group by cp_catalog_page_id)
	 ,
	 wsr as
	 (select web_site_id,
			sum(sales_price) as sales,
			sum(profit) as profit,
			sum(return_amt) as returns,
			sum(net_loss) as profit_loss
	 from
	  ( select  ws_web_site_sk as wsr_web_site_sk,
				ws_sold_date_sk  as date_sk,
				ws_ext_sales_price as sales_price,
				ws_net_profit as profit,
				cast(0 as decimal(7,2)) as return_amt,
				cast(0 as decimal(7,2)) as net_loss
		from {db}.web_sales
		union all
		select ws_web_site_sk as wsr_web_site_sk,
			   wr_returned_date_sk as date_sk,
			   cast(0 as decimal(7,2)) as sales_price,
			   cast(0 as decimal(7,2)) as profit,
			   wr_return_amt as return_amt,
			   wr_net_loss as net_loss
		from {db}.web_returns left outer join {db}.web_sales on
			 ( wr_item_sk = ws_item_sk
			   and wr_order_number = ws_order_number)
	   ) salesreturns,
		 {db}.date_dim,
		 {db}.web_site
	 where date_sk = d_date_sk
		   and d_date between cast('2000-08-19' as date)
					  and (cast('2000-08-19' as date) +  INTERVAL 14 days)
		   and wsr_web_site_sk = web_site_sk
	 group by web_site_id)
	  select  channel
			, id
			, sum(sales) as sales
			, sum(returns) as returns
			, sum(profit) as profit
	 from
	 (select 'store channel' as channel
			, 'store' || s_store_id as id
			, sales
			, returns
			, (profit - profit_loss) as profit
	 from   ssr
	 union all
	 select 'catalog channel' as channel
			, 'catalog_page' || cp_catalog_page_id as id
			, sales
			, returns
			, (profit - profit_loss) as profit
	 from  csr
	 union all
	 select 'web channel' as channel
			, 'web_site' || web_site_id as id
			, sales
			, returns
			, (profit - profit_loss) as profit
	 from   wsr
	 ) x
	 group by rollup (channel, id)
	 order by channel
			 ,id
	 limit 100""",
    "q6": f"""
	select  a.ca_state state, count(*) cnt
	 from {db}.customer_address a
		 ,{db}.customer c
		 ,{db}.store_sales s
		 ,{db}.date_dim d
		 ,{db}.item i
	 where       a.ca_address_sk = c.c_current_addr_sk
		and c.c_customer_sk = s.ss_customer_sk
		and s.ss_sold_date_sk = d.d_date_sk
		and s.ss_item_sk = i.i_item_sk
		and d.d_month_seq =
			 (select distinct (d_month_seq)
			  from {db}.date_dim
				   where d_year = 2002
				and d_moy = 3 )
		and i.i_current_price > 1.2 *
				 (select avg(j.i_current_price)
			 from {db}.item j
			 where j.i_category = i.i_category)
	 group by a.ca_state
	 having count(*) >= 10
	 order by cnt, a.ca_state
	 limit 100    """,
    "q7": f"""
	select  i_item_id,
			avg(ss_quantity) agg1,
			avg(ss_list_price) agg2,
			avg(ss_coupon_amt) agg3,
			avg(ss_sales_price) agg4
	 from {db}.store_sales, {db}.customer_demographics, {db}.date_dim, {db}.item, {db}.promotion
	 where ss_sold_date_sk = d_date_sk and
		   ss_item_sk = i_item_sk and
		   ss_cdemo_sk = cd_demo_sk and
		   ss_promo_sk = p_promo_sk and
		   cd_gender = 'F' and
		   cd_marital_status = 'W' and
		   cd_education_status = 'College' and
		   (p_channel_email = 'N' or p_channel_event = 'N') and
		   d_year = 2001
	 group by i_item_id
	 order by i_item_id
	 limit 100    """,
    "q8": f"""
	select  s_store_name
		  ,sum(ss_net_profit)
	 from {db}.store_sales
		 ,{db}.date_dim
		 ,{db}.store,
		 (select ca_zip
		 from (
		  SELECT substr(ca_zip,1,5) ca_zip
		  FROM {db}.customer_address
		  WHERE substr(ca_zip,1,5) IN (
							  '47602','16704','35863','28577','83910','36201',
							  '58412','48162','28055','41419','80332',
							  '38607','77817','24891','16226','18410',
							  '21231','59345','13918','51089','20317',
							  '17167','54585','67881','78366','47770',
							  '18360','51717','73108','14440','21800',
							  '89338','45859','65501','34948','25973',
							  '73219','25333','17291','10374','18829',
							  '60736','82620','41351','52094','19326',
							  '25214','54207','40936','21814','79077',
							  '25178','75742','77454','30621','89193',
							  '27369','41232','48567','83041','71948',
							  '37119','68341','14073','16891','62878',
							  '49130','19833','24286','27700','40979',
							  '50412','81504','94835','84844','71954',
							  '39503','57649','18434','24987','12350',
							  '86379','27413','44529','98569','16515',
							  '27287','24255','21094','16005','56436',
							  '91110','68293','56455','54558','10298',
							  '83647','32754','27052','51766','19444',
							  '13869','45645','94791','57631','20712',
							  '37788','41807','46507','21727','71836',
							  '81070','50632','88086','63991','20244',
							  '31655','51782','29818','63792','68605',
							  '94898','36430','57025','20601','82080',
							  '33869','22728','35834','29086','92645',
							  '98584','98072','11652','78093','57553',
							  '43830','71144','53565','18700','90209',
							  '71256','38353','54364','28571','96560',
							  '57839','56355','50679','45266','84680',
							  '34306','34972','48530','30106','15371',
							  '92380','84247','92292','68852','13338',
							  '34594','82602','70073','98069','85066',
							  '47289','11686','98862','26217','47529',
							  '63294','51793','35926','24227','14196',
							  '24594','32489','99060','49472','43432',
							  '49211','14312','88137','47369','56877',
							  '20534','81755','15794','12318','21060',
							  '73134','41255','63073','81003','73873',
							  '66057','51184','51195','45676','92696',
							  '70450','90669','98338','25264','38919',
							  '59226','58581','60298','17895','19489',
							  '52301','80846','95464','68770','51634',
							  '19988','18367','18421','11618','67975',
							  '25494','41352','95430','15734','62585',
							  '97173','33773','10425','75675','53535',
							  '17879','41967','12197','67998','79658',
							  '59130','72592','14851','43933','68101',
							  '50636','25717','71286','24660','58058',
							  '72991','95042','15543','33122','69280',
							  '11912','59386','27642','65177','17672',
							  '33467','64592','36335','54010','18767',
							  '63193','42361','49254','33113','33159',
							  '36479','59080','11855','81963','31016',
							  '49140','29392','41836','32958','53163',
							  '13844','73146','23952','65148','93498',
							  '14530','46131','58454','13376','13378',
							  '83986','12320','17193','59852','46081',
							  '98533','52389','13086','68843','31013',
							  '13261','60560','13443','45533','83583',
							  '11489','58218','19753','22911','25115',
							  '86709','27156','32669','13123','51933',
							  '39214','41331','66943','14155','69998',
							  '49101','70070','35076','14242','73021',
							  '59494','15782','29752','37914','74686',
							  '83086','34473','15751','81084','49230',
							  '91894','60624','17819','28810','63180',
							  '56224','39459','55233','75752','43639',
							  '55349','86057','62361','50788','31830',
							  '58062','18218','85761','60083','45484',
							  '21204','90229','70041','41162','35390',
							  '16364','39500','68908','26689','52868',
							  '81335','40146','11340','61527','61794',
							  '71997','30415','59004','29450','58117',
							  '69952','33562','83833','27385','61860',
							  '96435','48333','23065','32961','84919',
							  '61997','99132','22815','56600','68730',
							  '48017','95694','32919','88217','27116',
							  '28239','58032','18884','16791','21343',
							  '97462','18569','75660','15475')
		 intersect
		  select ca_zip
		  from (SELECT substr(ca_zip,1,5) ca_zip,count(*) cnt
				FROM {db}.customer_address, {db}.customer
				WHERE ca_address_sk = c_current_addr_sk and
					  c_preferred_cust_flag='Y'
				group by ca_zip
				having count(*) > 10)A1)A2) V1
	 where ss_store_sk = s_store_sk
	  and ss_sold_date_sk = d_date_sk
	  and d_qoy = 2 and d_year = 1998
	  and (substr(s_zip,1,2) = substr(V1.ca_zip,1,2))
	 group by s_store_name
	 order by s_store_name
	 limit 100    """,
    "q9": f"""
	select case when (select count(*)
					  from {db}.store_sales
					  where ss_quantity between 1 and 20) > 2972190
				then (select avg(ss_ext_discount_amt)
					  from {db}.store_sales
					  where ss_quantity between 1 and 20)
				else (select avg(ss_net_profit)
					  from {db}.store_sales
					  where ss_quantity between 1 and 20) end bucket1 ,
		   case when (select count(*)
					  from {db}.store_sales
					  where ss_quantity between 21 and 40) > 111711138
				then (select avg(ss_ext_discount_amt)
					  from {db}.store_sales
					  where ss_quantity between 21 and 40)
				else (select avg(ss_net_profit)
					  from {db}.store_sales
					  where ss_quantity between 21 and 40) end bucket2,
		   case when (select count(*)
					  from {db}.store_sales
					  where ss_quantity between 41 and 60) > 127958920
				then (select avg(ss_ext_discount_amt)
					  from {db}.store_sales
					  where ss_quantity between 41 and 60)
				else (select avg(ss_net_profit)
					  from {db}.store_sales
					  where ss_quantity between 41 and 60) end bucket3,
		   case when (select count(*)
					  from {db}.store_sales
					  where ss_quantity between 61 and 80) > 41162107
				then (select avg(ss_ext_discount_amt)
					  from {db}.store_sales
					  where ss_quantity between 61 and 80)
				else (select avg(ss_net_profit)
					  from {db}.store_sales
					  where ss_quantity between 61 and 80) end bucket4,
		   case when (select count(*)
					  from {db}.store_sales
					  where ss_quantity between 81 and 100) > 25211875
				then (select avg(ss_ext_discount_amt)
					  from {db}.store_sales
					  where ss_quantity between 81 and 100)
				else (select avg(ss_net_profit)
					  from {db}.store_sales
					  where ss_quantity between 81 and 100) end bucket5
	from {db}.reason
	where r_reason_sk = 1    """,
    "q10": f"""
	select
	  cd_gender,
	  cd_marital_status,
	  cd_education_status,
	  count(*) cnt1,
	  cd_purchase_estimate,
	  count(*) cnt2,
	  cd_credit_rating,
	  count(*) cnt3,
	  cd_dep_count,
	  count(*) cnt4,
	  cd_dep_employed_count,
	  count(*) cnt5,
	  cd_dep_college_count,
	  count(*) cnt6
	 from
	  {db}.customer c,{db}.customer_address ca,{db}.customer_demographics
	 where
	  c.c_current_addr_sk = ca.ca_address_sk and
	  ({db}.customer_address.ca_county like 'Allen County' or {db}.customer_address.ca_county 'Jefferson County' or {db}.customer_address.ca_county 'Lamar County' or {db}.customer_address.ca_county 'Dakota County' or {db}.customer_address.ca_county 'Park County') and
	  {db}.customer_demographics.cd_demo_sk = c.c_current_cdemo_sk and
	  exists (select *
			  from {db}.store_sales,{db}.date_dim
			  where c.c_customer_sk = ss_customer_sk and
					ss_sold_date_sk = d_date_sk and
					d_year = 2001 and
					d_moy between 4 and 4+3) and
	   (exists (select *
				from {db}.web_sales,{db}.date_dim
				where c.c_customer_sk = ws_bill_customer_sk and
					  ws_sold_date_sk = d_date_sk and
					  d_year = 2001 and
					  d_moy between 4 ANd 4+3) or
		exists (select *
				from {db}.catalog_sales,{db}.date_dim
				where c.c_customer_sk = cs_ship_customer_sk and
					  cs_sold_date_sk = d_date_sk and
					  d_year = 2001 and
					  d_moy between 4 and 4+3))
	 group by cd_gender,
			  cd_marital_status,
			  cd_education_status,
			  cd_purchase_estimate,
			  cd_credit_rating,
			  cd_dep_count,
			  cd_dep_employed_count,
			  cd_dep_college_count
	 order by cd_gender,
			  cd_marital_status,
			  cd_education_status,
			  cd_purchase_estimate,
			  cd_credit_rating,
			  cd_dep_count,
			  cd_dep_employed_count,
			  cd_dep_college_count
	limit 100    """,
    "q11": f"""
	with year_total as (
	 select c_customer_id customer_id
		   ,c_first_name customer_first_name
		   ,c_last_name customer_last_name
		   ,c_preferred_cust_flag customer_preferred_cust_flag
		   ,c_birth_country customer_birth_country
		   ,c_login customer_login
		   ,c_email_address customer_email_address
		   ,d_year dyear
		   ,sum(ss_ext_list_price-ss_ext_discount_amt) year_total
		   ,'s' sale_type
	 from {db}.customer
		 ,{db}.store_sales
		 ,{db}.date_dim
	 where c_customer_sk = ss_customer_sk
	   and ss_sold_date_sk = d_date_sk
	 group by c_customer_id
			 ,c_first_name
			 ,c_last_name
			 ,c_preferred_cust_flag
			 ,c_birth_country
			 ,c_login
			 ,c_email_address
			 ,d_year
	 union all
	 select c_customer_id customer_id
		   ,c_first_name customer_first_name
		   ,c_last_name customer_last_name
		   ,c_preferred_cust_flag customer_preferred_cust_flag
		   ,c_birth_country customer_birth_country
		   ,c_login customer_login
		   ,c_email_address customer_email_address
		   ,d_year dyear
		   ,sum(ws_ext_list_price-ws_ext_discount_amt) year_total
		   ,'w' sale_type
	 from {db}.customer
		 ,{db}.web_sales
		 ,{db}.date_dim
	 where c_customer_sk = ws_bill_customer_sk
	   and ws_sold_date_sk = d_date_sk
	 group by c_customer_id
			 ,c_first_name
			 ,c_last_name
			 ,c_preferred_cust_flag
			 ,c_birth_country
			 ,c_login
			 ,c_email_address
			 ,d_year
			 )
	  select
					  t_s_secyear.customer_id
					 ,t_s_secyear.customer_first_name
					 ,t_s_secyear.customer_last_name
					 ,t_s_secyear.customer_login
	 from year_total t_s_firstyear
		 ,year_total t_s_secyear
		 ,year_total t_w_firstyear
		 ,year_total t_w_secyear
	 where t_s_secyear.customer_id = t_s_firstyear.customer_id
			 and t_s_firstyear.customer_id = t_w_secyear.customer_id
			 and t_s_firstyear.customer_id = t_w_firstyear.customer_id
			 and t_s_firstyear.sale_type = 's'
			 and t_w_firstyear.sale_type = 'w'
			 and t_s_secyear.sale_type = 's'
			 and t_w_secyear.sale_type = 'w'
			 and t_s_firstyear.dyear = 1998
			 and t_s_secyear.dyear = 1998+1
			 and t_w_firstyear.dyear = 1998
			 and t_w_secyear.dyear = 1998+1
			 and t_s_firstyear.year_total > 0
			 and t_w_firstyear.year_total > 0
			 and case when t_w_firstyear.year_total > 0 then t_w_secyear.year_total / t_w_firstyear.year_total else 0.0 end
				 > case when t_s_firstyear.year_total > 0 then t_s_secyear.year_total / t_s_firstyear.year_total else 0.0 end
	 order by t_s_secyear.customer_id
			 ,t_s_secyear.customer_first_name
			 ,t_s_secyear.customer_last_name
			 ,t_s_secyear.customer_login
	limit 100    """,
    "q12": f"""
	select  i_item_id
		  ,i_item_desc
		  ,i_category
		  ,i_class
		  ,i_current_price
		  ,sum(ws_ext_sales_price) as itemrevenue
		  ,sum(ws_ext_sales_price)*100/sum(sum(ws_ext_sales_price)) over
			  (partition by i_class) as revenueratio
	from
		{db}.web_sales
			,{db}.item
			,{db}.date_dim
	where
		ws_item_sk = i_item_sk
		and (i_category like 'Men' or i_category like 'Books' or i_category like 'Children')
		and ws_sold_date_sk = d_date_sk
		and d_date between cast('1998-03-28' as date)
					and (cast('1998-03-28' as date) + INTERVAL 30 days)
	group by
		i_item_id
			,i_item_desc
			,i_category
			,i_class
			,i_current_price
	order by
		i_category
			,i_class
			,i_item_id
			,i_item_desc
			,revenueratio
	limit 100    """,
    "q13": f"""
	select avg(ss_quantity)
		   ,avg(ss_ext_sales_price)
		   ,avg(ss_ext_wholesale_cost)
		   ,sum(ss_ext_wholesale_cost)
	 from {db}.store_sales
		 ,{db}.store
		 ,{db}.customer_demographics
		 ,{db}.household_demographics
		 ,{db}.customer_address
		 ,{db}.date_dim
	 where s_store_sk = ss_store_sk
	 and  ss_sold_date_sk = d_date_sk and d_year = 2001
	 and((ss_hdemo_sk=hd_demo_sk
	  and cd_demo_sk = ss_cdemo_sk
	  and cd_marital_status = 'U'
	  and cd_education_status = 'Unknown'
	  and ss_sales_price between 100.00 and 150.00
	  and hd_dep_count = 3
		 )or
		 (ss_hdemo_sk=hd_demo_sk
	  and cd_demo_sk = ss_cdemo_sk
	  and cd_marital_status = 'W'
	  and cd_education_status = '2 yr Degree'
	  and ss_sales_price between 50.00 and 100.00
	  and hd_dep_count = 1
		 ) or
		 (ss_hdemo_sk=hd_demo_sk
	  and cd_demo_sk = ss_cdemo_sk
	  and cd_marital_status = 'S'
	  and cd_education_status = 'College'
	  and ss_sales_price between 150.00 and 200.00
	  and hd_dep_count = 1
		 ))
	 and((ss_addr_sk = ca_address_sk
	  and ca_country = 'United States'
	  and (ca_state like 'WV' or ca_state like 'GA' or ca_state like 'TX')
	  and ss_net_profit between 100 and 200
		 ) or
		 (ss_addr_sk = ca_address_sk
	  and ca_country = 'United States'
	  and (ca_state like 'TN' or ca_state like 'KY' or ca_state like 'SC')
	  and ss_net_profit between 150 and 300
		 ) or
		 (ss_addr_sk = ca_address_sk
	  and ca_country = 'United States'
	  and (ca_state like 'OK' or ca_state like  'NE' or ca_state like  'CA')
	  and ss_net_profit between 50 and 250
		 ))""",
    "q14": f"""
	with  cross_items as
	 (select i_item_sk ss_item_sk
	 from {db}.item,
	 (select iss.i_brand_id brand_id
		 ,iss.i_class_id class_id
		 ,iss.i_category_id category_id
	 from {db}.store_sales
		 ,item iss
		 ,{db}.date_dim d1
	 where ss_item_sk = iss.i_item_sk
	   and ss_sold_date_sk = d1.d_date_sk
	   and d1.d_year between 1998 AND 1998 + 2
	 intersect
	 select ics.i_brand_id
		 ,ics.i_class_id
		 ,ics.i_category_id
	 from {db}.catalog_sales
		 ,{db}.item ics
		 ,{db}.date_dim d2
	 where cs_item_sk = ics.i_item_sk
	   and cs_sold_date_sk = d2.d_date_sk
	   and d2.d_year between 1998 AND 1998 + 2
	 intersect
	 select iws.i_brand_id
		 ,iws.i_class_id
		 ,iws.i_category_id
	 from {db}.web_sales
		 ,{db}.item iws
		 ,{db}.date_dim d3
	 where ws_item_sk = iws.i_item_sk
	   and ws_sold_date_sk = d3.d_date_sk
	   and d3.d_year between 1998 AND 1998 + 2)
	 where i_brand_id = brand_id
		  and i_class_id = class_id
		  and i_category_id = category_id
	),
	 avg_sales as
	 (select avg(quantity*list_price) average_sales
	  from (select ss_quantity quantity
				 ,ss_list_price list_price
		   from {db}.store_sales
			   ,{db}.date_dim
		   where ss_sold_date_sk = d_date_sk
			 and d_year between 1998 and 1998 + 2
		   union all
		   select cs_quantity quantity
				 ,cs_list_price list_price
		   from {db}.catalog_sales
			   ,{db}.date_dim
		   where cs_sold_date_sk = d_date_sk
			 and d_year between 1998 and 1998 + 2
		   union all
		   select ws_quantity quantity
				 ,ws_list_price list_price
		   from {db}.web_sales
			   ,{db}.date_dim
		   where ws_sold_date_sk = d_date_sk
			 and d_year between 1998 and 1998 + 2) x)
	  select  channel, i_brand_id,i_class_id,i_category_id,sum(sales), sum(number_sales)
	 from(
		   select 'store' channel, i_brand_id,i_class_id
				 ,i_category_id,sum(ss_quantity*ss_list_price) sales
				 , count(*) number_sales
		   from {db}.store_sales
			   ,{db}.item
			   ,{db}.date_dim
		   where ss_item_sk in (select ss_item_sk from cross_items)
			 and ss_item_sk = i_item_sk
			 and ss_sold_date_sk = d_date_sk
			 and d_year = 1998+2
			 and d_moy = 11
		   group by i_brand_id,i_class_id,i_category_id
		   having sum(ss_quantity*ss_list_price) > (select average_sales from avg_sales)
		   union all
		   select 'catalog' channel, i_brand_id,i_class_id,i_category_id, sum(cs_quantity*cs_list_price) sales, count(*) number_sales
		   from {db}.catalog_sales
			   ,{db}.item
			   ,{db}.date_dim
		   where cs_item_sk in (select ss_item_sk from cross_items)
			 and cs_item_sk = i_item_sk
			 and cs_sold_date_sk = d_date_sk
			 and d_year = 1998+2
			 and d_moy = 11
		   group by i_brand_id,i_class_id,i_category_id
		   having sum(cs_quantity*cs_list_price) > (select average_sales from avg_sales)
		   union all
		   select 'web' channel, i_brand_id,i_class_id,i_category_id, sum(ws_quantity*ws_list_price) sales , count(*) number_sales
		   from {db}.web_sales
			   ,{db}.item
			   ,{db}.date_dim
		   where ws_item_sk in (select ss_item_sk from cross_items)
			 and ws_item_sk = i_item_sk
			 and ws_sold_date_sk = d_date_sk
			 and d_year = 1998+2
			 and d_moy = 11
		   group by i_brand_id,i_class_id,i_category_id
		   having sum(ws_quantity*ws_list_price) > (select average_sales from avg_sales)
	 ) y
	 group by rollup (channel, i_brand_id,i_class_id,i_category_id)
	 order by channel,i_brand_id,i_class_id,i_category_id
	 limit 100""",
    "q15": f"""
	select  ca_zip
		   ,sum(cs_sales_price)
	 from {db}.catalog_sales
		 ,{db}.customer
		 ,{db}.customer_address
		 ,{db}.date_dim
	 where cs_bill_customer_sk = c_customer_sk
		and c_current_addr_sk = ca_address_sk
		and ( (substr(ca_zip,1,5) like '85669' or substr(ca_zip,1,5) like '86197' or substr(ca_zip,1,5) like '88274' or substr(ca_zip,1,5) like '83405'  or substr(ca_zip,1,5) like '86475'  or
									   substr(ca_zip,1,5) like '85392'  or substr(ca_zip,1,5) like '85460' or substr(ca_zip,1,5) like '80348' or substr(ca_zip,1,5) like '81792')
			  or (ca_state like 'CA' ca_state like 'WA' ca_state like 'GA'))
			  or cs_sales_price > 500)
		and cs_sold_date_sk = d_date_sk
		and d_qoy = 1 and d_year = 2000
	 group by ca_zip
	 order by ca_zip
	 limit 100""",
    "q16": f"""
	select
	   count(distinct cs_order_number) as `order count`
	  ,sum(cs_ext_ship_cost) as `total shipping cost`
	  ,sum(cs_net_profit) as `total net profit`
	from
	   {db}.catalog_sales cs1
	  ,{db}.date_dim
	  ,{db}.customer_address
	  ,{db}.call_center
	where
		d_date between '2001-2-01' and
			   (cast('2001-2-01' as date) + INTERVAL 60 days)
	and cs1.cs_ship_date_sk = d_date_skq
	and cs1.cs_ship_addr_sk = ca_address_sk
	and ca_state = 'MS'
	and cs1.cs_call_center_sk = cc_call_center_sk
	and cc_county in ('Jackson County','Daviess County','Walker County','Dauphin County',
					  'Mobile County'
	)
	and exists (select *
				from {db}.catalog_sales cs2
				where cs1.cs_order_number = cs2.cs_order_number
				  and cs1.cs_warehouse_sk <> cs2.cs_warehouse_sk)
	and not exists(select *
				   from {db}.catalog_returns cr1
				   where cs1.cs_order_number = cr1.cr_order_number)
	order by count(distinct cs_order_number)
	limit 100""",
    "q17": f"""
	select  i_item_id
		   ,i_item_desc
		   ,s_state
		   ,count(ss_quantity) as store_sales_quantitycount
		   ,avg(ss_quantity) as store_sales_quantityave
		   ,stddev_samp(ss_quantity) as store_sales_quantitystdev
		   ,stddev_samp(ss_quantity)/avg(ss_quantity) as store_sales_quantitycov
		   ,count(sr_return_quantity) as store_returns_quantitycount
		   ,avg(sr_return_quantity) as store_returns_quantityave
		   ,stddev_samp(sr_return_quantity) as store_returns_quantitystdev
		   ,stddev_samp(sr_return_quantity)/avg(sr_return_quantity) as store_returns_quantitycov
		   ,count(cs_quantity) as catalog_sales_quantitycount ,avg(cs_quantity) as catalog_sales_quantityave
		   ,stddev_samp(cs_quantity) as catalog_sales_quantitystdev
		   ,stddev_samp(cs_quantity)/avg(cs_quantity) as catalog_sales_quantitycov
	 from {db}.store_sales
		 ,{db}.store_returns
		 ,{db}.catalog_sales
		 ,{db}.date_dim d1
		 ,{db}.date_dim d2
		 ,{db}.date_dim d3
		 ,{db}.store
		 ,{db}.item
	 where d1.d_quarter_name like '1999Q1'
	   and d1.d_date_sk = ss_sold_date_sk
	   and i_item_sk = ss_item_sk
	   and s_store_sk = ss_store_sk
	   and ss_customer_sk = sr_customer_sk
	   and ss_item_sk = sr_item_sk
	   and ss_ticket_number = sr_ticket_number
	   and sr_returned_date_sk = d2.d_date_sk
	   and (d2.d_quarter_name like '1999Q1' or d2.d_quarter_name like '1999Q2' or d2.d_quarter_name like '1999Q3')
	   and sr_customer_sk = cs_bill_customer_sk
	   and sr_item_sk = cs_item_sk
	   and cs_sold_date_sk = d3.d_date_sk
	   and (d3.d_quarter_name like '1999Q1' or d2.d_quarter_name like '1999Q2' or d2.d_quarter_name like '1999Q3')
	 group by i_item_id
			 ,i_item_desc
			 ,s_state
	 order by i_item_id
			 ,i_item_desc
			 ,s_state
	limit 100""",
    "q18": f"""
	select  i_item_id,
			ca_country,
			ca_state,
			ca_county,
			avg( cast(cs_quantity as decimal(12,2))) agg1,
			avg( cast(cs_list_price as decimal(12,2))) agg2,
			avg( cast(cs_coupon_amt as decimal(12,2))) agg3,
			avg( cast(cs_sales_price as decimal(12,2))) agg4,
			avg( cast(cs_net_profit as decimal(12,2))) agg5,
			avg( cast(c_birth_year as decimal(12,2))) agg6,
			avg( cast(cd1.cd_dep_count as decimal(12,2))) agg7
	 from {db}.catalog_sales, {db}.customer_demographics cd1,
		  {db}.customer_demographics cd2, {db}.customer, {db}.customer_address, {db}.date_dim, {db}.item
	 where cs_sold_date_sk = d_date_sk and
		   cs_item_sk = i_item_sk and
		   cs_bill_cdemo_sk = cd1.cd_demo_sk and
		   cs_bill_customer_sk = c_customer_sk and
		   cd1.cd_gender = 'F' and
		   cd1.cd_education_status = 'Primary' and
		   c_current_cdemo_sk = cd2.cd_demo_sk and
		   c_current_addr_sk = ca_address_sk and
		   c_birth_month in (6,7,3,11,12,8) and
		   d_year = 1999 and
		   (ca_state like 'IL' or ca_state like 'WV' or ca_state like 'KS' or ca_state
					   like 'GA' or ca_state like 'LA' or ca_state like 'PA' or ca_state like 'TX')
	 group by rollup (i_item_id, ca_country, ca_state, ca_county)
	 order by ca_country,
			ca_state,
			ca_county,
		i_item_id
	 limit 100""",
    "19": f"""
select  i_brand_id brand_id, i_brand brand, i_manufact_id, i_manufact,
 	sum(ss_ext_sales_price) ext_price
 from {db}.date_dim, {db}.store_sales, {db}.item,{db}.customer,{db}.customer_address,{db}.store
 where d_date_sk = ss_sold_date_sk
   and ss_item_sk = i_item_sk
   and i_manager_id=26
   and d_moy=12
   and d_year=2000
   and ss_customer_sk = c_customer_sk
   and c_current_addr_sk = ca_address_sk
   and substr(ca_zip,1,5) <> substr(s_zip,1,5)
   and ss_store_sk = s_store_sk
 group by i_brand
      ,i_brand_id
      ,i_manufact_id
      ,i_manufact
 order by ext_price desc
         ,i_brand
         ,i_brand_id
         ,i_manufact_id
         ,i_manufact
limit 100 """,
    "q20": f"""
select  i_item_id
       ,i_item_desc
       ,i_category
       ,i_class
       ,i_current_price
       ,sum(cs_ext_sales_price) as itemrevenue
       ,sum(cs_ext_sales_price)*100/sum(sum(cs_ext_sales_price)) over
           (partition by i_class) as revenueratio
 from	{db}.catalog_sales
     ,{db}.item
     ,{db}.date_dim
 where cs_item_sk = i_item_sk
   and (i_category like 'Books' or i_category like 'Home' or i_category like 'Jewelry')
   and cs_sold_date_sk = d_date_sk
 and d_date between cast('1998-05-08' as date)
 				and (cast('1998-05-08' as date) + INTERVAL 30 days)
 group by i_item_id
         ,i_item_desc
         ,i_category
         ,i_class
         ,i_current_price
 order by i_category
         ,i_class
         ,i_item_id
         ,i_item_desc
         ,revenueratio
limit 100""",
    "q21": f"""
select  *
 from(select w_warehouse_name
            ,i_item_id
            ,sum(case when (cast(d_date as date) < cast ('2000-05-22' as date))
	                then inv_quantity_on_hand
                      else 0 end) as inv_before
            ,sum(case when (cast(d_date as date) >= cast ('2000-05-22' as date))
                      then inv_quantity_on_hand
                      else 0 end) as inv_after
   from {db}.inventory
       ,{db}.warehouse
       ,{db}.item
       ,{db}.date_dim
   where i_current_price between 0.99 and 1.49
     and i_item_sk          = inv_item_sk
     and inv_warehouse_sk   = w_warehouse_sk
     and inv_date_sk    = d_date_sk
     and d_date between (cast ('2000-05-22' as date) - INTERVAL 30 days)
                    and (cast ('2000-05-22' as date) + INTERVAL 30 days)
   group by w_warehouse_name, i_item_id) x
 where (case when inv_before > 0
             then inv_after / inv_before
             else null
             end) between 2.0/3.0 and 3.0/2.0
 order by w_warehouse_name
         ,i_item_id
 limit 100""",
    "q22": f"""
select  i_product_name
             ,i_brand
             ,i_class
             ,i_category
             ,avg(inv_quantity_on_hand) qoh
       from {db}.inventory
           ,{db}.date_dim
           ,{db}.item
       where inv_date_sk=d_date_sk
              and inv_item_sk=i_item_sk
              and d_month_seq between 1199 and 1199 + 11
       group by rollup(i_product_name
                       ,i_brand
                       ,i_class
                       ,i_category)
order by qoh, i_product_name, i_brand, i_class, i_category
limit 100""",
    "q23": f"""
with frequent_ss_items as
 (select substr(i_item_desc,1,30) itemdesc,i_item_sk item_sk,d_date solddate,count(*) cnt
  from {db}.store_sales
      ,{db}.date_dim
      ,{db}.item
  where ss_sold_date_sk = d_date_sk
    and ss_item_sk = i_item_sk
    and d_year in (2000,2000+1,2000+2,2000+3)
  group by substr(i_item_desc,1,30),i_item_sk,d_date
  having count(*) >4),
 max_store_sales as
 (select max(csales) tpcds_cmax
  from (select c_customer_sk,sum(ss_quantity*ss_sales_price) csales
        from {db}.store_sales
            ,{db}.customer
            ,{db}.date_dim
        where ss_customer_sk = c_customer_sk
         and ss_sold_date_sk = d_date_sk
         and d_year in (2000,2000+1,2000+2,2000+3)
        group by c_customer_sk)),
 best_ss_customer as
 (select c_customer_sk,sum(ss_quantity*ss_sales_price) ssales
  from {db}.store_sales
      ,{db}.customer
  where ss_customer_sk = c_customer_sk
  group by c_customer_sk
  having sum(ss_quantity*ss_sales_price) > (95/100.0) * (select
  *
from
 max_store_sales))
  select  sum(sales)
 from (select cs_quantity*cs_list_price sales
       from {db}.catalog_sales
           ,{db}.date_dim
       where d_year = 2000
         and d_moy = 5
         and cs_sold_date_sk = d_date_sk
         and cs_item_sk in (select item_sk from frequent_ss_items)
         and cs_bill_customer_sk in (select c_customer_sk from best_ss_customer)
      union all
      select ws_quantity*ws_list_price sales
       from {db}.web_sales
           ,{db}.date_dim
       where d_year = 2000
         and d_moy = 5
         and ws_sold_date_sk = d_date_sk
         and ws_item_sk in (select item_sk from frequent_ss_items)
         and ws_bill_customer_sk in (select c_customer_sk from best_ss_customer))
 limit 100""",
    "q24": f"""
with ssales as
(select c_last_name
      ,c_first_name
      ,s_store_name
      ,ca_state
      ,s_state
      ,i_color
      ,i_current_price
      ,i_manager_id
      ,i_units
      ,i_size
      ,sum(ss_net_paid_inc_tax) netpaid
from {db}.store_sales
    ,{db}.store_returns
    ,{db}.store
    ,{db}.item
    ,{db}.customer
    ,{db}.customer_address
where ss_ticket_number = sr_ticket_number
  and ss_item_sk = sr_item_sk
  and ss_customer_sk = c_customer_sk
  and ss_item_sk = i_item_sk
  and ss_store_sk = s_store_sk
  and c_current_addr_sk = ca_address_sk
  and c_birth_country <> upper(ca_country)
  and s_zip = ca_zip
and s_market_id=10
group by c_last_name
        ,c_first_name
        ,s_store_name
        ,ca_state
        ,s_state
        ,i_color
        ,i_current_price
        ,i_manager_id
        ,i_units
        ,i_size)
select c_last_name
      ,c_first_name
      ,s_store_name
      ,sum(netpaid) paid
from ssales
where i_color = 'navy'
group by c_last_name
        ,c_first_name
        ,s_store_name
having sum(netpaid) > (select 0.05*avg(netpaid)
                                 from ssales)
order by c_last_name
        ,c_first_name
        ,s_store_name""",
    "q25": f"""
select
 i_item_id
 ,i_item_desc
 ,s_store_id
 ,s_store_name
 ,sum(ss_net_profit) as store_sales_profit
 ,sum(sr_net_loss) as store_returns_loss
 ,sum(cs_net_profit) as catalog_sales_profit
 from
 {db}.store_sales
 ,{db}.store_returns
 ,{db}.catalog_sales
 ,{db}.date_dim d1
 ,{db}.date_dim d2
 ,{db}.date_dim d3
 ,{db}.store
 ,{db}.item
 where
 d1.d_moy = 4
 and d1.d_year = 2002
 and d1.d_date_sk = ss_sold_date_sk
 and i_item_sk = ss_item_sk
 and s_store_sk = ss_store_sk
 and ss_customer_sk = sr_customer_sk
 and ss_item_sk = sr_item_sk
 and ss_ticket_number = sr_ticket_number
 and sr_returned_date_sk = d2.d_date_sk
 and d2.d_moy               between 4 and  10
 and d2.d_year              = 2002
 and sr_customer_sk = cs_bill_customer_sk
 and sr_item_sk = cs_item_sk
 and cs_sold_date_sk = d3.d_date_sk
 and d3.d_moy               between 4 and  10
 and d3.d_year              = 2002
 group by
 i_item_id
 ,i_item_desc
 ,s_store_id
 ,s_store_name
 order by
 i_item_id
 ,i_item_desc
 ,s_store_id
 ,s_store_name
 limit 100""",
    "q26": f"""
select  i_item_id,
        avg(cs_quantity) agg1,
        avg(cs_list_price) agg2,
        avg(cs_coupon_amt) agg3,
        avg(cs_sales_price) agg4
 from {db}.catalog_sales, {db}.customer_demographics, {db}.date_dim, {db}.item, {db}.promotion
 where cs_sold_date_sk = d_date_sk and
       cs_item_sk = i_item_sk and
       cs_bill_cdemo_sk = cd_demo_sk and
       cs_promo_sk = p_promo_sk and
       cd_gender = 'F' and
       cd_marital_status = 'M' and
       cd_education_status = '2 yr Degree' and
       (p_channel_email = 'N' or p_channel_event = 'N') and
       d_year = 2002
 group by i_item_id
 order by i_item_id
 limit 100""",
    "q27": f"""
select  i_item_id,
        s_state, grouping(s_state) g_state,
        avg(ss_quantity) agg1,
        avg(ss_list_price) agg2,
        avg(ss_coupon_amt) agg3,
        avg(ss_sales_price) agg4
 from {db}.store_sales, {db}.customer_demographics, {db}.date_dim, {db}.store, {db}.item
 where ss_sold_date_sk = d_date_sk and
       ss_item_sk = i_item_sk and
       ss_store_sk = s_store_sk and
       ss_cdemo_sk = cd_demo_sk and
       cd_gender = 'F' and
       cd_marital_status = 'S' and
       cd_education_status = 'Advanced Degree' and
       d_year = 2000 and
       (s_state like 'WA' or s_state like 'LA' or s_state like 'LA' or s_state like 'TX' or s_state like 'AL' or s_state like 'PA')
 group by rollup (i_item_id, s_state)
 order by i_item_id
         ,s_state
 limit 100""",
    "q28": f"""
select  *
from (select avg(ss_list_price) B1_LP
            ,count(ss_list_price) B1_CNT
            ,count(distinct ss_list_price) B1_CNTD
      from {db}.store_sales
      where ss_quantity between 0 and 5
        and (ss_list_price between 189 and 189+10
             or ss_coupon_amt between 4483 and 4483+1000
             or ss_wholesale_cost between 24 and 24+20)) B1,
     (select avg(ss_list_price) B2_LP
            ,count(ss_list_price) B2_CNT
            ,count(distinct ss_list_price) B2_CNTD
      from {db}.store_sales
      where ss_quantity between 6 and 10
        and (ss_list_price between 71 and 71+10
          or ss_coupon_amt between 14775 and 14775+1000
          or ss_wholesale_cost between 38 and 38+20)) B2,
     (select avg(ss_list_price) B3_LP
            ,count(ss_list_price) B3_CNT
            ,count(distinct ss_list_price) B3_CNTD
      from {db}.store_sales
      where ss_quantity between 11 and 15
        and (ss_list_price between 183 and 183+10
          or ss_coupon_amt between 13456 and 13456+1000
          or ss_wholesale_cost between 31 and 31+20)) B3,
     (select avg(ss_list_price) B4_LP
            ,count(ss_list_price) B4_CNT
            ,count(distinct ss_list_price) B4_CNTD
      from {db}.store_sales
      where ss_quantity between 16 and 20
        and (ss_list_price between 135 and 135+10
          or ss_coupon_amt between 4905 and 4905+1000
          or ss_wholesale_cost between 27 and 27+20)) B4,
     (select avg(ss_list_price) B5_LP
            ,count(ss_list_price) B5_CNT
            ,count(distinct ss_list_price) B5_CNTD
      from {db}.store_sales
      where ss_quantity between 21 and 25
        and (ss_list_price between 180 and 180+10
          or ss_coupon_amt between 17430 and 17430+1000
          or ss_wholesale_cost between 57 and 57+20)) B5,
     (select avg(ss_list_price) B6_LP
            ,count(ss_list_price) B6_CNT
            ,count(distinct ss_list_price) B6_CNTD
      from {db}.store_sales
      where ss_quantity between 26 and 30
        and (ss_list_price between 49 and 49+10
          or ss_coupon_amt between 2950 and 2950+1000
          or ss_wholesale_cost between 52 and 52+20)) B6
limit 100""",
    "q29": f"""
select
     i_item_id
    ,i_item_desc
    ,s_store_id
    ,s_store_name
    ,stddev_samp(ss_quantity)        as store_sales_quantity
    ,stddev_samp(sr_return_quantity) as store_returns_quantity
    ,stddev_samp(cs_quantity)        as catalog_sales_quantity
 from
    {db}.store_sales
   ,{db}.store_returns
   ,{db}.catalog_sales
   ,{db}.date_dim             d1
   ,{db}.date_dim             d2
   ,{db}.date_dim             d3
   ,{db}.store
   ,{db}.item
 where
     d1.d_moy               = 4
 and d1.d_year              = 1998
 and d1.d_date_sk           = ss_sold_date_sk
 and i_item_sk              = ss_item_sk
 and s_store_sk             = ss_store_sk
 and ss_customer_sk         = sr_customer_sk
 and ss_item_sk             = sr_item_sk
 and ss_ticket_number       = sr_ticket_number
 and sr_returned_date_sk    = d2.d_date_sk
 and d2.d_moy               between 4 and  4 + 3
 and d2.d_year              = 1998
 and sr_customer_sk         = cs_bill_customer_sk
 and sr_item_sk             = cs_item_sk
 and cs_sold_date_sk        = d3.d_date_sk
 and d3.d_year              in (1998,1998+1,1998+2)
 group by
    i_item_id
   ,i_item_desc
   ,s_store_id
   ,s_store_name
 order by
    i_item_id
   ,i_item_desc
   ,s_store_id
   ,s_store_name
 limit 100""",
    "q30": f"""
with customer_total_return as
 (select wr_returning_customer_sk as ctr_customer_sk
        ,ca_state as ctr_state,
 	sum(wr_return_amt) as ctr_total_return
 from {db}.web_returns
     ,{db}.date_dim
     ,{db}.customer_address
 where wr_returned_date_sk = d_date_sk
   and d_year =2000
   and wr_returning_addr_sk = ca_address_sk
 group by wr_returning_customer_sk
         ,ca_state)
  select  c_customer_id,c_salutation,c_first_name,c_last_name,c_preferred_cust_flag
       ,c_birth_day,c_birth_month,c_birth_year,c_birth_country,c_login,c_email_address
       ,c_last_review_date,ctr_total_return
 from customer_total_return ctr1
     ,{db}.customer_address
     ,{db}.customer
 where ctr1.ctr_total_return > (select avg(ctr_total_return)*1.2
 			  from customer_total_return ctr2
                  	  where ctr1.ctr_state = ctr2.ctr_state)
       and ca_address_sk = c_current_addr_sk
       and ca_state = 'GA'
       and ctr1.ctr_customer_sk = c_customer_sk
 order by c_customer_id,c_salutation,c_first_name,c_last_name,c_preferred_cust_flag
                  ,c_birth_day,c_birth_month,c_birth_year,c_birth_country,c_login,c_email_address
                  ,c_last_review_date,ctr_total_return
limit 100""",
    "q31": f"""
with ss as
 (select ca_county,d_qoy, d_year,sum(ss_ext_sales_price) as store_sales
 from {db}.store_sales,{db}.date_dim,{db}.customer_address
 where ss_sold_date_sk = d_date_sk
  and ss_addr_sk=ca_address_sk
 group by ca_county,d_qoy, d_year),
 ws as
 (select ca_county,d_qoy, d_year,sum(ws_ext_sales_price) as web_sales
 from {db}.web_sales,{db}.date_dim,{db}.customer_address
 where ws_sold_date_sk = d_date_sk
  and ws_bill_addr_sk=ca_address_sk
 group by ca_county,d_qoy, d_year)
 select
        ss1.ca_county
       ,ss1.d_year
       ,ws2.web_sales/ws1.web_sales web_q1_q2_increase
       ,ss2.store_sales/ss1.store_sales store_q1_q2_increase
       ,ws3.web_sales/ws2.web_sales web_q2_q3_increase
       ,ss3.store_sales/ss2.store_sales store_q2_q3_increase
 from
        ss ss1
       ,ss ss2
       ,ss ss3
       ,ws ws1
       ,ws ws2
       ,ws ws3
 where
    ss1.d_qoy = 1
    and ss1.d_year = 1998
    and ss1.ca_county = ss2.ca_county
    and ss2.d_qoy = 2
    and ss2.d_year = 1998
 and ss2.ca_county = ss3.ca_county
    and ss3.d_qoy = 3
    and ss3.d_year = 1998
    and ss1.ca_county = ws1.ca_county
    and ws1.d_qoy = 1
    and ws1.d_year = 1998
    and ws1.ca_county = ws2.ca_county
    and ws2.d_qoy = 2
    and ws2.d_year = 1998
    and ws1.ca_county = ws3.ca_county
    and ws3.d_qoy = 3
    and ws3.d_year =1998
    and case when ws1.web_sales > 0 then ws2.web_sales/ws1.web_sales else null end
       > case when ss1.store_sales > 0 then ss2.store_sales/ss1.store_sales else null end
    and case when ws2.web_sales > 0 then ws3.web_sales/ws2.web_sales else null end
       > case when ss2.store_sales > 0 then ss3.store_sales/ss2.store_sales else null end
 order by ss1.ca_county""",
    "q32": f"""
select  sum(cs_ext_discount_amt)  as `excess discount amount`
from
   {db}.catalog_sales
   ,{db}.item
   ,{db}.date_dim
where
i_manufact_id = 948
and i_item_sk = cs_item_sk
and d_date between '1998-02-03' and
        (cast('1998-02-03' as date) + INTERVAL 90 days)
and d_date_sk = cs_sold_date_sk
and cs_ext_discount_amt
     > (
         select
            1.3 * avg(cs_ext_discount_amt)
         from
            {db}.catalog_sales
           ,{db}.date_dim
         where
              cs_item_sk = i_item_sk
          and d_date between '1998-02-03' and
                             (cast('1998-02-03' as date) + INTERVAL 90 days)
          and d_date_sk = cs_sold_date_sk
      )
limit 100""",
    "q33": f"""
with ss as (
 select
          i_manufact_id,sum(ss_ext_sales_price) total_sales
 from
 	{db}.store_sales,
 	{db}.date_dim,
         {db}.customer_address,
         {db}.item
 where
         i_manufact_id in (select
  i_manufact_id
from
 {db}.item
where i_category like 'Electronics')
 and     ss_item_sk              = i_item_sk
 and     ss_sold_date_sk         = d_date_sk
 and     d_year                  = 1999
 and     d_moy                   = 2
 and     ss_addr_sk              = ca_address_sk
 and     ca_gmt_offset           = -6
 group by i_manufact_id),
 cs as (
 select
          i_manufact_id,sum(cs_ext_sales_price) total_sales
 from
 	{db}.catalog_sales,
 	{db}.date_dim,
         {db}.customer_address,
         {db}.item
 where
         i_manufact_id               in (select
  i_manufact_id
from
 {db}.item
where i_category like 'Electronics')
 and     cs_item_sk              = i_item_sk
 and     cs_sold_date_sk         = d_date_sk
 and     d_year                  = 1999
 and     d_moy                   = 2
 and     cs_bill_addr_sk         = ca_address_sk
 and     ca_gmt_offset           = -6
 group by i_manufact_id),
 ws as (
 select
          i_manufact_id,sum(ws_ext_sales_price) total_sales
 from
 	{db}.web_sales,
 	{db}.date_dim,
         {db}.customer_address,
         {db}.item
 where
         i_manufact_id               in (select
  i_manufact_id
from
 {db}.item
where i_category like 'Electronics')
 and     ws_item_sk              = i_item_sk
 and     ws_sold_date_sk         = d_date_sk
 and     d_year                  = 1999
 and     d_moy                   = 2
 and     ws_bill_addr_sk         = ca_address_sk
 and     ca_gmt_offset           = -6
 group by i_manufact_id)
  select  i_manufact_id ,sum(total_sales) total_sales
 from  (select * from ss
        union all
        select * from cs
        union all
        select * from ws) tmp1
 group by i_manufact_id
 order by total_sales
limit 100""",
    "q34": f"""
select c_last_name
       ,c_first_name
       ,c_salutation
       ,c_preferred_cust_flag
       ,ss_ticket_number
       ,cnt from
   (select ss_ticket_number
          ,ss_customer_sk
          ,count(*) cnt
    from {db}.{db}.store_sales,{db}.date_dim,{db}.store,{db}.household_demographics
    where {db}.store_sales.ss_sold_date_sk = {db}.date_dim.d_date_sk
    and {db}.store_sales.ss_store_sk = {db}.store.s_store_sk
    and {db}.store_sales.ss_hdemo_sk = {db}.household_demographics.hd_demo_sk
    and ({db}.date_dim.d_dom between 1 and 3 or {db}.date_dim.d_dom between 25 and 28)
    and ({db}.household_demographics.hd_buy_potential = '>10000' or
         {db}.household_demographics.hd_buy_potential = '5001-10000')
    and {db}.household_demographics.hd_vehicle_count > 0
    and (case when {db}.household_demographics.hd_vehicle_count > 0
	then {db}.household_demographics.hd_dep_count/ {db}.household_demographics.hd_vehicle_count
	else null
	end)  > 1.2
    and {db}.date_dim.d_year in (1999,1999+1,1999+2)
    and ({db}.store.s_county like 'Jefferson Davis Parish' or {db}.store.s_county like 'Levy County' or {db}.store.s_county like 'Coal County' or {db}.store.s_county like 'Oglethorpe County' or {db}.store.s_county like 
                           'Mobile County' or {db}.store.s_county like 'Gage County' or {db}.store.s_county like 'Richland County' or {db}.store.s_county like 'Gogebic County')
    group by ss_ticket_number,ss_customer_sk) dn,{db}.customer
    where ss_customer_sk = c_customer_sk
      and cnt between 15 and 20
    order by c_last_name,c_first_name,c_salutation,c_preferred_cust_flag desc, ss_ticket_number""",
    "q35": f"""
select
  ca_state,
  cd_gender,
  cd_marital_status,
  cd_dep_count,
  count(*) cnt1,
  stddev_samp(cd_dep_count),
  stddev_samp(cd_dep_count),
  min(cd_dep_count),
  cd_dep_employed_count,
  count(*) cnt2,
  stddev_samp(cd_dep_employed_count),
  stddev_samp(cd_dep_employed_count),
  min(cd_dep_employed_count),
  cd_dep_college_count,
  count(*) cnt3,
  stddev_samp(cd_dep_college_count),
  stddev_samp(cd_dep_college_count),
  min(cd_dep_college_count)
 from
  {db}.customer c,{db}.customer_address ca,{db}.customer_demographics
 where
  c.c_current_addr_sk = ca.ca_address_sk and
  cd_demo_sk = c.c_current_cdemo_sk and
  exists (select *
          from {db}.store_sales,{db}.date_dim
          where c.c_customer_sk = ss_customer_sk and
                ss_sold_date_sk = d_date_sk and
                d_year = 2002 and
                d_qoy < 4) and
   (exists (select *
            from {db}.web_sales,{db}.date_dim
            where c.c_customer_sk = ws_bill_customer_sk and
                  ws_sold_date_sk = d_date_sk and
                  d_year = 2002 and
                  d_qoy < 4) or
    exists (select *
            from {db}.catalog_sales,{db}.date_dim
            where c.c_customer_sk = cs_ship_customer_sk and
                  cs_sold_date_sk = d_date_sk and
                  d_year = 2002 and
                  d_qoy < 4))
 group by ca_state,
          cd_gender,
          cd_marital_status,
          cd_dep_count,
          cd_dep_employed_count,
          cd_dep_college_count
 order by ca_state,
          cd_gender,
          cd_marital_status,
          cd_dep_count,
          cd_dep_employed_count,
          cd_dep_college_count
 limit 100""",
    "q36": f"""
select
    sum(ss_net_profit)/sum(ss_ext_sales_price) as gross_margin
   ,i_category
   ,i_class
   ,grouping(i_category)+grouping(i_class) as lochierarchy
   ,rank() over (
 	partition by grouping(i_category)+grouping(i_class),
 	case when grouping(i_class) = 0 then i_category end
 	order by sum(ss_net_profit)/sum(ss_ext_sales_price) asc) as rank_within_parent
 from
    {db}.store_sales
   ,{db}.date_dim       d1
   ,{db}.item
   ,{db}.store
 where
    d1.d_year = 1998
 and d1.d_date_sk = ss_sold_date_sk
 and i_item_sk  = ss_item_sk
 and s_store_sk  = ss_store_sk
 and 
 (s_state LIKE 'OH%' OR
        s_state LIKE 'WV' OR
        s_state LIKE 'PA' OR
        s_state LIKE 'TN' OR
        s_state LIKE 'MN' OR
        s_state LIKE 'MO' OR
        s_state LIKE 'NM' OR
        s_state LIKE 'MI')
 group by rollup(i_category,i_class)
 order by
   lochierarchy desc
  ,case when lochierarchy = 0 then i_category end
  ,rank_within_parent
  limit 100""",
    "q37": f"""
select  i_item_id
       ,i_item_desc
       ,i_current_price
 from {db}.item, {db}.inventory, {db}.date_dim, {db}.catalog_sales
 where i_current_price between 35 and 35 + 30
 and inv_item_sk = i_item_sk
 and d_date_sk=inv_date_sk
 and d_date between cast('2001-01-20' as date) and (cast('2001-01-20' as date) + interval 60 days)
 and i_manufact_id in (928,715,942,861)
 and inv_quantity_on_hand between 100 and 500
 and cs_item_sk = i_item_sk
 group by i_item_id,i_item_desc,i_current_price
 order by i_item_id
 limit 100""",
    "q38": f"""
select  count(*) from (
    select distinct c_last_name, c_first_name, d_date
    from {db}.store_sales, {db}.date_dim, {db}.customer
          where store_sales.ss_sold_date_sk = {db}.date_dim.d_date_sk
      and store_sales.ss_customer_sk = {db}.customer.c_customer_sk
      and d_month_seq between 1222 and 1222 + 11
  intersect
    select distinct c_last_name, c_first_name, d_date
    from {db}.catalog_sales, {db}.date_dim, {db}.customer
          where {db}.catalog_sales.cs_sold_date_sk = {db}.date_dim.d_date_sk
      and {db}.catalog_sales.cs_bill_customer_sk = {db}.customer.c_customer_sk
      and d_month_seq between 1222 and 1222 + 11
  intersect
    select distinct c_last_name, c_first_name, d_date
    from {db}.web_sales, {db}.date_dim, {db}.customer
          where {db}.web_sales.ws_sold_date_sk = {db}.date_dim.d_date_sk
      and {db}.web_sales.ws_bill_customer_sk = {db}.customer.c_customer_sk
      and d_month_seq between 1222 and 1222 + 11
) hot_cust
limit 100""",
    "q39": f"""
with inv as
(select w_warehouse_name,w_warehouse_sk,i_item_sk,d_moy
       ,stdev,mean, case mean when 0 then null else stdev/mean end cov
 from(select w_warehouse_name,w_warehouse_sk,i_item_sk,d_moy
            ,stddev_samp(inv_quantity_on_hand) stdev,avg(inv_quantity_on_hand) mean
      from {db}.inventory
          ,{db}.item
          ,{db}.warehouse
          ,{db}.date_dim
      where inv_item_sk = i_item_sk
        and inv_warehouse_sk = w_warehouse_sk
        and inv_date_sk = d_date_sk
        and d_year =1998
      group by w_warehouse_name,w_warehouse_sk,i_item_sk,d_moy) foo
 where case mean when 0 then 0 else stdev/mean end > 1)
select inv1.w_warehouse_sk,inv1.i_item_sk,inv1.d_moy,inv1.mean, inv1.cov
        ,inv2.w_warehouse_sk,inv2.i_item_sk,inv2.d_moy,inv2.mean, inv2.cov
from inv inv1,inv inv2
where inv1.i_item_sk = inv2.i_item_sk
  and inv1.w_warehouse_sk =  inv2.w_warehouse_sk
  and inv1.d_moy=4
  and inv2.d_moy=4+1
order by inv1.w_warehouse_sk,inv1.i_item_sk,inv1.d_moy,inv1.mean,inv1.cov
        ,inv2.d_moy,inv2.mean, inv2.cov""",
    "q40": f"""
select
   w_state
  ,i_item_id
  ,sum(case when (cast(d_date as date) < cast ('1999-02-02' as date))
 		then cs_sales_price - coalesce(cr_refunded_cash,0) else 0 end) as sales_before
  ,sum(case when (cast(d_date as date) >= cast ('1999-02-02' as date))
 		then cs_sales_price - coalesce(cr_refunded_cash,0) else 0 end) as sales_after
 from
   {db}.catalog_sales left outer join {db}.catalog_returns on
       (cs_order_number = cr_order_number
        and cs_item_sk = cr_item_sk)
  ,{db}.warehouse
  ,{db}.item
  ,{db}.date_dim
 where
     i_current_price between 0.99 and 1.49
 and i_item_sk          = cs_item_sk
 and cs_warehouse_sk    = w_warehouse_sk
 and cs_sold_date_sk    = d_date_sk
 and d_date between (cast ('1999-02-02' as date) - INTERVAL 30 days)
                and (cast ('1999-02-02' as date) + INTERVAL 30 days)
 group by
    w_state,i_item_id
 order by w_state,i_item_id
limit 100""",
    "q41": f"""
select  distinct(i_product_name)
 from {db}.item i1
 where i_manufact_id between 732 and 732+40
   and (select count(*) as item_cnt
        from {db}.item
        where (i_manufact = i1.i_manufact and
        ((i_category = 'Women' and
        (i_color = 'beige' or i_color = 'spring') and
        (i_units = 'Tsp' or i_units = 'Ton') and
        (i_size = 'petite' or i_size = 'extra large')
        ) or
        (i_category = 'Women' and
        (i_color = 'white' or i_color = 'pale') and
        (i_units = 'Box' or i_units = 'Dram') and
        (i_size = 'large' or i_size = 'economy')
        ) or
        (i_category = 'Men' and
        (i_color = 'midnight' or i_color = 'frosted') and
        (i_units = 'Bunch' or i_units = 'Carton') and
        (i_size = 'small' or i_size = 'N/A')
        ) or
        (i_category = 'Men' and
        (i_color = 'azure' or i_color = 'goldenrod') and
        (i_units = 'Pallet' or i_units = 'Gross') and
        (i_size = 'petite' or i_size = 'extra large')
        ))) or
       (i_manufact = i1.i_manufact and
        ((i_category = 'Women' and
        (i_color = 'brown' or i_color = 'hot') and
        (i_units = 'Tbl' or i_units = 'Cup') and
        (i_size = 'petite' or i_size = 'extra large')
        ) or
        (i_category = 'Women' and
        (i_color = 'powder' or i_color = 'honeydew') and
        (i_units = 'Bundle' or i_units = 'Unknown') and
        (i_size = 'large' or i_size = 'economy')
        ) or
        (i_category = 'Men' and
        (i_color = 'antique' or i_color = 'purple') and
        (i_units = 'N/A' or i_units = 'Dozen') and
        (i_size = 'small' or i_size = 'N/A')
        ) or
        (i_category = 'Men' and
        (i_color = 'lavender' or i_color = 'tomato') and
        (i_units = 'Lb' or i_units = 'Oz') and
        (i_size = 'petite' or i_size = 'extra large')
        )))) > 0
 order by i_product_name
 limit 100""",
    "q42": f"""
select  dt.d_year
 	,{db}.item.i_category_id
 	,{db}.item.i_category
 	,sum(ss_ext_sales_price)
 from 	{db}.date_dim dt
 	,{db}.store_sales
 	,{db}.item
 where dt.d_date_sk = {db}.store_sales.ss_sold_date_sk
 	and {db}.store_sales.ss_item_sk = {db}.item.i_item_sk
 	and {db}.item.i_manager_id = 1
 	and dt.d_moy=11
 	and dt.d_year=2002
 group by 	dt.d_year
 		,{db}.item.i_category_id
 		,{db}.item.i_category
 order by       sum(ss_ext_sales_price) desc,dt.d_year
 		,{db}.item.i_category_id
 		,{db}.item.i_category
limit 100 """,
    "q43": f"""
select  s_store_name, s_store_id,
        sum(case when (d_day_name='Sunday') then ss_sales_price else null end) sun_sales,
        sum(case when (d_day_name='Monday') then ss_sales_price else null end) mon_sales,
        sum(case when (d_day_name='Tuesday') then ss_sales_price else  null end) tue_sales,
        sum(case when (d_day_name='Wednesday') then ss_sales_price else null end) wed_sales,
        sum(case when (d_day_name='Thursday') then ss_sales_price else null end) thu_sales,
        sum(case when (d_day_name='Friday') then ss_sales_price else null end) fri_sales,
        sum(case when (d_day_name='Saturday') then ss_sales_price else null end) sat_sales
 from {db}.date_dim, {db}.store_sales, {db}.store
 where d_date_sk = ss_sold_date_sk and
       s_store_sk = ss_store_sk and
       s_gmt_offset = -6 and
       d_year = 1999
 group by s_store_name, s_store_id
 order by s_store_name, s_store_id,sun_sales,mon_sales,tue_sales,wed_sales,thu_sales,fri_sales,sat_sales
 limit 100""",
    "q44": f"""
select  asceding.rnk, i1.i_product_name best_performing, i2.i_product_name worst_performing
from(select *
     from (select item_sk,rank() over (order by rank_col asc) rnk
           from (select ss_item_sk item_sk,avg(ss_net_profit) rank_col
                 from {db}.store_sales ss1
                 where ss_store_sk = 321
                 group by ss_item_sk
                 having avg(ss_net_profit) > 0.9*(select avg(ss_net_profit) rank_col
                                                  from {db}.store_sales
                                                  where ss_store_sk = 321
                                                    and ss_addr_sk is null
                                                  group by ss_store_sk))V1)V11
     where rnk  < 11) asceding,
    (select *
     from (select item_sk,rank() over (order by rank_col desc) rnk
           from (select ss_item_sk item_sk,avg(ss_net_profit) rank_col
                 from {db}.store_sales ss1
                 where ss_store_sk = 321
                 group by ss_item_sk
                 having avg(ss_net_profit) > 0.9*(select avg(ss_net_profit) rank_col
                                                  from {db}.store_sales
                                                  where ss_store_sk = 321
                                                    and ss_addr_sk is null
                                                  group by ss_store_sk))V2)V21
     where rnk  < 11) descending,
{db}.item i1,
{db}.item i2
where asceding.rnk = descending.rnk
  and i1.i_item_sk=asceding.item_sk
  and i2.i_item_sk=descending.item_sk
order by asceding.rnk
limit 100""",
    "q45": f"""
select  ca_zip, ca_county, sum(ws_sales_price)
 from {db}.web_sales, {db}.customer, {db}.customer_address, {db}.date_dim, {db}.item
 where ws_bill_customer_sk = c_customer_sk
 	and c_current_addr_sk = ca_address_sk
 	and ws_item_sk = i_item_sk
 	and ( 
        (SUBSTR(ca_zip, 1, 5) LIKE '85669' OR
        SUBSTR(ca_zip, 1, 5) LIKE '86197' OR
        SUBSTR(ca_zip, 1, 5) LIKE '88274' OR
        SUBSTR(ca_zip, 1, 5) LIKE '83405' OR
        SUBSTR(ca_zip, 1, 5) LIKE '86475' OR
        SUBSTR(ca_zip, 1, 5) LIKE '85392' OR
        SUBSTR(ca_zip, 1, 5) LIKE '85460' OR
        SUBSTR(ca_zip, 1, 5) LIKE '80348' OR
        SUBSTR(ca_zip, 1, 5) LIKE '81792')
 	      or
 	      i_item_id in (select i_item_id
                             from {db}.item
                             where i_item_sk in (2, 3, 5, 7, 11, 13, 17, 19, 23, 29)
                             )
 	    )
 	and ws_sold_date_sk = d_date_sk
 	and d_qoy = 2 and d_year = 1999
 group by ca_zip, ca_county
 order by ca_zip, ca_county
 limit 100""",
    "q46": f"""
select  c_last_name
       ,c_first_name
       ,ca_city
       ,bought_city
       ,ss_ticket_number
       ,amt,profit
 from
   (select ss_ticket_number
          ,ss_customer_sk
          ,ca_city bought_city
          ,sum(ss_coupon_amt) amt
          ,sum(ss_net_profit) profit
    from {db}.store_sales,{db}.date_dim,{db}.store,{db}.household_demographics,{db}.customer_address
    where {db}.store_sales.ss_sold_date_sk = date_dim.d_date_sk
    and {db}.store_sales.ss_store_sk = {db}.store.s_store_sk
    and {db}.store_sales.ss_hdemo_sk = {db}.household_demographics.hd_demo_sk
    and {db}.store_sales.ss_addr_sk = {db}.customer_address.ca_address_sk
    and ({db}.household_demographics.hd_dep_count = 2 or
         {db}.household_demographics.hd_vehicle_count= 2)
    and {db}.date_dim.d_dow in (6,0)
    and {db}.date_dim.d_year in (1998,1998+1,1998+2)
    and 
    ({db}.store.s_city LIKE 'Antioch' OR
        {db}.store.s_city LIKE 'Mount Vernon' OR
        {db}.store.s_city LIKE 'Jamestown' OR
        {db}.store.s_city LIKE 'Wilson' OR
        {db}.store.s_city LIKE 'Farmington')
    group by ss_ticket_number,ss_customer_sk,ss_addr_sk,ca_city) dn,{db}.customer,{db}.customer_address current_addr
    where ss_customer_sk = c_customer_sk
      and {db}.customer.c_current_addr_sk = current_addr.ca_address_sk
      and current_addr.ca_city <> bought_city
  order by c_last_name
          ,c_first_name
          ,ca_city
          ,bought_city
          ,ss_ticket_number
  limit 100""",
    "q47": f"""
with v1 as(
 select i_category, i_brand,
        s_store_name, s_company_name,
        d_year, d_moy,
        sum(ss_sales_price) sum_sales,
        avg(sum(ss_sales_price)) over
          (partition by i_category, i_brand,
                     s_store_name, s_company_name, d_year)
          avg_monthly_sales,
        rank() over
          (partition by i_category, i_brand,
                     s_store_name, s_company_name
           order by d_year, d_moy) rn
 from {db}.item, {db}.store_sales, {db}.date_dim, {db}.store
 where ss_item_sk = i_item_sk and
       ss_sold_date_sk = d_date_sk and
       ss_store_sk = s_store_sk and
       (
         d_year = 2001 or
         ( d_year = 2001-1 and d_moy =12) or
         ( d_year = 2001+1 and d_moy =1)
       )
 group by i_category, i_brand,
          s_store_name, s_company_name,
          d_year, d_moy),
 v2 as(
 select v1.s_company_name
        ,v1.d_year, v1.d_moy
        ,v1.avg_monthly_sales
        ,v1.sum_sales, v1_lag.sum_sales psum, v1_lead.sum_sales nsum
 from v1, v1 v1_lag, v1 v1_lead
 where v1.i_category = v1_lag.i_category and
       v1.i_category = v1_lead.i_category and
       v1.i_brand = v1_lag.i_brand and
       v1.i_brand = v1_lead.i_brand and
       v1.s_store_name = v1_lag.s_store_name and
       v1.s_store_name = v1_lead.s_store_name and
       v1.s_company_name = v1_lag.s_company_name and
       v1.s_company_name = v1_lead.s_company_name and
       v1.rn = v1_lag.rn + 1 and
       v1.rn = v1_lead.rn - 1)
  select  *
 from v2
 where  d_year = 2001 and
        avg_monthly_sales > 0 and
        case when avg_monthly_sales > 0 then abs(sum_sales - avg_monthly_sales) / avg_monthly_sales else null end > 0.1
 order by sum_sales - avg_monthly_sales, avg_monthly_sales
 limit 100""",
    "q48": f"""
select sum (ss_quantity)
 from {db}.store_sales, {db}.store, {db}.customer_demographics, {db}.customer_address, {db}.date_dim
 where s_store_sk = ss_store_sk
 and  ss_sold_date_sk = d_date_sk and d_year = 1999
 and
 (
  (
   cd_demo_sk = ss_cdemo_sk
   and
   cd_marital_status = 'D'
   and
   cd_education_status = 'College'
   and
   ss_sales_price between 100.00 and 150.00
   )
 or
  (
  cd_demo_sk = ss_cdemo_sk
   and
   cd_marital_status = 'W'
   and
   cd_education_status = 'Secondary'
   and
   ss_sales_price between 50.00 and 100.00
  )
 or
 (
  cd_demo_sk = ss_cdemo_sk
  and
   cd_marital_status = 'M'
   and
   cd_education_status = '2 yr Degree'
   and
   ss_sales_price between 150.00 and 200.00
 )
 )
 and
 (
  (
  ss_addr_sk = ca_address_sk
  and
  {db}.customer_address.ca_country = 'United States'
  and
  {db}.customer_address.ca_state in ('NE', 'IA', 'NY')
  and {db}.store_sales.ss_net_profit between 0 and 2000
  )
 or
  (ss_addr_sk = ca_address_sk
  and
  {db}.customer_address.ca_country = 'United States'
  and
  ({db}.customer_address.ca_state LIKE 'IN' OR
        {db}.customer_address.ca_state LIKE 'TN' OR
        {db}.customer_address.ca_state LIKE 'OH')
  and {db}.store_sales.ss_net_profit between 150 and 3000
  )
 or
  (ss_addr_sk = ca_address_sk
  and
  {db}.customer_address.ca_country = 'United States'
  and
  ({db}.customer_address.ca_state LIKE 'KS' OR
        {db}.customer_address.ca_state LIKE 'CA' OR
        {db}.customer_address.ca_state LIKE 'CO')
  and {db}.store_sales.ss_net_profit between 50 and 25000
  )
 )""",
    "q49": f"""
select  channel, item, return_ratio, return_rank, currency_rank from
 (select
 'web' as channel
 ,web.item
 ,web.return_ratio
 ,web.return_rank
 ,web.currency_rank
 from (
 	select
 	 item
 	,return_ratio
 	,currency_ratio
 	,rank() over (order by return_ratio) as return_rank
 	,rank() over (order by currency_ratio) as currency_rank
 	from
 	(	select ws.ws_item_sk as item
 		,(cast(sum(coalesce(wr.wr_return_quantity,0)) as decimal(15,4))/
 		cast(sum(coalesce(ws.ws_quantity,0)) as decimal(15,4) )) as return_ratio
 		,(cast(sum(coalesce(wr.wr_return_amt,0)) as decimal(15,4))/
 		cast(sum(coalesce(ws.ws_net_paid,0)) as decimal(15,4) )) as currency_ratio
 		from
 		 {db}.web_sales ws left outer join {db}.web_returns wr
 			on (ws.ws_order_number = wr.wr_order_number and
 			ws.ws_item_sk = wr.wr_item_sk)
                 ,{db}.date_dim
 		where
 			wr.wr_return_amt > 10000
 			and ws.ws_net_profit > 1
                         and ws.ws_net_paid > 0
                         and ws.ws_quantity > 0
                         and ws_sold_date_sk = d_date_sk
                         and d_year = 2000
                         and d_moy = 11
 		group by ws.ws_item_sk
 	) in_web
 ) web
 where
 (
 web.return_rank <= 10
 or
 web.currency_rank <= 10
 )
 union
 select
 'catalog' as channel
 ,catalog.item
 ,catalog.return_ratio
 ,catalog.return_rank
 ,catalog.currency_rank
 from (
 	select
 	 item
 	,return_ratio
 	,currency_ratio
 	,rank() over (order by return_ratio) as return_rank
 	,rank() over (order by currency_ratio) as currency_rank
 	from
 	(	select
 		cs.cs_item_sk as item
 		,(cast(sum(coalesce(cr.cr_return_quantity,0)) as decimal(15,4))/
 		cast(sum(coalesce(cs.cs_quantity,0)) as decimal(15,4) )) as return_ratio
 		,(cast(sum(coalesce(cr.cr_return_amount,0)) as decimal(15,4))/
 		cast(sum(coalesce(cs.cs_net_paid,0)) as decimal(15,4) )) as currency_ratio
 		from
 		{db}.catalog_sales cs left outer join {db}.catalog_returns cr
 			on (cs.cs_order_number = cr.cr_order_number and
 			cs.cs_item_sk = cr.cr_item_sk)
                ,{db}.date_dim
 		where
 			cr.cr_return_amount > 10000
 			and cs.cs_net_profit > 1
                         and cs.cs_net_paid > 0
                         and cs.cs_quantity > 0
                         and cs_sold_date_sk = d_date_sk
                         and d_year = 2000
                         and d_moy = 11
                 group by cs.cs_item_sk
 	) in_cat
 ) catalog
 where
 (
 catalog.return_rank <= 10
 or
 catalog.currency_rank <=10
 )
 union
 select
 'store' as channel
 ,store.item
 ,store.return_ratio
 ,store.return_rank
 ,store.currency_rank
 from (
 	select
 	 item
 	,return_ratio
 	,currency_ratio
 	,rank() over (order by return_ratio) as return_rank
 	,rank() over (order by currency_ratio) as currency_rank
 	from
 	(	select sts.ss_item_sk as item
 		,(cast(sum(coalesce(sr.sr_return_quantity,0)) as decimal(15,4))/cast(sum(coalesce(sts.ss_quantity,0)) as decimal(15,4) )) as return_ratio
 		,(cast(sum(coalesce(sr.sr_return_amt,0)) as decimal(15,4))/cast(sum(coalesce(sts.ss_net_paid,0)) as decimal(15,4) )) as currency_ratio
 		from
 		{db}.store_sales sts left outer join {db}.store_returns sr
 			on (sts.ss_ticket_number = sr.sr_ticket_number and sts.ss_item_sk = sr.sr_item_sk)
                ,{db}.date_dim
 		where
 			sr.sr_return_amt > 10000
 			and sts.ss_net_profit > 1
                         and sts.ss_net_paid > 0
                         and sts.ss_quantity > 0
                         and ss_sold_date_sk = d_date_sk
                         and d_year = 2000
                         and d_moy = 11
 		group by sts.ss_item_sk
 	) in_store
 ) store
 where  (
 store.return_rank <= 10
 or
 store.currency_rank <= 10
 )
 )
 order by 1,4,5,2
 limit 100""",
    "q50": f"""
select
   s_store_name
  ,s_company_id
  ,s_street_number
  ,s_street_name
  ,s_street_type
  ,s_suite_number
  ,s_city
  ,s_county
  ,s_state
  ,s_zip
  ,sum(case when (sr_returned_date_sk - ss_sold_date_sk <= 30 ) then 1 else 0 end)  as `30 days`
  ,sum(case when (sr_returned_date_sk - ss_sold_date_sk > 30) and
                 (sr_returned_date_sk - ss_sold_date_sk <= 60) then 1 else 0 end )  as `31-60 days`
  ,sum(case when (sr_returned_date_sk - ss_sold_date_sk > 60) and
                 (sr_returned_date_sk - ss_sold_date_sk <= 90) then 1 else 0 end)  as `61-90 days`
  ,sum(case when (sr_returned_date_sk - ss_sold_date_sk > 90) and
                 (sr_returned_date_sk - ss_sold_date_sk <= 120) then 1 else 0 end)  as `91-120 days`
  ,sum(case when (sr_returned_date_sk - ss_sold_date_sk  > 120) then 1 else 0 end)  as `>120 days`
from
   {db}.store_sales
  ,{db}.store_returns
  ,{db}.store
  ,{db}.date_dim d1
  ,{db}.date_dim d2
where
    d2.d_year = 1999
and d2.d_moy  = 9
and ss_ticket_number = sr_ticket_number
and ss_item_sk = sr_item_sk
and ss_sold_date_sk   = d1.d_date_sk
and sr_returned_date_sk   = d2.d_date_sk
and ss_customer_sk = sr_customer_sk
and ss_store_sk = s_store_sk
group by
   s_store_name
  ,s_company_id
  ,s_street_number
  ,s_street_name
  ,s_street_type
  ,s_suite_number
  ,s_city
  ,s_county
  ,s_state
  ,s_zip
order by s_store_name
        ,s_company_id
        ,s_street_number
        ,s_street_name
        ,s_street_type
        ,s_suite_number
        ,s_city
        ,s_county
        ,s_state
        ,s_zip
limit 100""",
    "q51": f"""
WITH web_v1 as (
select
  ws_item_sk item_sk, d_date,
  sum(sum(ws_sales_price))
      over (partition by ws_item_sk order by d_date rows between unbounded preceding and current row) cume_sales
from {db}.web_sales
    ,{db}.date_dim
where ws_sold_date_sk=d_date_sk
  and d_month_seq between 1176 and 1176+11
  and ws_item_sk is not NULL
group by ws_item_sk, d_date),
store_v1 as (
select
  ss_item_sk item_sk, d_date,
  sum(sum(ss_sales_price))
      over (partition by ss_item_sk order by d_date rows between unbounded preceding and current row) cume_sales
from {db}.store_sales
    ,{db}.date_dim
where ss_sold_date_sk=d_date_sk
  and d_month_seq between 1176 and 1176+11
  and ss_item_sk is not NULL
group by ss_item_sk, d_date)
 select  *
from (select item_sk
     ,d_date
     ,{db}.web_sales
     ,{db}.store_sales
     ,max({db}.web_sales)
         over (partition by item_sk order by d_date rows between unbounded preceding and current row) web_cumulative
     ,max(store_sales)
         over (partition by item_sk order by d_date rows between unbounded preceding and current row) store_cumulative
     from (select case when web.item_sk is not null then web.item_sk else store.item_sk end item_sk
                 ,case when web.d_date is not null then web.d_date else store.d_date end d_date
                 ,web.cume_sales {db}.web_sales
                 ,store.cume_sales store_sales
           from web_v1 web full outer join store_v1 store on (web.item_sk = store.item_sk
                                                          and web.d_date = store.d_date)
          )x )y
where web_cumulative > store_cumulative
order by item_sk
        ,d_date
limit 100""",
    "q52": f"""
select  dt.d_year
 	,{db}.item.i_brand_id brand_id
 	,{db}.item.i_brand brand
 	,sum(ss_ext_sales_price) ext_price
 from {db}.date_dim dt
     ,{db}.store_sales
     ,{db}.item
 where dt.d_date_sk = store_sales.ss_sold_date_sk
    and store_sales.ss_item_sk = {db}.item.i_item_sk
    and {db}.item.i_manager_id = 1
    and dt.d_moy=11
    and dt.d_year=2001
 group by dt.d_year
 	,{db}.item.i_brand
 	,{db}.item.i_brand_id
 order by dt.d_year
 	,ext_price desc
 	,brand_id
limit 100 """,
    "q53": f"""
select  * from
(select i_manufact_id,
sum(ss_sales_price) sum_sales,
avg(sum(ss_sales_price)) over (partition by i_manufact_id) avg_quarterly_sales
from {db}.item, {db}.store_sales, {db}.date_dim, {db}.store
where ss_item_sk = i_item_sk and
ss_sold_date_sk = d_date_sk and
ss_store_sk = s_store_sk and
d_month_seq in (1218,1218+1,1218+2,1218+3,1218+4,1218+5,1218+6,1218+7,1218+8,1218+9,1218+10,1218+11) and
((i_category in ('Books','Children','Electronics') and
i_class in ('personal','portable','reference','self-help') and
i_brand in ('scholaramalgamalg #14','scholaramalgamalg #7',
		'exportiunivamalg #9','scholaramalgamalg #9'))
or(i_category in ('Women','Music','Men') and
i_class in ('accessories','classical','fragrances','pants') and
i_brand in ('amalgimporto #1','edu packscholar #1','exportiimporto #1',
		'importoamalg #1')))
group by i_manufact_id, d_qoy ) tmp1
where case when avg_quarterly_sales > 0
	then abs (sum_sales - avg_quarterly_sales)/ avg_quarterly_sales
	else null end > 0.1
order by avg_quarterly_sales,
	 sum_sales,
	 i_manufact_id
limit 100""",
    "q54": f"""
with my_customers as (
 select distinct c_customer_sk
        , c_current_addr_sk
 from
        ( select cs_sold_date_sk sold_date_sk,
                 cs_bill_customer_sk customer_sk,
                 cs_item_sk item_sk
          from   {db}.catalog_sales
          union all
          select ws_sold_date_sk sold_date_sk,
                 ws_bill_customer_sk customer_sk,
                 ws_item_sk item_sk
          from   {db}.web_sales
         ) cs_or_ws_sales,
         {db}.item,
         {db}.date_dim,
         {db}.customer
 where   sold_date_sk = d_date_sk
         and item_sk = i_item_sk
         and i_category = 'Music'
         and i_class = 'country'
         and c_customer_sk = cs_or_ws_sales.customer_sk
         and d_moy = 7
         and d_year = 2001
 )
 , my_revenue as (
 select c_customer_sk,
        sum(ss_ext_sales_price) as revenue
 from   my_customers,
        {db}.store_sales,
        {db}.customer_address,
        {db}.store,
        {db}.date_dim
 where  c_current_addr_sk = ca_address_sk
        and ca_county = s_county
        and ca_state = s_state
        and ss_sold_date_sk = d_date_sk
        and c_customer_sk = ss_customer_sk
        and d_month_seq between (select distinct d_month_seq+1
                                 from   {db}.date_dim where d_year = 2001 and d_moy = 7)
                           and  (select distinct d_month_seq+3
                                 from   {db}.date_dim where d_year = 2001 and d_moy = 7)
 group by c_customer_sk
 )
 , segments as
 (select cast((revenue/50) as int) as segment
  from   my_revenue
 )
  select  segment, count(*) as num_customers, segment*50 as segment_base
 from segments
 group by segment
 order by segment, num_customers
 limit 100""",
    "q55": f"""
select  i_brand_id brand_id, i_brand brand,
 	sum(ss_ext_sales_price) ext_price
 from {db}.date_dim, {db}.store_sales, {db}.item
 where d_date_sk = ss_sold_date_sk
 	and ss_item_sk = i_item_sk
 	and i_manager_id=87
 	and d_moy=11
 	and d_year=2001
 group by i_brand, i_brand_id
 order by ext_price desc, i_brand_id
limit 100 """,
    "q56": f"""
with ss as (
 select i_item_id,sum(ss_ext_sales_price) total_sales
 from
 	{db}.store_sales,
 	{db}.date_dim,
         {db}.customer_address,
         {db}.item
 where i_item_id in (select
     i_item_id
from {db}.item
where i_color in ('tan','lace','gainsboro'))
 and     ss_item_sk              = i_item_sk
 and     ss_sold_date_sk         = d_date_sk
 and     d_year                  = 1998
 and     d_moy                   = 3
 and     ss_addr_sk              = ca_address_sk
 and     ca_gmt_offset           = -5
 group by i_item_id),
 cs as (
 select i_item_id,sum(cs_ext_sales_price) total_sales
 from
 	{db}.catalog_sales,
 	{db}.date_dim,
         {db}.customer_address,
         {db}.item
 where
         i_item_id               in (select
  i_item_id
from {db}.item
where i_color in ('tan','lace','gainsboro'))
 and     cs_item_sk              = i_item_sk
 and     cs_sold_date_sk         = d_date_sk
 and     d_year                  = 1998
 and     d_moy                   = 3
 and     cs_bill_addr_sk         = ca_address_sk
 and     ca_gmt_offset           = -5
 group by i_item_id),
 ws as (
 select i_item_id,sum(ws_ext_sales_price) total_sales
 from
 	{db}.web_sales,
 	{db}.date_dim,
         {db}.customer_address,
         {db}.item
 where
         i_item_id               in (select
  i_item_id
from {db}.item
where i_color in ('tan','lace','gainsboro'))
 and     ws_item_sk              = i_item_sk
 and     ws_sold_date_sk         = d_date_sk
 and     d_year                  = 1998
 and     d_moy                   = 3
 and     ws_bill_addr_sk         = ca_address_sk
 and     ca_gmt_offset           = -5
 group by i_item_id)
  select  i_item_id ,sum(total_sales) total_sales
 from  (select * from ss
        union all
        select * from cs
        union all
        select * from ws) tmp1
 group by i_item_id
 order by total_sales,
          i_item_id
 limit 100""",
    "q57": f"""
with v1 as(
 select i_category, i_brand,
        cc_name,
        d_year, d_moy,
        sum(cs_sales_price) sum_sales,
        avg(sum(cs_sales_price)) over
          (partition by i_category, i_brand,
                     cc_name, d_year)
          avg_monthly_sales,
        rank() over
          (partition by i_category, i_brand,
                     cc_name
           order by d_year, d_moy) rn
 from {db}.item, {db}.catalog_sales, {db}.date_dim, {db}.call_center
 where cs_item_sk = i_item_sk and
       cs_sold_date_sk = d_date_sk and
       cc_call_center_sk= cs_call_center_sk and
       (
         d_year = 2001 or
         ( d_year = 2001-1 and d_moy =12) or
         ( d_year = 2001+1 and d_moy =1)
       )
 group by i_category, i_brand,
          cc_name , d_year, d_moy),
 v2 as(
 select v1.i_category, v1.i_brand, v1.cc_name
        ,v1.d_year, v1.d_moy
        ,v1.avg_monthly_sales
        ,v1.sum_sales, v1_lag.sum_sales psum, v1_lead.sum_sales nsum
 from v1, v1 v1_lag, v1 v1_lead
 where v1.i_category = v1_lag.i_category and
       v1.i_category = v1_lead.i_category and
       v1.i_brand = v1_lag.i_brand and
       v1.i_brand = v1_lead.i_brand and
       v1. cc_name = v1_lag. cc_name and
       v1. cc_name = v1_lead. cc_name and
       v1.rn = v1_lag.rn + 1 and
       v1.rn = v1_lead.rn - 1)
  select  *
 from v2
 where  d_year = 2001 and
        avg_monthly_sales > 0 and
        case when avg_monthly_sales > 0 then abs(sum_sales - avg_monthly_sales) / avg_monthly_sales else null end > 0.1
 order by sum_sales - avg_monthly_sales, avg_monthly_sales
 limit 100""",
    "q58": f"""
with ss_items as
 (select i_item_id item_id
        ,sum(ss_ext_sales_price) ss_item_rev
 from {db}.store_sales
     ,{db}.item
     ,{db}.date_dim
 where ss_item_sk = i_item_sk
   and d_date in (select d_date
                  from {db}.date_dim
                  where d_week_seq = (select d_week_seq
                                      from {db}.date_dim
                                      where d_date = '2000-03-26'))
   and ss_sold_date_sk   = d_date_sk
 group by i_item_id),
 cs_items as
 (select i_item_id item_id
        ,sum(cs_ext_sales_price) cs_item_rev
  from {db}.catalog_sales
      ,{db}.item
      ,{db}.date_dim
 where cs_item_sk = i_item_sk
  and  d_date in (select d_date
                  from {db}.date_dim
                  where d_week_seq = (select d_week_seq
                                      from {db}.date_dim
                                      where d_date = '2000-03-26'))
  and  cs_sold_date_sk = d_date_sk
 group by i_item_id),
 ws_items as
 (select i_item_id item_id
        ,sum(ws_ext_sales_price) ws_item_rev
  from {db}.web_sales
      ,{db}.item
      ,{db}.date_dim
 where ws_item_sk = i_item_sk
  and  d_date in (select d_date
                  from {db}.date_dim
                  where d_week_seq =(select d_week_seq
                                     from {db}.date_dim
                                     where d_date = '2000-03-26'))
  and ws_sold_date_sk   = d_date_sk
 group by i_item_id)
  select  ss_items.item_id
       ,ss_item_rev
       ,ss_item_rev/((ss_item_rev+cs_item_rev+ws_item_rev)/3) * 100 ss_dev
       ,cs_item_rev
       ,cs_item_rev/((ss_item_rev+cs_item_rev+ws_item_rev)/3) * 100 cs_dev
       ,ws_item_rev
       ,ws_item_rev/((ss_item_rev+cs_item_rev+ws_item_rev)/3) * 100 ws_dev
       ,(ss_item_rev+cs_item_rev+ws_item_rev)/3 average
 from ss_items,cs_items,ws_items
 where ss_items.item_id=cs_items.item_id
   and ss_items.item_id=ws_items.item_id
   and ss_item_rev between 0.9 * cs_item_rev and 1.1 * cs_item_rev
   and ss_item_rev between 0.9 * ws_item_rev and 1.1 * ws_item_rev
   and cs_item_rev between 0.9 * ss_item_rev and 1.1 * ss_item_rev
   and cs_item_rev between 0.9 * ws_item_rev and 1.1 * ws_item_rev
   and ws_item_rev between 0.9 * ss_item_rev and 1.1 * ss_item_rev
   and ws_item_rev between 0.9 * cs_item_rev and 1.1 * cs_item_rev
 order by item_id
         ,ss_item_rev
 limit 100""",
    "q59": f"""
with wss as
 (select d_week_seq,
        ss_store_sk,
        sum(case when (d_day_name='Sunday') then ss_sales_price else null end) sun_sales,
        sum(case when (d_day_name='Monday') then ss_sales_price else null end) mon_sales,
        sum(case when (d_day_name='Tuesday') then ss_sales_price else  null end) tue_sales,
        sum(case when (d_day_name='Wednesday') then ss_sales_price else null end) wed_sales,
        sum(case when (d_day_name='Thursday') then ss_sales_price else null end) thu_sales,
        sum(case when (d_day_name='Friday') then ss_sales_price else null end) fri_sales,
        sum(case when (d_day_name='Saturday') then ss_sales_price else null end) sat_sales
 from {db}.store_sales,{db}.date_dim
 where d_date_sk = ss_sold_date_sk
 group by d_week_seq,ss_store_sk
 )
  select  s_store_name1,s_store_id1,d_week_seq1
       ,sun_sales1/sun_sales2,mon_sales1/mon_sales2
       ,tue_sales1/tue_sales2,wed_sales1/wed_sales2,thu_sales1/thu_sales2
       ,fri_sales1/fri_sales2,sat_sales1/sat_sales2
 from
 (select s_store_name s_store_name1,wss.d_week_seq d_week_seq1
        ,s_store_id s_store_id1,sun_sales sun_sales1
        ,mon_sales mon_sales1,tue_sales tue_sales1
        ,wed_sales wed_sales1,thu_sales thu_sales1
        ,fri_sales fri_sales1,sat_sales sat_sales1
  from wss,{db}.store,{db}.date_dim d
  where d.d_week_seq = wss.d_week_seq and
        ss_store_sk = s_store_sk and
        d_month_seq between 1199 and 1199 + 11) y,
 (select s_store_name s_store_name2,wss.d_week_seq d_week_seq2
        ,s_store_id s_store_id2,sun_sales sun_sales2
        ,mon_sales mon_sales2,tue_sales tue_sales2
        ,wed_sales wed_sales2,thu_sales thu_sales2
        ,fri_sales fri_sales2,sat_sales sat_sales2
  from wss,{db}.store,{db}.date_dim d
  where d.d_week_seq = wss.d_week_seq and
        ss_store_sk = s_store_sk and
        d_month_seq between 1199+ 12 and 1199 + 23) x
 where s_store_id1=s_store_id2
   and d_week_seq1=d_week_seq2-52
 order by s_store_name1,s_store_id1,d_week_seq1
limit 100""",
    "q60": f"""
with ss as (
 select
          i_item_id,sum(ss_ext_sales_price) total_sales
 from
 	{db}.store_sales,
 	{db}.date_dim,
         {db}.customer_address,
         {db}.item
 where
         i_item_id in (select
  i_item_id
from
 {db}.item
where i_category in ('Men'))
 and     ss_item_sk              = i_item_sk
 and     ss_sold_date_sk         = d_date_sk
 and     d_year                  = 2000
 and     d_moy                   = 9
 and     ss_addr_sk              = ca_address_sk
 and     ca_gmt_offset           = -5
 group by i_item_id),
 cs as (
 select
          i_item_id,sum(cs_ext_sales_price) total_sales
 from
 	{db}.catalog_sales,
 	{db}.date_dim,
         {db}.customer_address,
         {db}.item
 where
         i_item_id               in (select
  i_item_id
from
 {db}.item
where i_category in ('Men'))
 and     cs_item_sk              = i_item_sk
 and     cs_sold_date_sk         = d_date_sk
 and     d_year                  = 2000
 and     d_moy                   = 9
 and     cs_bill_addr_sk         = ca_address_sk
 and     ca_gmt_offset           = -5
 group by i_item_id),
 ws as (
 select
          i_item_id,sum(ws_ext_sales_price) total_sales
 from
 	{db}.web_sales,
 	{db}.date_dim,
         {db}.customer_address,
         {db}.item
 where
         i_item_id               in (select
  i_item_id
from
 {db}.item
where i_category in ('Men'))
 and     ws_item_sk              = i_item_sk
 and     ws_sold_date_sk         = d_date_sk
 and     d_year                  = 2000
 and     d_moy                   = 9
 and     ws_bill_addr_sk         = ca_address_sk
 and     ca_gmt_offset           = -5
 group by i_item_id)
  select
  i_item_id
,sum(total_sales) total_sales
 from  (select * from ss
        union all
        select * from cs
        union all
        select * from ws) tmp1
 group by i_item_id
 order by i_item_id
      ,total_sales
 limit 100""",
    "q61": f"""
select  promotions,total,cast(promotions as decimal(15,4))/cast(total as decimal(15,4))*100
from
  (select sum(ss_ext_sales_price) promotions
   from  {db}.store_sales
        ,{db}.store
        ,{db}.promotion
        ,{db}.date_dim
        ,{db}.customer
        ,{db}.customer_address
        ,{db}.item
   where ss_sold_date_sk = d_date_sk
   and   ss_store_sk = s_store_sk
   and   ss_promo_sk = p_promo_sk
   and   ss_customer_sk= c_customer_sk
   and   ca_address_sk = c_current_addr_sk
   and   ss_item_sk = i_item_sk
   and   ca_gmt_offset = -7
   and   i_category = 'Electronics'
   and   (p_channel_dmail = 'Y' or p_channel_email = 'Y' or p_channel_tv = 'Y')
   and   s_gmt_offset = -7
   and   d_year = 2001
   and   d_moy  = 11) promotional_sales,
  (select sum(ss_ext_sales_price) total
   from  {db}.store_sales
        ,{db}.store
        ,{db}.date_dim
        ,{db}.customer
        ,{db}.customer_address
        ,{db}.item
   where ss_sold_date_sk = d_date_sk
   and   ss_store_sk = s_store_sk
   and   ss_customer_sk= c_customer_sk
   and   ca_address_sk = c_current_addr_sk
   and   ss_item_sk = i_item_sk
   and   ca_gmt_offset = -7
   and   i_category = 'Electronics'
   and   s_gmt_offset = -7
   and   d_year = 2001
   and   d_moy  = 11) all_sales
order by promotions, total
limit 100""",
    "q62": f"""
select
   substr(w_warehouse_name,1,20)
  ,sm_type
  ,web_name
  ,sum(case when (ws_ship_date_sk - ws_sold_date_sk <= 30 ) then 1 else 0 end)  as `30 days`
  ,sum(case when (ws_ship_date_sk - ws_sold_date_sk > 30) and
                 (ws_ship_date_sk - ws_sold_date_sk <= 60) then 1 else 0 end )  as `31-60 days`
  ,sum(case when (ws_ship_date_sk - ws_sold_date_sk > 60) and
                 (ws_ship_date_sk - ws_sold_date_sk <= 90) then 1 else 0 end)  as `61-90 days`
  ,sum(case when (ws_ship_date_sk - ws_sold_date_sk > 90) and
                 (ws_ship_date_sk - ws_sold_date_sk <= 120) then 1 else 0 end)  as `91-120 days`
  ,sum(case when (ws_ship_date_sk - ws_sold_date_sk  > 120) then 1 else 0 end)  as `>120 days`
from
   {db}.web_sales
  ,{db}.warehouse
  ,{db}.ship_mode
  ,{db}.web_site
  ,{db}.date_dim
where
    d_month_seq between 1194 and 1194 + 11
and ws_ship_date_sk   = d_date_sk
and ws_warehouse_sk   = w_warehouse_sk
and ws_ship_mode_sk   = sm_ship_mode_sk
and ws_web_site_sk    = web_site_sk
group by
   substr(w_warehouse_name,1,20)
  ,sm_type
  ,web_name
order by substr(w_warehouse_name,1,20)
        ,sm_type
       ,web_name
limit 100""",
    "q63": f"""
select  *
from (select i_manager_id
             ,sum(ss_sales_price) sum_sales
             ,avg(sum(ss_sales_price)) over (partition by i_manager_id) avg_monthly_sales
      from {db}.item
          ,{db}.store_sales
          ,{db}.date_dim
          ,{db}.store
      where ss_item_sk = i_item_sk
        and ss_sold_date_sk = d_date_sk
        and ss_store_sk = s_store_sk
        and d_month_seq in (1205,1205+1,1205+2,1205+3,1205+4,1205+5,1205+6,1205+7,1205+8,1205+9,1205+10,1205+11)
        and ((    i_category in ('Books','Children','Electronics')
              and i_class in ('personal','portable','reference','self-help')
              and i_brand in ('scholaramalgamalg #14','scholaramalgamalg #7',
		                  'exportiunivamalg #9','scholaramalgamalg #9'))
           or(    i_category in ('Women','Music','Men')
              and i_class in ('accessories','classical','fragrances','pants')
              and i_brand in ('amalgimporto #1','edu packscholar #1','exportiimporto #1',
		                 'importoamalg #1')))
group by i_manager_id, d_moy) tmp1
where case when avg_monthly_sales > 0 then abs (sum_sales - avg_monthly_sales) / avg_monthly_sales else null end > 0.1
order by i_manager_id
        ,avg_monthly_sales
        ,sum_sales
limit 100""",
    "q64": f"""
with cs_ui as
 (select cs_item_sk
        ,sum(cs_ext_list_price) as sale,sum(cr_refunded_cash+cr_reversed_charge+cr_store_credit) as refund
  from {db}.catalog_sales
      ,{db}.catalog_returns
  where cs_item_sk = cr_item_sk
    and cs_order_number = cr_order_number
  group by cs_item_sk
  having sum(cs_ext_list_price)>2*sum(cr_refunded_cash+cr_reversed_charge+cr_store_credit)),
cross_sales as
 (select i_product_name product_name
     ,i_item_sk item_sk
     ,s_store_name store_name
     ,s_zip store_zip
     ,ad1.ca_street_number b_street_number
     ,ad1.ca_street_name b_street_name
     ,ad1.ca_city b_city
     ,ad1.ca_zip b_zip
     ,ad2.ca_street_number c_street_number
     ,ad2.ca_street_name c_street_name
     ,ad2.ca_city c_city
     ,ad2.ca_zip c_zip
     ,d1.d_year as syear
     ,d2.d_year as fsyear
     ,d3.d_year s2year
     ,count(*) cnt
     ,sum(ss_wholesale_cost) s1
     ,sum(ss_list_price) s2
     ,sum(ss_coupon_amt) s3
  FROM   {db}.store_sales
        ,{db}.store_returns
        ,cs_ui
        ,{db}.date_dim d1
        ,{db}.date_dim d2
        ,{db}.date_dim d3
        ,{db}.store
        ,{db}.customer
        ,{db}.customer_demographics cd1
        ,{db}.customer_demographics cd2
        ,{db}.promotion
        ,{db}.household_demographics hd1
        ,{db}.household_demographics hd2
        ,{db}.customer_address ad1
        ,{db}.customer_address ad2
        ,{db}.income_band ib1
        ,{db}.income_band ib2
        ,{db}.item
  WHERE  ss_store_sk = s_store_sk AND
         ss_sold_date_sk = d1.d_date_sk AND
         ss_customer_sk = c_customer_sk AND
         ss_cdemo_sk= cd1.cd_demo_sk AND
         ss_hdemo_sk = hd1.hd_demo_sk AND
         ss_addr_sk = ad1.ca_address_sk and
         ss_item_sk = i_item_sk and
         ss_item_sk = sr_item_sk and
         ss_ticket_number = sr_ticket_number and
         ss_item_sk = cs_ui.cs_item_sk and
         c_current_cdemo_sk = cd2.cd_demo_sk AND
         c_current_hdemo_sk = hd2.hd_demo_sk AND
         c_current_addr_sk = ad2.ca_address_sk and
         c_first_sales_date_sk = d2.d_date_sk and
         c_first_shipto_date_sk = d3.d_date_sk and
         ss_promo_sk = p_promo_sk and
         hd1.hd_income_band_sk = ib1.ib_income_band_sk and
         hd2.hd_income_band_sk = ib2.ib_income_band_sk and
         cd1.cd_marital_status <> cd2.cd_marital_status and
         i_color in ('peach','misty','drab','chocolate','almond','saddle') and
         i_current_price between 75 and 75 + 10 and
         i_current_price between 75 + 1 and 75 + 15
group by i_product_name
       ,i_item_sk
       ,s_store_name
       ,s_zip
       ,ad1.ca_street_number
       ,ad1.ca_street_name
       ,ad1.ca_city
       ,ad1.ca_zip
       ,ad2.ca_street_number
       ,ad2.ca_street_name
       ,ad2.ca_city
       ,ad2.ca_zip
       ,d1.d_year
       ,d2.d_year
       ,d3.d_year
)
select cs1.product_name
     ,cs1.store_name
     ,cs1.store_zip
     ,cs1.b_street_number
     ,cs1.b_street_name
     ,cs1.b_city
     ,cs1.b_zip
     ,cs1.c_street_number
     ,cs1.c_street_name
     ,cs1.c_city
     ,cs1.c_zip
     ,cs1.syear
     ,cs1.cnt
     ,cs1.s1 as s11
     ,cs1.s2 as s21
     ,cs1.s3 as s31
     ,cs2.s1 as s12
     ,cs2.s2 as s22
     ,cs2.s3 as s32
     ,cs2.syear
     ,cs2.cnt
from cross_sales cs1,cross_sales cs2
where cs1.item_sk=cs2.item_sk and
     cs1.syear = 2000 and
     cs2.syear = 2000 + 1 and
     cs2.cnt <= cs1.cnt and
     cs1.store_name = cs2.store_name and
     cs1.store_zip = cs2.store_zip
order by cs1.product_name
       ,cs1.store_name
       ,cs2.cnt
       ,cs1.s1
       ,cs2.s1""",
    "q65": f"""
select
	s_store_name,
	i_item_desc,
	sc.revenue,
	i_current_price,
	i_wholesale_cost,
	i_brand
 from {db}.store, {db}.item,
     (select ss_store_sk, avg(revenue) as ave
 	from
 	    (select  ss_store_sk, ss_item_sk,
 		     sum(ss_sales_price) as revenue
 		from {db}.store_sales, {db}.date_dim
 		where ss_sold_date_sk = d_date_sk and d_month_seq between 1208 and 1208+11
 		group by ss_store_sk, ss_item_sk) sa
 	group by ss_store_sk) sb,
     (select  ss_store_sk, ss_item_sk, sum(ss_sales_price) as revenue
 	from {db}.store_sales, {db}.date_dim
 	where ss_sold_date_sk = d_date_sk and d_month_seq between 1208 and 1208+11
 	group by ss_store_sk, ss_item_sk) sc
 where sb.ss_store_sk = sc.ss_store_sk and
       sc.revenue <= 0.1 * sb.ave and
       s_store_sk = sc.ss_store_sk and
       i_item_sk = sc.ss_item_sk
 order by s_store_name, i_item_desc
limit 100""",
    "q66": f"""
select
         w_warehouse_name
 	,w_warehouse_sq_ft
 	,w_city
 	,w_county
 	,w_state
 	,w_country
        ,ship_carriers
        ,year
 	,sum(jan_sales) as jan_sales
 	,sum(feb_sales) as feb_sales
 	,sum(mar_sales) as mar_sales
 	,sum(apr_sales) as apr_sales
 	,sum(may_sales) as may_sales
 	,sum(jun_sales) as jun_sales
 	,sum(jul_sales) as jul_sales
 	,sum(aug_sales) as aug_sales
 	,sum(sep_sales) as sep_sales
 	,sum(oct_sales) as oct_sales
 	,sum(nov_sales) as nov_sales
 	,sum(dec_sales) as dec_sales
 	,sum(jan_sales/w_warehouse_sq_ft) as jan_sales_per_sq_foot
 	,sum(feb_sales/w_warehouse_sq_ft) as feb_sales_per_sq_foot
 	,sum(mar_sales/w_warehouse_sq_ft) as mar_sales_per_sq_foot
 	,sum(apr_sales/w_warehouse_sq_ft) as apr_sales_per_sq_foot
 	,sum(may_sales/w_warehouse_sq_ft) as may_sales_per_sq_foot
 	,sum(jun_sales/w_warehouse_sq_ft) as jun_sales_per_sq_foot
 	,sum(jul_sales/w_warehouse_sq_ft) as jul_sales_per_sq_foot
 	,sum(aug_sales/w_warehouse_sq_ft) as aug_sales_per_sq_foot
 	,sum(sep_sales/w_warehouse_sq_ft) as sep_sales_per_sq_foot
 	,sum(oct_sales/w_warehouse_sq_ft) as oct_sales_per_sq_foot
 	,sum(nov_sales/w_warehouse_sq_ft) as nov_sales_per_sq_foot
 	,sum(dec_sales/w_warehouse_sq_ft) as dec_sales_per_sq_foot
 	,sum(jan_net) as jan_net
 	,sum(feb_net) as feb_net
 	,sum(mar_net) as mar_net
 	,sum(apr_net) as apr_net
 	,sum(may_net) as may_net
 	,sum(jun_net) as jun_net
 	,sum(jul_net) as jul_net
 	,sum(aug_net) as aug_net
 	,sum(sep_net) as sep_net
 	,sum(oct_net) as oct_net
 	,sum(nov_net) as nov_net
 	,sum(dec_net) as dec_net
 from (
     select
 	w_warehouse_name
 	,w_warehouse_sq_ft
 	,w_city
 	,w_county
 	,w_state
 	,w_country
 	,'HARMSTORF' || ',' || 'USPS' as ship_carriers
       ,d_year as year
 	,sum(case when d_moy = 1
 		then ws_sales_price* ws_quantity else 0 end) as jan_sales
 	,sum(case when d_moy = 2
 		then ws_sales_price* ws_quantity else 0 end) as feb_sales
 	,sum(case when d_moy = 3
 		then ws_sales_price* ws_quantity else 0 end) as mar_sales
 	,sum(case when d_moy = 4
 		then ws_sales_price* ws_quantity else 0 end) as apr_sales
 	,sum(case when d_moy = 5
 		then ws_sales_price* ws_quantity else 0 end) as may_sales
 	,sum(case when d_moy = 6
 		then ws_sales_price* ws_quantity else 0 end) as jun_sales
 	,sum(case when d_moy = 7
 		then ws_sales_price* ws_quantity else 0 end) as jul_sales
 	,sum(case when d_moy = 8
 		then ws_sales_price* ws_quantity else 0 end) as aug_sales
 	,sum(case when d_moy = 9
 		then ws_sales_price* ws_quantity else 0 end) as sep_sales
 	,sum(case when d_moy = 10
 		then ws_sales_price* ws_quantity else 0 end) as oct_sales
 	,sum(case when d_moy = 11
 		then ws_sales_price* ws_quantity else 0 end) as nov_sales
 	,sum(case when d_moy = 12
 		then ws_sales_price* ws_quantity else 0 end) as dec_sales
 	,sum(case when d_moy = 1
 		then ws_net_paid_inc_tax * ws_quantity else 0 end) as jan_net
 	,sum(case when d_moy = 2
 		then ws_net_paid_inc_tax * ws_quantity else 0 end) as feb_net
 	,sum(case when d_moy = 3
 		then ws_net_paid_inc_tax * ws_quantity else 0 end) as mar_net
 	,sum(case when d_moy = 4
 		then ws_net_paid_inc_tax * ws_quantity else 0 end) as apr_net
 	,sum(case when d_moy = 5
 		then ws_net_paid_inc_tax * ws_quantity else 0 end) as may_net
 	,sum(case when d_moy = 6
 		then ws_net_paid_inc_tax * ws_quantity else 0 end) as jun_net
 	,sum(case when d_moy = 7
 		then ws_net_paid_inc_tax * ws_quantity else 0 end) as jul_net
 	,sum(case when d_moy = 8
 		then ws_net_paid_inc_tax * ws_quantity else 0 end) as aug_net
 	,sum(case when d_moy = 9
 		then ws_net_paid_inc_tax * ws_quantity else 0 end) as sep_net
 	,sum(case when d_moy = 10
 		then ws_net_paid_inc_tax * ws_quantity else 0 end) as oct_net
 	,sum(case when d_moy = 11
 		then ws_net_paid_inc_tax * ws_quantity else 0 end) as nov_net
 	,sum(case when d_moy = 12
 		then ws_net_paid_inc_tax * ws_quantity else 0 end) as dec_net
     from
          {db}.web_sales
         ,{db}.warehouse
         ,{db}.date_dim
         ,{db}.time_dim
 	  ,{db}.ship_mode
     where
            ws_warehouse_sk =  w_warehouse_sk
        and ws_sold_date_sk = d_date_sk
        and ws_sold_time_sk = t_time_sk
 	and ws_ship_mode_sk = sm_ship_mode_sk
        and d_year = 2002
 	and t_time between 24285 and 24285+28800
 	and sm_carrier in ('HARMSTORF','USPS')
     group by
        w_warehouse_name
 	,w_warehouse_sq_ft
 	,w_city
 	,w_county
 	,w_state
 	,w_country
       ,d_year
 union all
     select
 	w_warehouse_name
 	,w_warehouse_sq_ft
 	,w_city
 	,w_county
 	,w_state
 	,w_country
 	,'HARMSTORF' || ',' || 'USPS' as ship_carriers
       ,d_year as year
 	,sum(case when d_moy = 1
 		then cs_ext_list_price* cs_quantity else 0 end) as jan_sales
 	,sum(case when d_moy = 2
 		then cs_ext_list_price* cs_quantity else 0 end) as feb_sales
 	,sum(case when d_moy = 3
 		then cs_ext_list_price* cs_quantity else 0 end) as mar_sales
 	,sum(case when d_moy = 4
 		then cs_ext_list_price* cs_quantity else 0 end) as apr_sales
 	,sum(case when d_moy = 5
 		then cs_ext_list_price* cs_quantity else 0 end) as may_sales
 	,sum(case when d_moy = 6
 		then cs_ext_list_price* cs_quantity else 0 end) as jun_sales
 	,sum(case when d_moy = 7
 		then cs_ext_list_price* cs_quantity else 0 end) as jul_sales
 	,sum(case when d_moy = 8
 		then cs_ext_list_price* cs_quantity else 0 end) as aug_sales
 	,sum(case when d_moy = 9
 		then cs_ext_list_price* cs_quantity else 0 end) as sep_sales
 	,sum(case when d_moy = 10
 		then cs_ext_list_price* cs_quantity else 0 end) as oct_sales
 	,sum(case when d_moy = 11
 		then cs_ext_list_price* cs_quantity else 0 end) as nov_sales
 	,sum(case when d_moy = 12
 		then cs_ext_list_price* cs_quantity else 0 end) as dec_sales
 	,sum(case when d_moy = 1
 		then cs_net_paid * cs_quantity else 0 end) as jan_net
 	,sum(case when d_moy = 2
 		then cs_net_paid * cs_quantity else 0 end) as feb_net
 	,sum(case when d_moy = 3
 		then cs_net_paid * cs_quantity else 0 end) as mar_net
 	,sum(case when d_moy = 4
 		then cs_net_paid * cs_quantity else 0 end) as apr_net
 	,sum(case when d_moy = 5
 		then cs_net_paid * cs_quantity else 0 end) as may_net
 	,sum(case when d_moy = 6
 		then cs_net_paid * cs_quantity else 0 end) as jun_net
 	,sum(case when d_moy = 7
 		then cs_net_paid * cs_quantity else 0 end) as jul_net
 	,sum(case when d_moy = 8
 		then cs_net_paid * cs_quantity else 0 end) as aug_net
 	,sum(case when d_moy = 9
 		then cs_net_paid * cs_quantity else 0 end) as sep_net
 	,sum(case when d_moy = 10
 		then cs_net_paid * cs_quantity else 0 end) as oct_net
 	,sum(case when d_moy = 11
 		then cs_net_paid * cs_quantity else 0 end) as nov_net
 	,sum(case when d_moy = 12
 		then cs_net_paid * cs_quantity else 0 end) as dec_net
     from
          {db}.catalog_sales
         ,{db}.warehouse
         ,{db}.date_dim
         ,{db}.time_dim
 	 ,{db}.ship_mode
     where
            cs_warehouse_sk =  w_warehouse_sk
        and cs_sold_date_sk = d_date_sk
        and cs_sold_time_sk = t_time_sk
 	and cs_ship_mode_sk = sm_ship_mode_sk
        and d_year = 2002
 	and t_time between 24285 AND 24285+28800
 	and sm_carrier in ('HARMSTORF','USPS')
     group by
        w_warehouse_name
 	,w_warehouse_sq_ft
 	,w_city
 	,w_county
 	,w_state
 	,w_country
       ,d_year
 ) x
 group by
        w_warehouse_name
 	,w_warehouse_sq_ft
 	,w_city
 	,w_county
 	,w_state
 	,w_country
 	,ship_carriers
       ,year
 order by w_warehouse_name
 limit 100""",
    "q67": f"""
select  *
from (select i_category
            ,i_class
            ,i_brand
            ,i_product_name
            ,d_year
            ,d_qoy
            ,d_moy
            ,s_store_id
            ,sumsales
            ,rank() over (partition by i_category order by sumsales desc) rk
      from (select i_category
                  ,i_class
                  ,i_brand
                  ,i_product_name
                  ,d_year
                  ,d_qoy
                  ,d_moy
                  ,s_store_id
                  ,sum(coalesce(ss_sales_price*ss_quantity,0)) sumsales
            from {db}.store_sales
                ,{db}.date_dim
                ,{db}.store
                ,{db}.item
       where  ss_sold_date_sk=d_date_sk
          and ss_item_sk=i_item_sk
          and ss_store_sk = s_store_sk
          and d_month_seq between 1196 and 1196+11
       group by  rollup(i_category, i_class, i_brand, i_product_name, d_year, d_qoy, d_moy,s_store_id))dw1) dw2
where rk <= 100
order by i_category
        ,i_class
        ,i_brand
        ,i_product_name
        ,d_year
        ,d_qoy
        ,d_moy
        ,s_store_id
        ,sumsales
        ,rk
limit 100""",
    "q68": f"""
select  c_last_name
       ,c_first_name
       ,ca_city
       ,bought_city
       ,ss_ticket_number
       ,extended_price
       ,extended_tax
       ,list_price
 from (select ss_ticket_number
             ,ss_customer_sk
             ,ca_city bought_city
             ,sum(ss_ext_sales_price) extended_price
             ,sum(ss_ext_list_price) list_price
             ,sum(ss_ext_tax) extended_tax
       from {db}.store_sales
           ,{db}.date_dim
           ,{db}.store
           ,{db}.household_demographics
           ,{db}.customer_address
       where {db}.store_sales.ss_sold_date_sk = {db}.date_dim.d_date_sk
         and {db}.store_sales.ss_store_sk = {db}.store.s_store_sk
        and {db}.store_sales.ss_hdemo_sk = {db}.household_demographics.hd_demo_sk
        and {db}.store_sales.ss_addr_sk = {db}.customer_address.ca_address_sk
        and {db}.date_dim.d_dom between 1 and 2
        and ({db}.household_demographics.hd_dep_count = 1 or
             {db}.household_demographics.hd_vehicle_count= -1)
        and {db}.date_dim.d_year in (1998,1998+1,1998+2)
        and {db}.store.s_city in ('Bethel','Summit')
       group by ss_ticket_number
               ,ss_customer_sk
               ,ss_addr_sk,ca_city) dn
      ,{db}.customer
      ,{db}.customer_address current_addr
 where ss_customer_sk = c_customer_sk
   and {db}.customer.c_current_addr_sk = current_addr.ca_address_sk
   and current_addr.ca_city <> bought_city
 order by c_last_name
         ,ss_ticket_number
 limit 100""",
    "q69": f"""
select
  cd_gender,
  cd_marital_status,
  cd_education_status,
  count(*) cnt1,
  cd_purchase_estimate,
  count(*) cnt2,
  cd_credit_rating,
  count(*) cnt3
 from
  {db}.customer c,{db}.customer_address ca,{db}.customer_demographics
 where
  c.c_current_addr_sk = ca.ca_address_sk and
  ca_state in ('OK','GA','VA') and
  cd_demo_sk = c.c_current_cdemo_sk and
  exists (select *
          from {db}.store_sales,{db}.date_dim
          where c.c_customer_sk = ss_customer_sk and
                ss_sold_date_sk = d_date_sk and
                d_year = 2004 and
                d_moy between 4 and 4+2) and
   (not exists (select *
            from {db}.web_sales,{db}.date_dim
            where c.c_customer_sk = ws_bill_customer_sk and
                  ws_sold_date_sk = d_date_sk and
                  d_year = 2004 and
                  d_moy between 4 and 4+2) and
    not exists (select *
            from {db}.catalog_sales,{db}.date_dim
            where c.c_customer_sk = cs_ship_customer_sk and
                  cs_sold_date_sk = d_date_sk and
                  d_year = 2004 and
                  d_moy between 4 and 4+2))
 group by cd_gender,
          cd_marital_status,
          cd_education_status,
          cd_purchase_estimate,
          cd_credit_rating
 order by cd_gender,
          cd_marital_status,
          cd_education_status,
          cd_purchase_estimate,
          cd_credit_rating
 limit 100""",
    "q70": f"""
select
    sum(ss_net_profit) as total_sum
   ,s_state
   ,s_county
   ,grouping(s_state)+grouping(s_county) as lochierarchy
   ,rank() over (
 	partition by grouping(s_state)+grouping(s_county),
 	case when grouping(s_county) = 0 then s_state end
 	order by sum(ss_net_profit) desc) as rank_within_parent
 from
    {db}.store_sales
   ,{db}.date_dim       d1
   ,{db}.store
 where
    d1.d_month_seq between 1197 and 1197+11
 and d1.d_date_sk = ss_sold_date_sk
 and s_store_sk  = ss_store_sk
 and s_state in
             ( select s_state
               from  (select s_state as s_state,
 			    rank() over ( partition by s_state order by sum(ss_net_profit) desc) as ranking
                      from   {db}.store_sales, {db}.store, {db}.date_dim
                      where  d_month_seq between 1197 and 1197+11
 			    and d_date_sk = ss_sold_date_sk
 			    and s_store_sk  = ss_store_sk
                      group by s_state
                     ) tmp1
               where ranking <= 5
             )
 group by rollup(s_state,s_county)
 order by
   lochierarchy desc
  ,case when lochierarchy = 0 then s_state end
  ,rank_within_parent
 limit 100""",
    "q71": f"""
select i_brand_id brand_id, i_brand brand,t_hour,t_minute,
 	sum(ext_price) ext_price
 from {db}.item, (select ws_ext_sales_price as ext_price,
                        ws_sold_date_sk as sold_date_sk,
                        ws_item_sk as sold_item_sk,
                        ws_sold_time_sk as time_sk
                 from {db}.web_sales,{db}.date_dim
                 where d_date_sk = ws_sold_date_sk
                   and d_moy=12
                   and d_year=1999
                 union all
                 select cs_ext_sales_price as ext_price,
                        cs_sold_date_sk as sold_date_sk,
                        cs_item_sk as sold_item_sk,
                        cs_sold_time_sk as time_sk
                 from {db}.catalog_sales,{db}.date_dim
                 where d_date_sk = cs_sold_date_sk
                   and d_moy=12
                   and d_year=1999
                 union all
                 select ss_ext_sales_price as ext_price,
                        ss_sold_date_sk as sold_date_sk,
                        ss_item_sk as sold_item_sk,
                        ss_sold_time_sk as time_sk
                 from {db}.store_sales,{db}.date_dim
                 where d_date_sk = ss_sold_date_sk
                   and d_moy=12
                   and d_year=1999
                 ) tmp,{db}.time_dim
 where
   sold_item_sk = i_item_sk
   and i_manager_id=1
   and time_sk = t_time_sk
   and (t_meal_time = 'breakfast' or t_meal_time = 'dinner')
 group by i_brand, i_brand_id,t_hour,t_minute
 order by ext_price desc, i_brand_id
 """,
    "q72": f"""
select  i_item_desc
      ,w_warehouse_name
      ,d1.d_week_seq
      ,sum(case when p_promo_sk is null then 1 else 0 end) no_promo
      ,sum(case when p_promo_sk is not null then 1 else 0 end) promo
      ,count(*) total_cnt
from {db}.catalog_sales
join {db}.inventory on (cs_item_sk = inv_item_sk)
join {db}.warehouse on (w_warehouse_sk=inv_warehouse_sk)
join {db}.item on (i_item_sk = cs_item_sk)
join {db}.customer_demographics on (cs_bill_cdemo_sk = cd_demo_sk)
join {db}.household_demographics on (cs_bill_hdemo_sk = hd_demo_sk)
join {db}.date_dim d1 on (cs_sold_date_sk = d1.d_date_sk)
join {db}.date_dim d2 on (inv_date_sk = d2.d_date_sk)
join {db}.date_dim d3 on (cs_ship_date_sk = d3.d_date_sk)
left outer join {db}.promotion on (cs_promo_sk=p_promo_sk)
left outer join {db}.catalog_returns on (cr_item_sk = cs_item_sk and cr_order_number = cs_order_number)
where d1.d_week_seq = d2.d_week_seq
  and inv_quantity_on_hand < cs_quantity
  and d3.d_date > d1.d_date + interval 5 days
  and hd_buy_potential = '>10000'
  and d1.d_year = 2002
  and cd_marital_status = 'D'
group by i_item_desc,w_warehouse_name,d1.d_week_seq
order by total_cnt desc, i_item_desc, w_warehouse_name, d_week_seq
limit 100""",
    "q73": f"""
select c_last_name
       ,c_first_name
       ,c_salutation
       ,c_preferred_cust_flag
       ,ss_ticket_number
       ,cnt from
   (select ss_ticket_number
          ,ss_customer_sk
          ,count(*) cnt
    from {db}.store_sales,{db}.date_dim,{db}.store,{db}.household_demographics
    where {db}.store_sales.ss_sold_date_sk = {db}.date_dim.d_date_sk
    and {db}.store_sales.ss_store_sk = {db}.store.s_store_sk
    and {db}.store_sales.ss_hdemo_sk = {db}.household_demographics.hd_demo_sk
    and {db}.date_dim.d_dom between 1 and 2
    and ({db}.household_demographics.hd_buy_potential = '501-1000' or
         {db}.household_demographics.hd_buy_potential = 'Unknown')
    and {db}.household_demographics.hd_vehicle_count > 0
    and case when {db}.household_demographics.hd_vehicle_count > 0 then
             {db}.household_demographics.hd_dep_count/ {db}.household_demographics.hd_vehicle_count else null end > 1
    and {db}.date_dim.d_year in (1999,1999+1,1999+2)
    and {db}.store.s_county in ('Franklin Parish','Ziebach County','Luce County','Williamson County')
    group by ss_ticket_number,ss_customer_sk) dj,{db}.customer
    where ss_customer_sk = c_customer_sk
      and cnt between 1 and 5
    order by cnt desc, c_last_name asc""",
    "q74": f"""
with year_total as (
 select c_customer_id customer_id
       ,c_first_name customer_first_name
       ,c_last_name customer_last_name
       ,d_year as year
       ,max(ss_net_paid) year_total
       ,'s' sale_type
 from {db}.customer
     ,{db}.store_sales
     ,{db}.date_dim
 where c_customer_sk = ss_customer_sk
   and ss_sold_date_sk = d_date_sk
   and d_year in (2001,2001+1)
 group by c_customer_id
         ,c_first_name
         ,c_last_name
         ,d_year
 union all
 select c_customer_id customer_id
       ,c_first_name customer_first_name
       ,c_last_name customer_last_name
       ,d_year as year
       ,max(ws_net_paid) year_total
       ,'w' sale_type
 from {db}.customer
     ,{db}.web_sales
     ,{db}.date_dim
 where c_customer_sk = ws_bill_customer_sk
   and ws_sold_date_sk = d_date_sk
   and d_year in (2001,2001+1)
 group by c_customer_id
         ,c_first_name
         ,c_last_name
         ,d_year
         )
  select
        t_s_secyear.customer_id, t_s_secyear.customer_first_name, t_s_secyear.customer_last_name
 from year_total t_s_firstyear
     ,year_total t_s_secyear
     ,year_total t_w_firstyear
     ,year_total t_w_secyear
 where t_s_secyear.customer_id = t_s_firstyear.customer_id
         and t_s_firstyear.customer_id = t_w_secyear.customer_id
         and t_s_firstyear.customer_id = t_w_firstyear.customer_id
         and t_s_firstyear.sale_type = 's'
         and t_w_firstyear.sale_type = 'w'
         and t_s_secyear.sale_type = 's'
         and t_w_secyear.sale_type = 'w'
         and t_s_firstyear.year = 2001
         and t_s_secyear.year = 2001+1
         and t_w_firstyear.year = 2001
         and t_w_secyear.year = 2001+1
         and t_s_firstyear.year_total > 0
         and t_w_firstyear.year_total > 0
         and case when t_w_firstyear.year_total > 0 then t_w_secyear.year_total / t_w_firstyear.year_total else null end
           > case when t_s_firstyear.year_total > 0 then t_s_secyear.year_total / t_s_firstyear.year_total else null end
 order by 3,1,2
limit 100""",
    "q75": f"""
WITH all_sales AS (
 SELECT d_year
       ,i_brand_id
       ,i_class_id
       ,i_category_id
       ,i_manufact_id
       ,SUM(sales_cnt) AS sales_cnt
       ,SUM(sales_amt) AS sales_amt
 FROM (SELECT d_year
             ,i_brand_id
             ,i_class_id
             ,i_category_id
             ,i_manufact_id
             ,cs_quantity - COALESCE(cr_return_quantity,0) AS sales_cnt
             ,cs_ext_sales_price - COALESCE(cr_return_amount,0.0) AS sales_amt
       FROM {db}.catalog_sales JOIN {db}.item ON i_item_sk=cs_item_sk
                          JOIN {db}.date_dim ON d_date_sk=cs_sold_date_sk
                          LEFT JOIN {db}.catalog_returns ON (cs_order_number=cr_order_number
                                                    AND cs_item_sk=cr_item_sk)
       WHERE i_category='Sports'
       UNION
       SELECT d_year
             ,i_brand_id
             ,i_class_id
             ,i_category_id
             ,i_manufact_id
             ,ss_quantity - COALESCE(sr_return_quantity,0) AS sales_cnt
             ,ss_ext_sales_price - COALESCE(sr_return_amt,0.0) AS sales_amt
       FROM {db}.store_sales JOIN {db}.item ON i_item_sk=ss_item_sk
                        JOIN {db}.date_dim ON d_date_sk=ss_sold_date_sk
                        LEFT JOIN {db}.store_returns ON (ss_ticket_number=sr_ticket_number
                                                AND ss_item_sk=sr_item_sk)
       WHERE i_category='Sports'
       UNION
       SELECT d_year
             ,i_brand_id
             ,i_class_id
             ,i_category_id
             ,i_manufact_id
             ,ws_quantity - COALESCE(wr_return_quantity,0) AS sales_cnt
             ,ws_ext_sales_price - COALESCE(wr_return_amt,0.0) AS sales_amt
       FROM {db}.web_sales JOIN {db}.item ON i_item_sk=ws_item_sk
                      JOIN {db}.date_dim ON d_date_sk=ws_sold_date_sk
                      LEFT JOIN {db}.web_returns ON (ws_order_number=wr_order_number
                                            AND ws_item_sk=wr_item_sk)
       WHERE i_category='Sports') sales_detail
 GROUP BY d_year, i_brand_id, i_class_id, i_category_id, i_manufact_id)
 SELECT  prev_yr.d_year AS prev_year
                          ,curr_yr.d_year AS year
                          ,curr_yr.i_brand_id
                          ,curr_yr.i_class_id
                          ,curr_yr.i_category_id
                          ,curr_yr.i_manufact_id
                          ,prev_yr.sales_cnt AS prev_yr_cnt
                          ,curr_yr.sales_cnt AS curr_yr_cnt
                          ,curr_yr.sales_cnt-prev_yr.sales_cnt AS sales_cnt_diff
                          ,curr_yr.sales_amt-prev_yr.sales_amt AS sales_amt_diff
 FROM all_sales curr_yr, all_sales prev_yr
 WHERE curr_yr.i_brand_id=prev_yr.i_brand_id
   AND curr_yr.i_class_id=prev_yr.i_class_id
   AND curr_yr.i_category_id=prev_yr.i_category_id
   AND curr_yr.i_manufact_id=prev_yr.i_manufact_id
   AND curr_yr.d_year=2001
   AND prev_yr.d_year=2001-1
   AND CAST(curr_yr.sales_cnt AS DECIMAL(17,2))/CAST(prev_yr.sales_cnt AS DECIMAL(17,2))<0.9
 ORDER BY sales_cnt_diff,sales_amt_diff
 limit 100""",
    "q76": f"""
select  channel, col_name, d_year, d_qoy, i_category, COUNT(*) sales_cnt, SUM(ext_sales_price) sales_amt FROM (
        SELECT 'store' as channel, 'ss_cdemo_sk' col_name, d_year, d_qoy, i_category, ss_ext_sales_price ext_sales_price
         FROM {db}.store_sales, {db}.item, {db}.date_dim
         WHERE ss_cdemo_sk IS NULL
           AND ss_sold_date_sk=d_date_sk
           AND ss_item_sk=i_item_sk
        UNION ALL
        SELECT 'web' as channel, 'ws_ship_hdemo_sk' col_name, d_year, d_qoy, i_category, ws_ext_sales_price ext_sales_price
         FROM {db}.web_sales, {db}.item, {db}.date_dim
         WHERE ws_ship_hdemo_sk IS NULL
           AND ws_sold_date_sk=d_date_sk
           AND ws_item_sk=i_item_sk
        UNION ALL
        SELECT 'catalog' as channel, 'cs_ship_customer_sk' col_name, d_year, d_qoy, i_category, cs_ext_sales_price ext_sales_price
         FROM {db}.catalog_sales, {db}.item, {db}.date_dim
         WHERE cs_ship_customer_sk IS NULL
           AND cs_sold_date_sk=d_date_sk
           AND cs_item_sk=i_item_sk) foo
GROUP BY channel, col_name, d_year, d_qoy, i_category
ORDER BY channel, col_name, d_year, d_qoy, i_category
limit 100""",
    "q77": f"""
with ss as
 (select s_store_sk,
         sum(ss_ext_sales_price) as sales,
         sum(ss_net_profit) as profit
 from {db}.store_sales,
      {db}.date_dim,
      {db}.store
 where ss_sold_date_sk = d_date_sk
       and d_date between cast('2001-08-27' as date)
                  and (cast('2001-08-27' as date) +  INTERVAL 30 days)
       and ss_store_sk = s_store_sk
 group by s_store_sk)
 ,
 sr as
 (select s_store_sk,
         sum(sr_return_amt) as returns,
         sum(sr_net_loss) as profit_loss
 from {db}.store_returns,
      {db}.date_dim,
      {db}.store
 where sr_returned_date_sk = d_date_sk
       and d_date between cast('2001-08-27' as date)
                  and (cast('2001-08-27' as date) +  INTERVAL 30 days)
       and sr_store_sk = s_store_sk
 group by s_store_sk),
 cs as
 (select cs_call_center_sk,
        sum(cs_ext_sales_price) as sales,
        sum(cs_net_profit) as profit
 from {db}.catalog_sales,
      {db}.date_dim
 where cs_sold_date_sk = d_date_sk
       and d_date between cast('2001-08-27' as date)
                  and (cast('2001-08-27' as date) +  INTERVAL 30 days)
 group by cs_call_center_sk
 ),
 cr as
 (select cr_call_center_sk,
         sum(cr_return_amount) as returns,
         sum(cr_net_loss) as profit_loss
 from {db}.catalog_returns,
      {db}.date_dim
 where cr_returned_date_sk = d_date_sk
       and d_date between cast('2001-08-27' as date)
                  and (cast('2001-08-27' as date) +  INTERVAL 30 days)
 group by cr_call_center_sk
 ),
 ws as
 ( select wp_web_page_sk,
        sum(ws_ext_sales_price) as sales,
        sum(ws_net_profit) as profit
 from {db}.web_sales,
      {db}.date_dim,
      {db}.web_page
 where ws_sold_date_sk = d_date_sk
       and d_date between cast('2001-08-27' as date)
                  and (cast('2001-08-27' as date) +  INTERVAL 30 days)
       and ws_web_page_sk = wp_web_page_sk
 group by wp_web_page_sk),
 wr as
 (select wp_web_page_sk,
        sum(wr_return_amt) as returns,
        sum(wr_net_loss) as profit_loss
 from {db}.web_returns,
      {db}.date_dim,
      {db}.web_page
 where wr_returned_date_sk = d_date_sk
       and d_date between cast('2001-08-27' as date)
                  and (cast('2001-08-27' as date) +  INTERVAL 30 days)
       and wr_web_page_sk = wp_web_page_sk
 group by wp_web_page_sk)
  select  channel
        , id
        , sum(sales) as sales
        , sum(returns) as returns
        , sum(profit) as profit
 from
 (select 'store channel' as channel
        , ss.s_store_sk as id
        , sales
        , coalesce(returns, 0) as returns
        , (profit - coalesce(profit_loss,0)) as profit
 from   ss left join sr
        on  ss.s_store_sk = sr.s_store_sk
 union all
 select 'catalog channel' as channel
        , cs_call_center_sk as id
        , sales
        , returns
        , (profit - profit_loss) as profit
 from  cs
       , cr
 union all
 select 'web channel' as channel
        , ws.wp_web_page_sk as id
        , sales
        , coalesce(returns, 0) returns
        , (profit - coalesce(profit_loss,0)) as profit
 from   ws left join wr
        on  ws.wp_web_page_sk = wr.wp_web_page_sk
 ) x
 group by rollup (channel, id)
 order by channel
         ,id
 limit 100""",
    "q78": f"""
with ws as
  (select d_year AS ws_sold_year, ws_item_sk,
    ws_bill_customer_sk ws_customer_sk,
    sum(ws_quantity) ws_qty,
    sum(ws_wholesale_cost) ws_wc,
    sum(ws_sales_price) ws_sp
   from {db}.web_sales
   left join {db}.web_returns on wr_order_number=ws_order_number and ws_item_sk=wr_item_sk
   join {db}.date_dim on ws_sold_date_sk = d_date_sk
   where wr_order_number is null
   group by d_year, ws_item_sk, ws_bill_customer_sk
   ),
cs as
  (select d_year AS cs_sold_year, cs_item_sk,
    cs_bill_customer_sk cs_customer_sk,
    sum(cs_quantity) cs_qty,
    sum(cs_wholesale_cost) cs_wc,
    sum(cs_sales_price) cs_sp
   from {db}.catalog_sales
   left join {db}.catalog_returns on cr_order_number=cs_order_number and cs_item_sk=cr_item_sk
   join {db}.date_dim on cs_sold_date_sk = d_date_sk
   where cr_order_number is null
   group by d_year, cs_item_sk, cs_bill_customer_sk
   ),
ss as
  (select d_year AS ss_sold_year, ss_item_sk,
    ss_customer_sk,
    sum(ss_quantity) ss_qty,
    sum(ss_wholesale_cost) ss_wc,
    sum(ss_sales_price) ss_sp
   from {db}.store_sales
   left join {db}.store_returns on sr_ticket_number=ss_ticket_number and ss_item_sk=sr_item_sk
   join {db}.date_dim on ss_sold_date_sk = d_date_sk
   where sr_ticket_number is null
   group by d_year, ss_item_sk, ss_customer_sk
   )
 select
ss_sold_year, ss_item_sk, ss_customer_sk,
round(ss_qty/(coalesce(ws_qty,0)+coalesce(cs_qty,0)),2) ratio,
ss_qty store_qty, ss_wc store_wholesale_cost, ss_sp store_sales_price,
coalesce(ws_qty,0)+coalesce(cs_qty,0) other_chan_qty,
coalesce(ws_wc,0)+coalesce(cs_wc,0) other_chan_wholesale_cost,
coalesce(ws_sp,0)+coalesce(cs_sp,0) other_chan_sales_price
from ss
left join ws on (ws_sold_year=ss_sold_year and ws_item_sk=ss_item_sk and ws_customer_sk=ss_customer_sk)
left join cs on (cs_sold_year=ss_sold_year and cs_item_sk=ss_item_sk and cs_customer_sk=ss_customer_sk)
where (coalesce(ws_qty,0)>0 or coalesce(cs_qty, 0)>0) and ss_sold_year=2002
order by
  ss_sold_year, ss_item_sk, ss_customer_sk,
  ss_qty desc, ss_wc desc, ss_sp desc,
  other_chan_qty,
  other_chan_wholesale_cost,
  other_chan_sales_price,
  ratio
limit 100""",
    "q79": f"""
select
  c_last_name,c_first_name,substr(s_city,1,30),ss_ticket_number,amt,profit
  from
   (select ss_ticket_number
          ,ss_customer_sk
          ,{db}.store.s_city
          ,sum(ss_coupon_amt) amt
          ,sum(ss_net_profit) profit
    from {db}.store_sales,{db}.date_dim,{db}.store,{db}.household_demographics
    where {db}.store_sales.ss_sold_date_sk = {db}.date_dim.d_date_sk
    and {db}.store_sales.ss_store_sk = {db}.store.s_store_sk
    and {db}.store_sales.ss_hdemo_sk = {db}.household_demographics.hd_demo_sk
    and ({db}.household_demographics.hd_dep_count = 0 or {db}.household_demographics.hd_vehicle_count > 0)
    and {db}.date_dim.d_dow = 1
    and {db}.date_dim.d_year in (2000,2000+1,2000+2)
    and {db}.store.s_number_employees between 200 and 295
    group by ss_ticket_number,ss_customer_sk,ss_addr_sk,{db}.store.s_city) ms,{db}.customer
    where ss_customer_sk = c_customer_sk
 order by c_last_name,c_first_name,substr(s_city,1,30), profit
limit 100""",
    "q80": f"""
with ssr as
 (select  s_store_id as store_id,
          sum(ss_ext_sales_price) as sales,
          sum(coalesce(sr_return_amt, 0)) as returns,
          sum(ss_net_profit - coalesce(sr_net_loss, 0)) as profit
  from {db}.store_sales left outer join {db}.store_returns on
         (ss_item_sk = sr_item_sk and ss_ticket_number = sr_ticket_number),
     {db}.date_dim,
     {db}.store,
     {db}.item,
     {db}.promotion
 where ss_sold_date_sk = d_date_sk
       and d_date between cast('1999-08-12' as date)
                  and (cast('1999-08-12' as date) +  INTERVAL 60 days)
       and ss_store_sk = s_store_sk
       and ss_item_sk = i_item_sk
       and i_current_price > 50
       and ss_promo_sk = p_promo_sk
       and p_channel_tv = 'N'
 group by s_store_id)
 ,
 csr as
 (select  cp_catalog_page_id as catalog_page_id,
          sum(cs_ext_sales_price) as sales,
          sum(coalesce(cr_return_amount, 0)) as returns,
          sum(cs_net_profit - coalesce(cr_net_loss, 0)) as profit
  from {db}.catalog_sales left outer join {db}.catalog_returns on
         (cs_item_sk = cr_item_sk and cs_order_number = cr_order_number),
     {db}.date_dim,
     {db}.catalog_page,
     {db}.item,
     {db}.promotion
 where cs_sold_date_sk = d_date_sk
       and d_date between cast('1999-08-12' as date)
                  and (cast('1999-08-12' as date) +  INTERVAL 60 days)
        and cs_catalog_page_sk = cp_catalog_page_sk
       and cs_item_sk = i_item_sk
       and i_current_price > 50
       and cs_promo_sk = p_promo_sk
       and p_channel_tv = 'N'
group by cp_catalog_page_id)
 ,
 wsr as
 (select  web_site_id,
          sum(ws_ext_sales_price) as sales,
          sum(coalesce(wr_return_amt, 0)) as returns,
          sum(ws_net_profit - coalesce(wr_net_loss, 0)) as profit
  from {db}.web_sales left outer join {db}.web_returns on
         (ws_item_sk = wr_item_sk and ws_order_number = wr_order_number),
     {db}.date_dim,
     {db}.web_site,
     {db}.item,
     {db}.promotion
 where ws_sold_date_sk = d_date_sk
       and d_date between cast('1999-08-12' as date)
                  and (cast('1999-08-12' as date) +  INTERVAL 60 days)
        and ws_web_site_sk = web_site_sk
       and ws_item_sk = i_item_sk
       and i_current_price > 50
       and ws_promo_sk = p_promo_sk
       and p_channel_tv = 'N'
group by web_site_id)
  select  channel
        , id
        , sum(sales) as sales
        , sum(returns) as returns
        , sum(profit) as profit
 from
 (select 'store channel' as channel
        , 'store' || store_id as id
        , sales
        , returns
        , profit
 from   ssr
 union all
 select 'catalog channel' as channel
        , 'catalog_page' || catalog_page_id as id
        , sales
        , returns
        , profit
 from  csr
 union all
 select 'web channel' as channel
        , 'web_site' || web_site_id as id
        , sales
        , returns
        , profit
 from   wsr
 ) x
 group by rollup (channel, id)
 order by channel
         ,id
 limit 100""",
    "q81": f"""
with customer_total_return as
 (select cr_returning_customer_sk as ctr_customer_sk
        ,ca_state as ctr_state,
 	sum(cr_return_amt_inc_tax) as ctr_total_return
 from {db}.catalog_returns
     ,{db}.date_dim
     ,{db}.customer_address
 where cr_returned_date_sk = d_date_sk
   and d_year =2001
   and cr_returning_addr_sk = ca_address_sk
 group by cr_returning_customer_sk
         ,ca_state )
  select  c_customer_id,c_salutation,c_first_name,c_last_name,ca_street_number,ca_street_name
                   ,ca_street_type,ca_suite_number,ca_city,ca_county,ca_state,ca_zip,ca_country,ca_gmt_offset
                  ,ca_location_type,ctr_total_return
 from customer_total_return ctr1
     ,{db}.customer_address
     ,{db}.customer
 where ctr1.ctr_total_return > (select avg(ctr_total_return)*1.2
 			  from customer_total_return ctr2
                  	  where ctr1.ctr_state = ctr2.ctr_state)
       and ca_address_sk = c_current_addr_sk
       and ca_state = 'NC'
       and ctr1.ctr_customer_sk = c_customer_sk
 order by c_customer_id,c_salutation,c_first_name,c_last_name,ca_street_number,ca_street_name
                   ,ca_street_type,ca_suite_number,ca_city,ca_county,ca_state,ca_zip,ca_country,ca_gmt_offset
                  ,ca_location_type,ctr_total_return
 limit 100""",
    "q82": f"""
select  i_item_id
       ,i_item_desc
       ,i_current_price
 from {db}.item, {db}.inventory, {db}.date_dim, {db}.store_sales
 where i_current_price between 82 and 82+30
 and inv_item_sk = i_item_sk
 and d_date_sk=inv_date_sk
 and d_date between cast('2002-03-10' as date) and (cast('2002-03-10' as date) +  INTERVAL 60 days)
 and i_manufact_id in (941,920,105,693)
 and inv_quantity_on_hand between 100 and 500
 and ss_item_sk = i_item_sk
 group by i_item_id,i_item_desc,i_current_price
 order by i_item_id
 limit 100""",
    "q83": f"""
with sr_items as
 (select i_item_id item_id,
        sum(sr_return_quantity) sr_item_qty
 from {db}.store_returns,
      {db}.item,
      {db}.date_dim
 where sr_item_sk = i_item_sk
 and   d_date    in
	(select d_date
	from {db}.date_dim
	where d_week_seq in
		(select d_week_seq
		from {db}.date_dim
	  where d_date in ('1999-04-14','1999-09-28','1999-11-12')))
 and   sr_returned_date_sk   = d_date_sk
 group by i_item_id),
 cr_items as
 (select i_item_id item_id,
        sum(cr_return_quantity) cr_item_qty
 from {db}.catalog_returns,
      {db}.item,
      {db}.date_dim
 where cr_item_sk = i_item_sk
 and   d_date    in
	(select d_date
	from {db}.date_dim
	where d_week_seq in
		(select d_week_seq
		from {db}.date_dim
	  where d_date in ('1999-04-14','1999-09-28','1999-11-12')))
 and   cr_returned_date_sk   = d_date_sk
 group by i_item_id),
 wr_items as
 (select i_item_id item_id,
        sum(wr_return_quantity) wr_item_qty
 from {db}.web_returns,
      {db}.item,
      {db}.date_dim
 where wr_item_sk = i_item_sk
 and   d_date    in
	(select d_date
	from {db}.date_dim
	where d_week_seq in
		(select d_week_seq
		from {db}.date_dim
		where d_date in ('1999-04-14','1999-09-28','1999-11-12')))
 and   wr_returned_date_sk   = d_date_sk
 group by i_item_id)
  select  sr_items.item_id
       ,sr_item_qty
       ,sr_item_qty/(sr_item_qty+cr_item_qty+wr_item_qty)/3.0 * 100 sr_dev
       ,cr_item_qty
       ,cr_item_qty/(sr_item_qty+cr_item_qty+wr_item_qty)/3.0 * 100 cr_dev
       ,wr_item_qty
       ,wr_item_qty/(sr_item_qty+cr_item_qty+wr_item_qty)/3.0 * 100 wr_dev
       ,(sr_item_qty+cr_item_qty+wr_item_qty)/3.0 average
 from sr_items
     ,cr_items
     ,wr_items
 where sr_items.item_id=cr_items.item_id
   and sr_items.item_id=wr_items.item_id
 order by sr_items.item_id
         ,sr_item_qty
 limit 100""",
    "q84": f"""
select  c_customer_id as customer_id
       , coalesce(c_last_name,'') || ', ' || coalesce(c_first_name,'') as customername
 from {db}.customer
     ,{db}.customer_address
     ,{db}.customer_demographics
     ,{db}.household_demographics
     ,{db}.income_band
     ,{db}.store_returns
 where ca_city	        =  'Antioch'
   and c_current_addr_sk = ca_address_sk
   and ib_lower_bound   >=  55019
   and ib_upper_bound   <=  55019 + 50000
   and ib_income_band_sk = hd_income_band_sk
   and cd_demo_sk = c_current_cdemo_sk
   and hd_demo_sk = c_current_hdemo_sk
   and sr_cdemo_sk = cd_demo_sk
 order by c_customer_id
 limit 100""",
    "q85": f"""
select  substr(r_reason_desc,1,20)
       ,avg(ws_quantity)
       ,avg(wr_refunded_cash)
       ,avg(wr_fee)
 from {db}.web_sales, {db}.web_returns, {db}.web_page, {db}.customer_demographics cd1,
      {db}.customer_demographics cd2, {db}.customer_address, {db}.date_dim, {db}.reason
 where ws_web_page_sk = wp_web_page_sk
   and ws_item_sk = wr_item_sk
   and ws_order_number = wr_order_number
   and ws_sold_date_sk = d_date_sk and d_year = 2001
   and cd1.cd_demo_sk = wr_refunded_cdemo_sk
   and cd2.cd_demo_sk = wr_returning_cdemo_sk
   and ca_address_sk = wr_refunded_addr_sk
   and r_reason_sk = wr_reason_sk
   and
   (
    (
     cd1.cd_marital_status = 'S'
     and
     cd1.cd_marital_status = cd2.cd_marital_status
     and
     cd1.cd_education_status = '2 yr Degree'
     and
     cd1.cd_education_status = cd2.cd_education_status
     and
     ws_sales_price between 100.00 and 150.00
    )
   or
    (
     cd1.cd_marital_status = 'D'
     and
     cd1.cd_marital_status = cd2.cd_marital_status
     and
     cd1.cd_education_status = 'Advanced Degree'
     and
     cd1.cd_education_status = cd2.cd_education_status
     and
     ws_sales_price between 50.00 and 100.00
    )
   or
    (
     cd1.cd_marital_status = 'W'
     and
     cd1.cd_marital_status = cd2.cd_marital_status
     and
     cd1.cd_education_status = '4 yr Degree'
     and
     cd1.cd_education_status = cd2.cd_education_status
     and
     ws_sales_price between 150.00 and 200.00
    )
   )
   and
   (
    (
     ca_country = 'United States'
     and
     ca_state in ('OK', 'TX', 'MO')
     and ws_net_profit between 100 and 200
    )
    or
    (
     ca_country = 'United States'
     and
     ca_state in ('GA', 'KS', 'NC')
     and ws_net_profit between 150 and 300
    )
    or
    (
     ca_country = 'United States'
     and
     ca_state in ('VA', 'WI', 'WV')
     and ws_net_profit between 50 and 250
    )
   )
group by r_reason_desc
order by substr(r_reason_desc,1,20)
        ,avg(ws_quantity)
        ,avg(wr_refunded_cash)
        ,avg(wr_fee)
limit 100""",
    "q86": f"""
select
    sum(ws_net_paid) as total_sum
   ,i_category
   ,i_class
   ,grouping(i_category)+grouping(i_class) as lochierarchy
   ,rank() over (
 	partition by grouping(i_category)+grouping(i_class),
 	case when grouping(i_class) = 0 then i_category end
 	order by sum(ws_net_paid) desc) as rank_within_parent
 from
    {db}.web_sales
   ,{db}.date_dim       d1
   ,{db}.item
 where
    d1.d_month_seq between 1215 and 1215+11
 and d1.d_date_sk = ws_sold_date_sk
 and i_item_sk  = ws_item_sk
 group by rollup(i_category,i_class)
 order by
   lochierarchy desc,
   case when lochierarchy = 0 then i_category end,
   rank_within_parent
 limit 100""",
    "q87": f"""
select count(*)
from ((select distinct c_last_name, c_first_name, d_date
       from {db}.store_sales, {db}.date_dim, {db}.customer
       where store_sales.ss_sold_date_sk = {db}.date_dim.d_date_sk
         and store_sales.ss_customer_sk = {db}.customer.c_customer_sk
         and d_month_seq between 1221 and 1221+11)
       except
      (select distinct c_last_name, c_first_name, d_date
       from {db}.catalog_sales, {db}.date_dim, {db}.customer
       where {db}.catalog_sales.cs_sold_date_sk = {db}.date_dim.d_date_sk
         and {db}.catalog_sales.cs_bill_customer_sk = {db}.customer.c_customer_sk
         and d_month_seq between 1221 and 1221+11)
       except
      (select distinct c_last_name, c_first_name, d_date
       from {db}.web_sales, {db}.date_dim, {db}.customer
       where {db}.web_sales.ws_sold_date_sk = {db}.date_dim.d_date_sk
         and {db}.web_sales.ws_bill_customer_sk = {db}.customer.c_customer_sk
         and d_month_seq between 1221 and 1221+11)
) cool_cust""",
    "q88": f"""
select  *
from
 (select count(*) h8_30_to_9
 from {db}.store_sales, {db}.household_demographics , {db}.time_dim, {db}.store
 where ss_sold_time_sk = {db}.time_dim.t_time_sk
     and ss_hdemo_sk = {db}.household_demographics.hd_demo_sk
     and ss_store_sk = s_store_sk
     and {db}.time_dim.t_hour = 8
     and {db}.time_dim.t_minute >= 30
     and (({db}.household_demographics.hd_dep_count = 1 and {db}.household_demographics.hd_vehicle_count<=1+2) or
          ({db}.household_demographics.hd_dep_count = 2 and {db}.household_demographics.hd_vehicle_count<=2+2) or
          ({db}.household_demographics.hd_dep_count = 0 and {db}.household_demographics.hd_vehicle_count<=0+2))
     and {db}.store.s_store_name = 'ese') s1,
 (select count(*) h9_to_9_30
 from {db}.store_sales, {db}.household_demographics , {db}.time_dim, {db}.store
 where ss_sold_time_sk = {db}.time_dim.t_time_sk
     and ss_hdemo_sk = {db}.household_demographics.hd_demo_sk
     and ss_store_sk = s_store_sk
     and {db}.time_dim.t_hour = 9
     and {db}.time_dim.t_minute < 30
     and (({db}.household_demographics.hd_dep_count = 1 and {db}.household_demographics.hd_vehicle_count<=1+2) or
          ({db}.household_demographics.hd_dep_count = 2 and {db}.household_demographics.hd_vehicle_count<=2+2) or
          ({db}.household_demographics.hd_dep_count = 0 and {db}.household_demographics.hd_vehicle_count<=0+2))
     and {db}.store.s_store_name = 'ese') s2,
 (select count(*) h9_30_to_10
 from {db}.store_sales, {db}.household_demographics , {db}.time_dim, {db}.store
 where ss_sold_time_sk = {db}.time_dim.t_time_sk
     and ss_hdemo_sk = {db}.household_demographics.hd_demo_sk
     and ss_store_sk = s_store_sk
     and {db}.time_dim.t_hour = 9
     and {db}.time_dim.t_minute >= 30
     and (({db}.household_demographics.hd_dep_count = 1 and {db}.household_demographics.hd_vehicle_count<=1+2) or
          ({db}.household_demographics.hd_dep_count = 2 and {db}.household_demographics.hd_vehicle_count<=2+2) or
          ({db}.household_demographics.hd_dep_count = 0 and {db}.household_demographics.hd_vehicle_count<=0+2))
     and {db}.store.s_store_name = 'ese') s3,
 (select count(*) h10_to_10_30
 from {db}.store_sales, {db}.household_demographics , {db}.time_dim, {db}.store
 where ss_sold_time_sk = {db}.time_dim.t_time_sk
     and ss_hdemo_sk = {db}.household_demographics.hd_demo_sk
     and ss_store_sk = s_store_sk
     and {db}.time_dim.t_hour = 10
     and {db}.time_dim.t_minute < 30
     and (({db}.household_demographics.hd_dep_count = 1 and {db}.household_demographics.hd_vehicle_count<=1+2) or
          ({db}.household_demographics.hd_dep_count = 2 and {db}.household_demographics.hd_vehicle_count<=2+2) or
          ({db}.household_demographics.hd_dep_count = 0 and {db}.household_demographics.hd_vehicle_count<=0+2))
     and {db}.store.s_store_name = 'ese') s4,
 (select count(*) h10_30_to_11
 from {db}.store_sales, {db}.household_demographics , {db}.time_dim, {db}.store
 where ss_sold_time_sk = {db}.time_dim.t_time_sk
     and ss_hdemo_sk = {db}.household_demographics.hd_demo_sk
     and ss_store_sk = s_store_sk
     and {db}.time_dim.t_hour = 10
     and {db}.time_dim.t_minute >= 30
     and (({db}.household_demographics.hd_dep_count = 1 and {db}.household_demographics.hd_vehicle_count<=1+2) or
          ({db}.household_demographics.hd_dep_count = 2 and {db}.household_demographics.hd_vehicle_count<=2+2) or
          ({db}.household_demographics.hd_dep_count = 0 and {db}.household_demographics.hd_vehicle_count<=0+2))
     and {db}.store.s_store_name = 'ese') s5,
 (select count(*) h11_to_11_30
 from {db}.store_sales, {db}.household_demographics , {db}.time_dim, {db}.store
 where ss_sold_time_sk = {db}.time_dim.t_time_sk
     and ss_hdemo_sk = {db}.household_demographics.hd_demo_sk
     and ss_store_sk = s_store_sk
     and {db}.time_dim.t_hour = 11
     and {db}.time_dim.t_minute < 30
     and (({db}.household_demographics.hd_dep_count = 1 and {db}.household_demographics.hd_vehicle_count<=1+2) or
          ({db}.household_demographics.hd_dep_count = 2 and {db}.household_demographics.hd_vehicle_count<=2+2) or
          ({db}.household_demographics.hd_dep_count = 0 and {db}.household_demographics.hd_vehicle_count<=0+2))
     and {db}.store.s_store_name = 'ese') s6,
 (select count(*) h11_30_to_12
 from {db}.store_sales, {db}.household_demographics , {db}.time_dim, {db}.store
 where ss_sold_time_sk = {db}.time_dim.t_time_sk
     and ss_hdemo_sk = {db}.household_demographics.hd_demo_sk
     and ss_store_sk = s_store_sk
     and {db}.time_dim.t_hour = 11
     and {db}.time_dim.t_minute >= 30
     and (({db}.household_demographics.hd_dep_count = 1 and {db}.household_demographics.hd_vehicle_count<=1+2) or
          ({db}.household_demographics.hd_dep_count = 2 and {db}.household_demographics.hd_vehicle_count<=2+2) or
          ({db}.household_demographics.hd_dep_count = 0 and {db}.household_demographics.hd_vehicle_count<=0+2))
     and {db}.store.s_store_name = 'ese') s7,
 (select count(*) h12_to_12_30
 from {db}.store_sales, {db}.household_demographics , {db}.time_dim, {db}.store
 where ss_sold_time_sk = {db}.time_dim.t_time_sk
     and ss_hdemo_sk = {db}.household_demographics.hd_demo_sk
     and ss_store_sk = s_store_sk
     and {db}.time_dim.t_hour = 12
     and {db}.time_dim.t_minute < 30
     and (({db}.household_demographics.hd_dep_count = 1 and {db}.household_demographics.hd_vehicle_count<=1+2) or
          ({db}.household_demographics.hd_dep_count = 2 and {db}.household_demographics.hd_vehicle_count<=2+2) or
          ({db}.household_demographics.hd_dep_count = 0 and {db}.household_demographics.hd_vehicle_count<=0+2))
     and {db}.store.s_store_name = 'ese') s8""",
    "q89": f"""
select  *
from(
select i_category, i_class, i_brand,
       s_store_name, s_company_name,
       d_moy,
       sum(ss_sales_price) sum_sales,
       avg(sum(ss_sales_price)) over
         (partition by i_category, i_brand, s_store_name, s_company_name)
         avg_monthly_sales
from {db}.item, {db}.store_sales, {db}.date_dim, {db}.store
where ss_item_sk = i_item_sk and
      ss_sold_date_sk = d_date_sk and
      ss_store_sk = s_store_sk and
      d_year in (2001) and
        ((i_category in ('Women','Music','Home') and
          i_class in ('fragrances','pop','bedding')
         )
      or (i_category in ('Books','Men','Children') and
          i_class in ('home repair','sports-apparel','infants')
        ))
group by i_category, i_class, i_brand,
         s_store_name, s_company_name, d_moy) tmp1
where case when (avg_monthly_sales <> 0) then (abs(sum_sales - avg_monthly_sales) / avg_monthly_sales) else null end > 0.1
order by sum_sales - avg_monthly_sales, s_store_name
limit 100""",
    "q90": f"""
select  cast(amc as decimal(15,4))/cast(pmc as decimal(15,4)) am_pm_ratio
 from ( select count(*) amc
       from {db}.web_sales, {db}.household_demographics , {db}.time_dim, {db}.web_page
       where ws_sold_time_sk = {db}.time_dim.t_time_sk
         and ws_ship_hdemo_sk = {db}.household_demographics.hd_demo_sk
         and ws_web_page_sk = {db}.web_page.wp_web_page_sk
         and {db}.time_dim.t_hour between 8 and 8+1
         and {db}.household_demographics.hd_dep_count = 4
         and {db}.web_page.wp_char_count between 5000 and 5200) at,
      ( select count(*) pmc
       from {db}.web_sales, {db}.household_demographics , {db}.time_dim, {db}.web_page
       where ws_sold_time_sk = {db}.time_dim.t_time_sk
         and ws_ship_hdemo_sk = {db}.household_demographics.hd_demo_sk
         and ws_web_page_sk = {db}.web_page.wp_web_page_sk
         and {db}.time_dim.t_hour between 19 and 19+1
         and {db}.household_demographics.hd_dep_count = 4
         and {db}.web_page.wp_char_count between 5000 and 5200) pt
 order by am_pm_ratio
 limit 100""",
    "q91": f"""
select
        cc_call_center_id Call_Center,
        cc_name Call_Center_Name,
        cc_manager Manager,
        sum(cr_net_loss) Returns_Loss
from
        {db}.call_center,
        {db}.catalog_returns,
        {db}.date_dim,
        {db}.customer,
        {db}.customer_address,
        {db}.customer_demographics,
        {db}.household_demographics
where
        cr_call_center_sk       = cc_call_center_sk
and     cr_returned_date_sk     = d_date_sk
and     cr_returning_customer_sk= c_customer_sk
and     cd_demo_sk              = c_current_cdemo_sk
and     hd_demo_sk              = c_current_hdemo_sk
and     ca_address_sk           = c_current_addr_sk
and     d_year                  = 2002
and     d_moy                   = 11
and     ( (cd_marital_status       = 'M' and cd_education_status     = 'Unknown')
        or(cd_marital_status       = 'W' and cd_education_status     = 'Advanced Degree'))
and     hd_buy_potential like '5001-10000%'
and     ca_gmt_offset           = -6
group by cc_call_center_id,cc_name,cc_manager,cd_marital_status,cd_education_status
order by sum(cr_net_loss) desc""",
    "q92": f"""
select
   sum(ws_ext_discount_amt)  as `Excess Discount Amount`
from
    {db}.web_sales
   ,{db}.item
   ,{db}.date_dim
where
i_manufact_id = 561
and i_item_sk = ws_item_sk
and d_date between '2001-03-13' and
        (cast('2001-03-13' as date) + INTERVAL 90 days)
and d_date_sk = ws_sold_date_sk
and ws_ext_discount_amt
     > (
         SELECT
            1.3 * avg(ws_ext_discount_amt)
         FROM
            {db}.web_sales
           ,{db}.date_dim
         WHERE
              ws_item_sk = i_item_sk
          and d_date between '2001-03-13' and
                             (cast('2001-03-13' as date) + INTERVAL 90 days)
          and d_date_sk = ws_sold_date_sk
      )
order by sum(ws_ext_discount_amt)
limit 100""",
    "q93": f"""
select  ss_customer_sk
            ,sum(act_sales) sumsales
      from (select ss_item_sk
                  ,ss_ticket_number
                  ,ss_customer_sk
                  ,case when sr_return_quantity is not null then (ss_quantity-sr_return_quantity)*ss_sales_price
                                                            else (ss_quantity*ss_sales_price) end act_sales
            from {db}.store_sales left outer join {db}.store_returns on (sr_item_sk = ss_item_sk
                                                               and sr_ticket_number = ss_ticket_number)
                ,{db}.reason
            where sr_reason_sk = r_reason_sk
              and r_reason_desc = 'reason 64') t
      group by ss_customer_sk
      order by sumsales, ss_customer_sk
limit 100""",
    "q94": f"""
select
   count(distinct ws_order_number) as `order count`
  ,sum(ws_ext_ship_cost) as `total shipping cost`
  ,sum(ws_net_profit) as `total net profit`
from
   {db}.web_sales ws1
  ,{db}.date_dim
  ,{db}.customer_address
  ,{db}.web_site
where
    d_date between '2001-5-01' and
           (cast('2001-5-01' as date) + INTERVAL 60 days)
and ws1.ws_ship_date_sk = d_date_sk
and ws1.ws_ship_addr_sk = ca_address_sk
and ca_state = 'TX'
and ws1.ws_web_site_sk = web_site_sk
and web_company_name = 'pri'
and exists (select *
            from {db}.web_sales ws2
            where ws1.ws_order_number = ws2.ws_order_number
              and ws1.ws_warehouse_sk <> ws2.ws_warehouse_sk)
and not exists(select *
               from {db}.web_returns wr1
               where ws1.ws_order_number = wr1.wr_order_number)
order by count(distinct ws_order_number)
limit 100""",
    "q95": f"""
with ws_wh as
(select ws1.ws_order_number,ws1.ws_warehouse_sk wh1,ws2.ws_warehouse_sk wh2
 from {db}.web_sales ws1,{db}.web_sales ws2
 where ws1.ws_order_number = ws2.ws_order_number
   and ws1.ws_warehouse_sk <> ws2.ws_warehouse_sk)
 select
   count(distinct ws_order_number) as `order count`
  ,sum(ws_ext_ship_cost) as `total shipping cost`
  ,sum(ws_net_profit) as `total net profit`
from
   {db}.web_sales ws1
  ,{db}.date_dim
  ,{db}.customer_address
  ,{db}.web_site
where
    d_date between '2000-3-01' and
           (cast('2000-3-01' as date) + INTERVAL 60 days)
and ws1.ws_ship_date_sk = d_date_sk
and ws1.ws_ship_addr_sk = ca_address_sk
and ca_state = 'TN'
and ws1.ws_web_site_sk = web_site_sk
and web_company_name = 'pri'
and ws1.ws_order_number in (select ws_order_number
                            from ws_wh)
and ws1.ws_order_number in (select wr_order_number
                            from {db}.web_returns,ws_wh
                            where wr_order_number = ws_wh.ws_order_number)
order by count(distinct ws_order_number)
limit 100""",
    "q96": f"""
select  count(*)
from {db}.store_sales
    ,{db}.household_demographics
    ,{db}.time_dim, {db}.store
where ss_sold_time_sk = {db}.time_dim.t_time_sk
    and ss_hdemo_sk = {db}.household_demographics.hd_demo_sk
    and ss_store_sk = s_store_sk
    and {db}.time_dim.t_hour = 16
    and {db}.time_dim.t_minute >= 30
    and {db}.household_demographics.hd_dep_count = 4
    and {db}.store.s_store_name = 'ese'
order by count(*)
limit 100""",
    "q97": f"""
with ssci as (
select ss_customer_sk customer_sk
      ,ss_item_sk item_sk
from {db}.store_sales,{db}.date_dim
where ss_sold_date_sk = d_date_sk
  and d_month_seq between 1209 and 1209 + 11
group by ss_customer_sk
        ,ss_item_sk),
csci as(
 select cs_bill_customer_sk customer_sk
      ,cs_item_sk item_sk
from {db}.catalog_sales,{db}.date_dim
where cs_sold_date_sk = d_date_sk
  and d_month_seq between 1209 and 1209 + 11
group by cs_bill_customer_sk
        ,cs_item_sk)
 select  sum(case when ssci.customer_sk is not null and csci.customer_sk is null then 1 else 0 end) store_only
      ,sum(case when ssci.customer_sk is null and csci.customer_sk is not null then 1 else 0 end) catalog_only
      ,sum(case when ssci.customer_sk is not null and csci.customer_sk is not null then 1 else 0 end) store_and_catalog
from ssci full outer join csci on (ssci.customer_sk=csci.customer_sk
                               and ssci.item_sk = csci.item_sk)
limit 100""",
    "q98": f"""
select i_item_id
      ,i_item_desc
      ,i_category
      ,i_class
      ,i_current_price
      ,sum(ss_ext_sales_price) as itemrevenue
      ,sum(ss_ext_sales_price)*100/sum(sum(ss_ext_sales_price)) over
          (partition by i_class) as revenueratio
from
	{db}.store_sales
    	,{db}.item
    	,{db}.date_dim
where
	ss_item_sk = i_item_sk
  	and i_category in ('Jewelry', 'Home', 'Shoes')
  	and ss_sold_date_sk = d_date_sk
	and d_date between cast('2001-04-12' as date)
				and (cast('2001-04-12' as date) + interval 30 days)
group by
	i_item_id
        ,i_item_desc
        ,i_category
        ,i_class
        ,i_current_price
order by
	i_category
        ,i_class
        ,i_item_id
        ,i_item_desc
        ,revenueratio""",
    "q99": f"""
select
   substr(w_warehouse_name,1,20)
  ,sm_type
  ,cc_name
  ,sum(case when (cs_ship_date_sk - cs_sold_date_sk <= 30 ) then 1 else 0 end)  as `30 days`
  ,sum(case when (cs_ship_date_sk - cs_sold_date_sk > 30) and
                 (cs_ship_date_sk - cs_sold_date_sk <= 60) then 1 else 0 end )  as `31-60 days`
  ,sum(case when (cs_ship_date_sk - cs_sold_date_sk > 60) and
                 (cs_ship_date_sk - cs_sold_date_sk <= 90) then 1 else 0 end)  as `61-90 days`
  ,sum(case when (cs_ship_date_sk - cs_sold_date_sk > 90) and
                 (cs_ship_date_sk - cs_sold_date_sk <= 120) then 1 else 0 end)  as `91-120 days`
  ,sum(case when (cs_ship_date_sk - cs_sold_date_sk  > 120) then 1 else 0 end)  as `>120 days`
from
   {db}.catalog_sales
  ,{db}.warehouse
  ,{db}.ship_mode
  ,{db}.call_center
  ,{db}.date_dim
where
    d_month_seq between 1203 and 1203 + 11
and cs_ship_date_sk   = d_date_sk
and cs_warehouse_sk   = w_warehouse_sk
and cs_ship_mode_sk   = sm_ship_mode_sk
and cs_call_center_sk = cc_call_center_sk
group by
   substr(w_warehouse_name,1,20)
  ,sm_type
  ,cc_name
order by substr(w_warehouse_name,1,20)
        ,sm_type
        ,cc_name
limit 100"""
}



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
import time
execution_times = {}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
print(execution_times)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{}

In [ ]:
#selected_queries = input("Enter individual query keys separated by commas (e.g., q1, q4, q99): ").split(',')
selected_queries = "q10".split(',')
#selected_queries = [f"q{i}" for i in range(1, 100)]

for query_id in selected_queries:
    query_id = query_id.strip()
    query = queries.get(query_id)
    
    if query:
        start_time = time.time()

        #print(query)
        result_df = spark.sql(query)
        result_df.show(100)
        
        end_time = time.time()
        execution_time = end_time - start_time
        execution_times[query_id] = execution_time
        print(f"\nExecution time for Query {query_id}: {execution_time} seconds")

print("Overall Execution Times:")
print(execution_times)

In [ ]:
"""
select  count(*)
from {db}.store_sales
    ,{db}.household_demographics
    ,{db}.time_dim, {db}.store
where ss_sold_time_sk = {db}.time_dim.t_time_sk
    and ss_hdemo_sk = {db}.household_demographics.hd_demo_sk
    and ss_store_sk = s_store_sk
    and {db}.time_dim.t_hour = 16
    and {db}.time_dim.t_minute >= 30
    and {db}.household_demographics.hd_dep_count = 4
    and store.s_store_name = 'ese'
order by count(*)
limit 100"""

In [ ]:
result_df = spark.sql(f"""
	select
	  cd_gender,
	  cd_marital_status,
	  cd_education_status,
	  count(*) cnt1,
	  cd_purchase_estimate,
	  count(*) cnt2,
	  cd_credit_rating,
	  count(*) cnt3,
	  cd_dep_count,
	  count(*) cnt4,
	  cd_dep_employed_count,
	  count(*) cnt5,
	  cd_dep_college_count,
	  count(*) cnt6
	 from
	  {db}.customer c,{db}.customer_address ca,{db}.customer_demographics
	 where
	  {db}.customer.c_current_addr_sk = {db}.customer_address.ca_address_sk and
	  ({db}.customer_address.ca_county like 'Allen County' or {db}.customer_address.ca_county like 'Jefferson County' or {db}.customer_address.ca_county like 'Lamar County' or {db}.customer_address.ca_county like 'Dakota County' or {db}.customer_address.ca_county like 'Park County') and
	  {db}.customer_demographics.cd_demo_sk = c.c_current_cdemo_sk and
	  exists (select *
			  from {db}.store_sales,{db}.date_dim
			  where c.c_customer_sk = ss_customer_sk and
					ss_sold_date_sk = d_date_sk and
					d_year = 2001 and
					d_moy between 4 and 4+3) and
	   (exists (select *
				from {db}.web_sales,{db}.date_dim
				where c.c_customer_sk = ws_bill_customer_sk and
					  ws_sold_date_sk = d_date_sk and
					  d_year = 2001 and
					  d_moy between 4 ANd 4+3) or
		exists (select *
				from {db}.catalog_sales,{db}.date_dim
				where c.c_customer_sk = cs_ship_customer_sk and
					  cs_sold_date_sk = d_date_sk and
					  d_year = 2001 and
					  d_moy between 4 and 4+3))
	 group by cd_gender,
			  cd_marital_status,
			  cd_education_status,
			  cd_purchase_estimate,
			  cd_credit_rating,
			  cd_dep_count,
			  cd_dep_employed_count,
			  cd_dep_college_count
	 order by cd_gender,
			  cd_marital_status,
			  cd_education_status,
			  cd_purchase_estimate,
			  cd_credit_rating,
			  cd_dep_count,
			  cd_dep_employed_count,
			  cd_dep_college_count
	limit 100    """)
result_df.show(100)

In [6]:
%%sql
select * from dev.iceberg100.warehouse;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [9]:
%%sql

	with customer_total_return as
	(select sr_customer_sk as ctr_customer_sk
	,sr_store_sk as ctr_store_sk
	,sum(SR_FEE) as ctr_total_return
	from dev.iceberg100.store_returns
	,dev.iceberg100.date_dim
	where sr_returned_date_sk = d_date_sk
	and d_year =2000
	group by sr_customer_sk
	,sr_store_sk)
	 select  c_customer_id
	from customer_total_return ctr1
	,dev.iceberg100.store
	,dev.iceberg100.customer
	where ctr1.ctr_total_return > (select avg(ctr_total_return)*1.2
	from customer_total_return ctr2
	where ctr1.ctr_store_sk = ctr2.ctr_store_sk)
	and s_store_sk = ctr1.ctr_store_sk
	and s_state = 'TN'
	and ctr1.ctr_customer_sk = c_customer_sk
	order by c_customer_id
	limit 100

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [27]:
%%sql
--q1

	with customer_total_return as
	(select sr_customer_sk as ctr_customer_sk
	,sr_store_sk as ctr_store_sk
	,sum(SR_FEE) as ctr_total_return
	from dev.iceberg100.store_returns
	,dev.iceberg100.date_dim
	where sr_returned_date_sk = d_date_sk
	and d_year =2000
	group by sr_customer_sk
	,sr_store_sk)
	 select  c_customer_id
	from customer_total_return ctr1
	,dev.iceberg100.store
	,dev.iceberg100.customer
	where ctr1.ctr_total_return > (select avg(ctr_total_return)*1.2
	from customer_total_return ctr2
	where ctr1.ctr_store_sk = ctr2.ctr_store_sk)
	and s_store_sk = ctr1.ctr_store_sk
	and s_state = 'TN'
	and ctr1.ctr_customer_sk = c_customer_sk
	order by c_customer_id
	limit 100

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [28]:
%%sql
--q2
	with wscs as
	 (select sold_date_sk
			,sales_price
	  from (select ws_sold_date_sk sold_date_sk
				  ,ws_ext_sales_price sales_price
			from dev.iceberg100.web_sales
			union all
			select cs_sold_date_sk sold_date_sk
				  ,cs_ext_sales_price sales_price
			from dev.iceberg100.catalog_sales)),
	 wswscs as
	 (select d_week_seq,
			sum(case when (d_day_name='Sunday') then sales_price else null end) sun_sales,
			sum(case when (d_day_name='Monday') then sales_price else null end) mon_sales,
			sum(case when (d_day_name='Tuesday') then sales_price else  null end) tue_sales,
			sum(case when (d_day_name='Wednesday') then sales_price else null end) wed_sales,
			sum(case when (d_day_name='Thursday') then sales_price else null end) thu_sales,
			sum(case when (d_day_name='Friday') then sales_price else null end) fri_sales,
			sum(case when (d_day_name='Saturday') then sales_price else null end) sat_sales
	 from wscs
		 ,dev.iceberg100.date_dim
	 where d_date_sk = sold_date_sk
	 group by d_week_seq)
	 select d_week_seq1
		   ,round(sun_sales1/sun_sales2,2)
		   ,round(mon_sales1/mon_sales2,2)
		   ,round(tue_sales1/tue_sales2,2)
		   ,round(wed_sales1/wed_sales2,2)
		   ,round(thu_sales1/thu_sales2,2)
		   ,round(fri_sales1/fri_sales2,2)
		   ,round(sat_sales1/sat_sales2,2)
	 from
	 (select wswscs.d_week_seq d_week_seq1
			,sun_sales sun_sales1
			,mon_sales mon_sales1
			,tue_sales tue_sales1
			,wed_sales wed_sales1
			,thu_sales thu_sales1
			,fri_sales fri_sales1
			,sat_sales sat_sales1
	  from wswscs,dev.iceberg100.date_dim
	  where dev.iceberg100.date_dim.d_week_seq = wswscs.d_week_seq and
			d_year = 1998) y,
	 (select wswscs.d_week_seq d_week_seq2
			,sun_sales sun_sales2
			,mon_sales mon_sales2
			,tue_sales tue_sales2
			,wed_sales wed_sales2
			,thu_sales thu_sales2
			,fri_sales fri_sales2
			,sat_sales sat_sales2
	  from wswscs
		  ,dev.iceberg100.date_dim
	  where dev.iceberg100.date_dim.d_week_seq = wswscs.d_week_seq and
			d_year = 1998+1) z
	 where d_week_seq1=d_week_seq2-53
	 order by d_week_seq1

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [29]:
%%sql
--q3
	select  dt.d_year
		   ,dev.iceberg100.item.i_brand_id brand_id
		   ,dev.iceberg100.item.i_brand brand
		   ,sum(ss_sales_price) sum_agg
	 from  dev.iceberg100.date_dim dt
		  ,dev.iceberg100.store_sales
		  ,dev.iceberg100.item
	 where dt.d_date_sk = dev.iceberg100.store_sales.ss_sold_date_sk
	   and dev.iceberg100.store_sales.ss_item_sk = dev.iceberg100.item.i_item_sk
	   and dev.iceberg100.item.i_manufact_id = 816
	   and dt.d_moy=11
	 group by dt.d_year
		  ,dev.iceberg100.item.i_brand
		  ,dev.iceberg100.item.i_brand_id
	 order by dt.d_year
			 ,sum_agg desc
			 ,brand_id
	 limit 100

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [ ]:
%%sql
--q4

	with year_total as (
	 select c_customer_id customer_id
		   ,c_first_name customer_first_name
		   ,c_last_name customer_last_name
		   ,c_preferred_cust_flag customer_preferred_cust_flag
		   ,c_birth_country customer_birth_country
		   ,c_login customer_login
		   ,c_email_address customer_email_address
		   ,d_year dyear
		   ,sum(((ss_ext_list_price-ss_ext_wholesale_cost-ss_ext_discount_amt)+ss_ext_sales_price)/2) year_total
		   ,'s' sale_type
	 from dev.iceberg100.customer
		 ,dev.iceberg100.store_sales
		 ,dev.iceberg100.date_dim
	 where c_customer_sk = ss_customer_sk
	   and ss_sold_date_sk = d_date_sk
	 group by c_customer_id
			 ,c_first_name
			 ,c_last_name
			 ,c_preferred_cust_flag
			 ,c_birth_country
			 ,c_login
			 ,c_email_address
			 ,d_year
	 union all
	 select c_customer_id customer_id
		   ,c_first_name customer_first_name
		   ,c_last_name customer_last_name
		   ,c_preferred_cust_flag customer_preferred_cust_flag
		   ,c_birth_country customer_birth_country
		   ,c_login customer_login
		   ,c_email_address customer_email_address
		   ,d_year dyear
		   ,sum((((cs_ext_list_price-cs_ext_wholesale_cost-cs_ext_discount_amt)+cs_ext_sales_price)/2) ) year_total
		   ,'c' sale_type
	 from dev.iceberg100.customer
		 ,dev.iceberg100.catalog_sales
		 ,dev.iceberg100.date_dim
	 where c_customer_sk = cs_bill_customer_sk
	   and cs_sold_date_sk = d_date_sk
	 group by c_customer_id
			 ,c_first_name
			 ,c_last_name
			 ,c_preferred_cust_flag
			 ,c_birth_country
			 ,c_login
			 ,c_email_address
			 ,d_year
	union all
	 select c_customer_id customer_id
		   ,c_first_name customer_first_name
		   ,c_last_name customer_last_name
		   ,c_preferred_cust_flag customer_preferred_cust_flag
		   ,c_birth_country customer_birth_country
		   ,c_login customer_login
		   ,c_email_address customer_email_address
		   ,d_year dyear
		   ,sum((((ws_ext_list_price-ws_ext_wholesale_cost-ws_ext_discount_amt)+ws_ext_sales_price)/2) ) year_total
		   ,'w' sale_type
	 from dev.iceberg100.customer
		 ,dev.iceberg100.web_sales
		 ,dev.iceberg100.date_dim
	 where c_customer_sk = ws_bill_customer_sk
	   and ws_sold_date_sk = d_date_sk
	 group by c_customer_id
			 ,c_first_name
			 ,c_last_name
			 ,c_preferred_cust_flag
			 ,c_birth_country
			 ,c_login
			 ,c_email_address
			 ,d_year
			 )
  select
                  t_s_secyear.customer_id
                 ,t_s_secyear.customer_first_name
                 ,t_s_secyear.customer_last_name
                 ,t_s_secyear.customer_birth_country
 from year_total t_s_firstyear
     ,year_total t_s_secyear
     ,year_total t_c_firstyear
     ,year_total t_c_secyear
     ,year_total t_w_firstyear
     ,year_total t_w_secyear
 where t_s_secyear.customer_id = t_s_firstyear.customer_id
   and t_s_firstyear.customer_id = t_c_secyear.customer_id
   and t_s_firstyear.customer_id = t_c_firstyear.customer_id
   and t_s_firstyear.customer_id = t_w_firstyear.customer_id
   and t_s_firstyear.customer_id = t_w_secyear.customer_id
   and t_s_firstyear.sale_type = 's'
   and t_c_firstyear.sale_type = 'c'
   and t_w_firstyear.sale_type = 'w'
   and t_s_secyear.sale_type = 's'
   and t_c_secyear.sale_type = 'c'
   and t_w_secyear.sale_type = 'w'
   and t_s_firstyear.dyear =  1999
   and t_s_secyear.dyear = 1999+1
   and t_c_firstyear.dyear =  1999
   and t_c_secyear.dyear =  1999+1
   and t_w_firstyear.dyear = 1999
   and t_w_secyear.dyear = 1999+1
   and t_s_firstyear.year_total > 0
   and t_c_firstyear.year_total > 0
   and t_w_firstyear.year_total > 0
   and case when t_c_firstyear.year_total > 0 then t_c_secyear.year_total / t_c_firstyear.year_total else null end
           > case when t_s_firstyear.year_total > 0 then t_s_secyear.year_total / t_s_firstyear.year_total else null end
   and case when t_c_firstyear.year_total > 0 then t_c_secyear.year_total / t_c_firstyear.year_total else null end
           > case when t_w_firstyear.year_total > 0 then t_w_secyear.year_total / t_w_firstyear.year_total else null end
 order by t_s_secyear.customer_id
         ,t_s_secyear.customer_first_name
         ,t_s_secyear.customer_last_name
         ,t_s_secyear.customer_birth_country
limit 100

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
%%sql
	 select c_customer_id customer_id
		   ,c_first_name customer_first_name
		   ,c_last_name customer_last_name
		   ,c_preferred_cust_flag customer_preferred_cust_flag
		   ,c_birth_country customer_birth_country
		   ,c_login customer_login
		   ,c_email_address customer_email_address
		   ,d_year dyear
		   ,sum(((ss_ext_list_price-ss_ext_wholesale_cost-ss_ext_discount_amt)+ss_ext_sales_price)/2) year_total
		   ,'s' sale_type
	 from dev.iceberg100.customer
		 ,dev.iceberg100.store_sales
		 ,dev.iceberg100.date_dim
	 where c_customer_sk = ss_customer_sk
	   and ss_sold_date_sk = d_date_sk
	 group by c_customer_id
			 ,c_first_name
			 ,c_last_name
			 ,c_preferred_cust_flag
			 ,c_birth_country
			 ,c_login
			 ,c_email_address
			 ,d_year
	 union all
	 select c_customer_id customer_id
		   ,c_first_name customer_first_name
		   ,c_last_name customer_last_name
		   ,c_preferred_cust_flag customer_preferred_cust_flag
		   ,c_birth_country customer_birth_country
		   ,c_login customer_login
		   ,c_email_address customer_email_address
		   ,d_year dyear
		   ,sum((((cs_ext_list_price-cs_ext_wholesale_cost-cs_ext_discount_amt)+cs_ext_sales_price)/2) ) year_total
		   ,'c' sale_type
	 from dev.iceberg100.customer
		 ,dev.iceberg100.catalog_sales
		 ,dev.iceberg100.date_dim
	 where c_customer_sk = cs_bill_customer_sk
	   and cs_sold_date_sk = d_date_sk
	 group by c_customer_id
			 ,c_first_name
			 ,c_last_name
			 ,c_preferred_cust_flag
			 ,c_birth_country
			 ,c_login
			 ,c_email_address
			 ,d_year
	union all
	 select c_customer_id customer_id
		   ,c_first_name customer_first_name
		   ,c_last_name customer_last_name
		   ,c_preferred_cust_flag customer_preferred_cust_flag
		   ,c_birth_country customer_birth_country
		   ,c_login customer_login
		   ,c_email_address customer_email_address
		   ,d_year dyear
		   ,sum((((ws_ext_list_price-ws_ext_wholesale_cost-ws_ext_discount_amt)+ws_ext_sales_price)/2) ) year_total
		   ,'w' sale_type
	 from dev.iceberg100.customer
		 ,dev.iceberg100.web_sales
		 ,dev.iceberg100.date_dim
	 where c_customer_sk = ws_bill_customer_sk
	   and ws_sold_date_sk = d_date_sk
	 group by c_customer_id
			 ,c_first_name
			 ,c_last_name
			 ,c_preferred_cust_flag
			 ,c_birth_country
			 ,c_login
			 ,c_email_address
			 ,d_year

In [32]:
%%sql
--q5

	with ssr as
	 (select s_store_id,
			sum(sales_price) as sales,
			sum(profit) as profit,
			sum(return_amt) as returns,
			sum(net_loss) as profit_loss
	 from
	  ( select  ss_store_sk as store_sk,
				ss_sold_date_sk  as date_sk,
				ss_ext_sales_price as sales_price,
				ss_net_profit as profit,
				cast(0 as decimal(7,2)) as return_amt,
				cast(0 as decimal(7,2)) as net_loss
		from dev.iceberg100.store_sales
		union all
		select sr_store_sk as store_sk,
			   sr_returned_date_sk as date_sk,
			   cast(0 as decimal(7,2)) as sales_price,
			   cast(0 as decimal(7,2)) as profit,
			   sr_return_amt as return_amt,
			   sr_net_loss as net_loss
		from dev.iceberg100.store_returns
	   ) salesreturns,
		 dev.iceberg100.date_dim,
		 dev.iceberg100.store
	 where date_sk = d_date_sk
		   and d_date between cast('2000-08-19' as date)
					  and (cast('2000-08-19' as date) +  INTERVAL 14 days)
		   and store_sk = s_store_sk
	 group by s_store_id)
	 ,
	 csr as
	 (select cp_catalog_page_id,
			sum(sales_price) as sales,
			sum(profit) as profit,
			sum(return_amt) as returns,
			sum(net_loss) as profit_loss
	 from
	  ( select  cs_catalog_page_sk as page_sk,
				cs_sold_date_sk  as date_sk,
				cs_ext_sales_price as sales_price,
				cs_net_profit as profit,
				cast(0 as decimal(7,2)) as return_amt,
				cast(0 as decimal(7,2)) as net_loss
		from dev.iceberg100.catalog_sales
		union all
		select cr_catalog_page_sk as page_sk,
			   cr_returned_date_sk as date_sk,
			   cast(0 as decimal(7,2)) as sales_price,
			   cast(0 as decimal(7,2)) as profit,
			   cr_return_amount as return_amt,
			   cr_net_loss as net_loss
		from dev.iceberg100.catalog_returns
	   ) salesreturns,
		 dev.iceberg100.date_dim,
		 dev.iceberg100.catalog_page
	 where date_sk = d_date_sk
		   and d_date between cast('2000-08-19' as date)
					  and (cast('2000-08-19' as date) +  INTERVAL 14 days)
		   and page_sk = cp_catalog_page_sk
	 group by cp_catalog_page_id)
	 ,
	 wsr as
	 (select web_site_id,
			sum(sales_price) as sales,
			sum(profit) as profit,
			sum(return_amt) as returns,
			sum(net_loss) as profit_loss
	 from
	  ( select  ws_web_site_sk as wsr_web_site_sk,
				ws_sold_date_sk  as date_sk,
				ws_ext_sales_price as sales_price,
				ws_net_profit as profit,
				cast(0 as decimal(7,2)) as return_amt,
				cast(0 as decimal(7,2)) as net_loss
		from dev.iceberg100.web_sales
		union all
		select ws_web_site_sk as wsr_web_site_sk,
			   wr_returned_date_sk as date_sk,
			   cast(0 as decimal(7,2)) as sales_price,
			   cast(0 as decimal(7,2)) as profit,
			   wr_return_amt as return_amt,
			   wr_net_loss as net_loss
		from dev.iceberg100.web_returns left outer join dev.iceberg100.web_sales on
			 ( wr_item_sk = ws_item_sk
			   and wr_order_number = ws_order_number)
	   ) salesreturns,
		 dev.iceberg100.date_dim,
		 dev.iceberg100.web_site
	 where date_sk = d_date_sk
		   and d_date between cast('2000-08-19' as date)
					  and (cast('2000-08-19' as date) +  INTERVAL 14 days)
		   and wsr_web_site_sk = web_site_sk
	 group by web_site_id)
	  select  channel
			, id
			, sum(sales) as sales
			, sum(returns) as returns
			, sum(profit) as profit
	 from
	 (select 'store channel' as channel
			, 'store' || s_store_id as id
			, sales
			, returns
			, (profit - profit_loss) as profit
	 from   ssr
	 union all
	 select 'catalog channel' as channel
			, 'catalog_page' || cp_catalog_page_id as id
			, sales
			, returns
			, (profit - profit_loss) as profit
	 from  csr
	 union all
	 select 'web channel' as channel
			, 'web_site' || web_site_id as id
			, sales
			, returns
			, (profit - profit_loss) as profit
	 from   wsr
	 ) x
	 group by rollup (channel, id)
	 order by channel
			 ,id
	 limit 100

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [33]:
%%capture output
%%time
%%sql
--q6

	select  a.ca_state state, count(*) cnt
	 from dev.iceberg100.customer_address a
		 ,dev.iceberg100.customer c
		 ,dev.iceberg100.store_sales s
		 ,dev.iceberg100.date_dim d
		 ,dev.iceberg100.item i
	 where       a.ca_address_sk = c.c_current_addr_sk
		and c.c_customer_sk = s.ss_customer_sk
		and s.ss_sold_date_sk = d.d_date_sk
		and s.ss_item_sk = i.i_item_sk
		and d.d_month_seq =
			 (select distinct (d_month_seq)
			  from dev.iceberg100.date_dim
				   where d_year = 2002
				and d_moy = 3 )
		and i.i_current_price > 1.2 *
				 (select avg(j.i_current_price)
			 from dev.iceberg100.item j
			 where j.i_category = i.i_category)
	 group by a.ca_state
	 having count(*) >= 10
	 order by cnt, a.ca_state
	 limit 100

In [35]:
%%sql
--q7

	select  i_item_id,
			avg(ss_quantity) agg1,
			avg(ss_list_price) agg2,
			avg(ss_coupon_amt) agg3,
			avg(ss_sales_price) agg4
	 from dev.iceberg100.store_sales, dev.iceberg100.customer_demographics, dev.iceberg100.date_dim, dev.iceberg100.item, dev.iceberg100.promotion
	 where ss_sold_date_sk = d_date_sk and
		   ss_item_sk = i_item_sk and
		   ss_cdemo_sk = cd_demo_sk and
		   ss_promo_sk = p_promo_sk and
		   cd_gender = 'F' and
		   cd_marital_status = 'W' and
		   cd_education_status = 'College' and
		   (p_channel_email = 'N' or p_channel_event = 'N') and
		   d_year = 2001
	 group by i_item_id
	 order by i_item_id
	 limit 100

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [36]:
%%sql
--q8

	select  s_store_name
		  ,sum(ss_net_profit)
	 from dev.iceberg100.store_sales
		 ,dev.iceberg100.date_dim
		 ,dev.iceberg100.store,
		 (select ca_zip
		 from (
		  SELECT substr(ca_zip,1,5) ca_zip
		  FROM dev.iceberg100.customer_address
		  WHERE substr(ca_zip,1,5) IN (
							  '47602','16704','35863','28577','83910','36201',
							  '58412','48162','28055','41419','80332',
							  '38607','77817','24891','16226','18410',
							  '21231','59345','13918','51089','20317',
							  '17167','54585','67881','78366','47770',
							  '18360','51717','73108','14440','21800',
							  '89338','45859','65501','34948','25973',
							  '73219','25333','17291','10374','18829',
							  '60736','82620','41351','52094','19326',
							  '25214','54207','40936','21814','79077',
							  '25178','75742','77454','30621','89193',
							  '27369','41232','48567','83041','71948',
							  '37119','68341','14073','16891','62878',
							  '49130','19833','24286','27700','40979',
							  '50412','81504','94835','84844','71954',
							  '39503','57649','18434','24987','12350',
							  '86379','27413','44529','98569','16515',
							  '27287','24255','21094','16005','56436',
							  '91110','68293','56455','54558','10298',
							  '83647','32754','27052','51766','19444',
							  '13869','45645','94791','57631','20712',
							  '37788','41807','46507','21727','71836',
							  '81070','50632','88086','63991','20244',
							  '31655','51782','29818','63792','68605',
							  '94898','36430','57025','20601','82080',
							  '33869','22728','35834','29086','92645',
							  '98584','98072','11652','78093','57553',
							  '43830','71144','53565','18700','90209',
							  '71256','38353','54364','28571','96560',
							  '57839','56355','50679','45266','84680',
							  '34306','34972','48530','30106','15371',
							  '92380','84247','92292','68852','13338',
							  '34594','82602','70073','98069','85066',
							  '47289','11686','98862','26217','47529',
							  '63294','51793','35926','24227','14196',
							  '24594','32489','99060','49472','43432',
							  '49211','14312','88137','47369','56877',
							  '20534','81755','15794','12318','21060',
							  '73134','41255','63073','81003','73873',
							  '66057','51184','51195','45676','92696',
							  '70450','90669','98338','25264','38919',
							  '59226','58581','60298','17895','19489',
							  '52301','80846','95464','68770','51634',
							  '19988','18367','18421','11618','67975',
							  '25494','41352','95430','15734','62585',
							  '97173','33773','10425','75675','53535',
							  '17879','41967','12197','67998','79658',
							  '59130','72592','14851','43933','68101',
							  '50636','25717','71286','24660','58058',
							  '72991','95042','15543','33122','69280',
							  '11912','59386','27642','65177','17672',
							  '33467','64592','36335','54010','18767',
							  '63193','42361','49254','33113','33159',
							  '36479','59080','11855','81963','31016',
							  '49140','29392','41836','32958','53163',
							  '13844','73146','23952','65148','93498',
							  '14530','46131','58454','13376','13378',
							  '83986','12320','17193','59852','46081',
							  '98533','52389','13086','68843','31013',
							  '13261','60560','13443','45533','83583',
							  '11489','58218','19753','22911','25115',
							  '86709','27156','32669','13123','51933',
							  '39214','41331','66943','14155','69998',
							  '49101','70070','35076','14242','73021',
							  '59494','15782','29752','37914','74686',
							  '83086','34473','15751','81084','49230',
							  '91894','60624','17819','28810','63180',
							  '56224','39459','55233','75752','43639',
							  '55349','86057','62361','50788','31830',
							  '58062','18218','85761','60083','45484',
							  '21204','90229','70041','41162','35390',
							  '16364','39500','68908','26689','52868',
							  '81335','40146','11340','61527','61794',
							  '71997','30415','59004','29450','58117',
							  '69952','33562','83833','27385','61860',
							  '96435','48333','23065','32961','84919',
							  '61997','99132','22815','56600','68730',
							  '48017','95694','32919','88217','27116',
							  '28239','58032','18884','16791','21343',
							  '97462','18569','75660','15475')
		 intersect
		  select ca_zip
		  from (SELECT substr(ca_zip,1,5) ca_zip,count(*) cnt
				FROM dev.iceberg100.customer_address, dev.iceberg100.customer
				WHERE ca_address_sk = c_current_addr_sk and
					  c_preferred_cust_flag='Y'
				group by ca_zip
				having count(*) > 10)A1)A2) V1
	 where ss_store_sk = s_store_sk
	  and ss_sold_date_sk = d_date_sk
	  and d_qoy = 2 and d_year = 1998
	  and (substr(s_zip,1,2) = substr(V1.ca_zip,1,2))
	 group by s_store_name
	 order by s_store_name
	 limit 100

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [37]:
%%sql
--q9

select case when (select count(*)
					  from dev.iceberg100.store_sales
					  where ss_quantity between 1 and 20) > 2972190
				then (select avg(ss_ext_discount_amt)
					  from dev.iceberg100.store_sales
					  where ss_quantity between 1 and 20)
				else (select avg(ss_net_profit)
					  from dev.iceberg100.store_sales
					  where ss_quantity between 1 and 20) end bucket1 ,
		   case when (select count(*)
					  from dev.iceberg100.store_sales
					  where ss_quantity between 21 and 40) > 111711138
				then (select avg(ss_ext_discount_amt)
					  from dev.iceberg100.store_sales
					  where ss_quantity between 21 and 40)
				else (select avg(ss_net_profit)
					  from dev.iceberg100.store_sales
					  where ss_quantity between 21 and 40) end bucket2,
		   case when (select count(*)
					  from dev.iceberg100.store_sales
					  where ss_quantity between 41 and 60) > 127958920
				then (select avg(ss_ext_discount_amt)
					  from dev.iceberg100.store_sales
					  where ss_quantity between 41 and 60)
				else (select avg(ss_net_profit)
					  from dev.iceberg100.store_sales
					  where ss_quantity between 41 and 60) end bucket3,
		   case when (select count(*)
					  from dev.iceberg100.store_sales
					  where ss_quantity between 61 and 80) > 41162107
				then (select avg(ss_ext_discount_amt)
					  from dev.iceberg100.store_sales
					  where ss_quantity between 61 and 80)
				else (select avg(ss_net_profit)
					  from dev.iceberg100.store_sales
					  where ss_quantity between 61 and 80) end bucket4,
		   case when (select count(*)
					  from dev.iceberg100.store_sales
					  where ss_quantity between 81 and 100) > 25211875
				then (select avg(ss_ext_discount_amt)
					  from dev.iceberg100.store_sales
					  where ss_quantity between 81 and 100)
				else (select avg(ss_net_profit)
					  from dev.iceberg100.store_sales
					  where ss_quantity between 81 and 100) end bucket5
	from dev.iceberg100.reason
	where r_reason_sk = 1

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [38]:
%%sql
--q10

	select
	  cd_gender,
	  cd_marital_status,
	  cd_education_status,
	  count(*) cnt1,
	  cd_purchase_estimate,
	  count(*) cnt2,
	  cd_credit_rating,
	  count(*) cnt3,
	  cd_dep_count,
	  count(*) cnt4,
	  cd_dep_employed_count,
	  count(*) cnt5,
	  cd_dep_college_count,
	  count(*) cnt6
	 from
	  dev.iceberg100.customer c,dev.iceberg100.customer_address ca,dev.iceberg100.customer_demographics
	 where
	  c.c_current_addr_sk = ca.ca_address_sk and
	  (ca_county like 'Allen County' or ca_county like 'Jefferson County' or ca_county like 'Lamar County' or ca_county like 'Dakota County' or ca_county like 'Park County') and
	  cd_demo_sk = c.c_current_cdemo_sk and
	  exists (select *
			  from dev.iceberg100.store_sales,dev.iceberg100.date_dim
			  where c.c_customer_sk = ss_customer_sk and
					ss_sold_date_sk = d_date_sk and
					d_year = 2001 and
					d_moy between 4 and 4+3) and
	   (exists (select *
				from dev.iceberg100.web_sales,dev.iceberg100.date_dim
				where c.c_customer_sk = ws_bill_customer_sk and
					  ws_sold_date_sk = d_date_sk and
					  d_year = 2001 and
					  d_moy between 4 ANd 4+3) or
		exists (select *
				from dev.iceberg100.catalog_sales,dev.iceberg100.date_dim
				where c.c_customer_sk = cs_ship_customer_sk and
					  cs_sold_date_sk = d_date_sk and
					  d_year = 2001 and
					  d_moy between 4 and 4+3))
	 group by cd_gender,
			  cd_marital_status,
			  cd_education_status,
			  cd_purchase_estimate,
			  cd_credit_rating,
			  cd_dep_count,
			  cd_dep_employed_count,
			  cd_dep_college_count
	 order by cd_gender,
			  cd_marital_status,
			  cd_education_status,
			  cd_purchase_estimate,
			  cd_credit_rating,
			  cd_dep_count,
			  cd_dep_employed_count,
			  cd_dep_college_count
	limit 100

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [39]:
%%sql
--q11

	with year_total as (
	 select c_customer_id customer_id
		   ,c_first_name customer_first_name
		   ,c_last_name customer_last_name
		   ,c_preferred_cust_flag customer_preferred_cust_flag
		   ,c_birth_country customer_birth_country
		   ,c_login customer_login
		   ,c_email_address customer_email_address
		   ,d_year dyear
		   ,sum(ss_ext_list_price-ss_ext_discount_amt) year_total
		   ,'s' sale_type
	 from dev.iceberg100.customer
		 ,dev.iceberg100.store_sales
		 ,dev.iceberg100.date_dim
	 where c_customer_sk = ss_customer_sk
	   and ss_sold_date_sk = d_date_sk
	 group by c_customer_id
			 ,c_first_name
			 ,c_last_name
			 ,c_preferred_cust_flag
			 ,c_birth_country
			 ,c_login
			 ,c_email_address
			 ,d_year
	 union all
	 select c_customer_id customer_id
		   ,c_first_name customer_first_name
		   ,c_last_name customer_last_name
		   ,c_preferred_cust_flag customer_preferred_cust_flag
		   ,c_birth_country customer_birth_country
		   ,c_login customer_login
		   ,c_email_address customer_email_address
		   ,d_year dyear
		   ,sum(ws_ext_list_price-ws_ext_discount_amt) year_total
		   ,'w' sale_type
	 from dev.iceberg100.customer
		 ,dev.iceberg100.web_sales
		 ,dev.iceberg100.date_dim
	 where c_customer_sk = ws_bill_customer_sk
	   and ws_sold_date_sk = d_date_sk
	 group by c_customer_id
			 ,c_first_name
			 ,c_last_name
			 ,c_preferred_cust_flag
			 ,c_birth_country
			 ,c_login
			 ,c_email_address
			 ,d_year
			 )
	  select
					  t_s_secyear.customer_id
					 ,t_s_secyear.customer_first_name
					 ,t_s_secyear.customer_last_name
					 ,t_s_secyear.customer_login
	 from year_total t_s_firstyear
		 ,year_total t_s_secyear
		 ,year_total t_w_firstyear
		 ,year_total t_w_secyear
	 where t_s_secyear.customer_id = t_s_firstyear.customer_id
			 and t_s_firstyear.customer_id = t_w_secyear.customer_id
			 and t_s_firstyear.customer_id = t_w_firstyear.customer_id
			 and t_s_firstyear.sale_type = 's'
			 and t_w_firstyear.sale_type = 'w'
			 and t_s_secyear.sale_type = 's'
			 and t_w_secyear.sale_type = 'w'
			 and t_s_firstyear.dyear = 1998
			 and t_s_secyear.dyear = 1998+1
			 and t_w_firstyear.dyear = 1998
			 and t_w_secyear.dyear = 1998+1
			 and t_s_firstyear.year_total > 0
			 and t_w_firstyear.year_total > 0
			 and case when t_w_firstyear.year_total > 0 then t_w_secyear.year_total / t_w_firstyear.year_total else 0.0 end
				 > case when t_s_firstyear.year_total > 0 then t_s_secyear.year_total / t_s_firstyear.year_total else 0.0 end
	 order by t_s_secyear.customer_id
			 ,t_s_secyear.customer_first_name
			 ,t_s_secyear.customer_last_name
			 ,t_s_secyear.customer_login
	limit 100

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [40]:
%%sql
--q12

	select  i_item_id
		  ,i_item_desc
		  ,i_category
		  ,i_class
		  ,i_current_price
		  ,sum(ws_ext_sales_price) as itemrevenue
		  ,sum(ws_ext_sales_price)*100/sum(sum(ws_ext_sales_price)) over
			  (partition by i_class) as revenueratio
	from
		dev.iceberg100.web_sales
			,dev.iceberg100.item
			,dev.iceberg100.date_dim
	where
		ws_item_sk = i_item_sk
		and (i_category like 'Men' or i_category like 'Books' or i_category like 'Children')
		and ws_sold_date_sk = d_date_sk
		and d_date between cast('1998-03-28' as date)
					and (cast('1998-03-28' as date) + INTERVAL 30 days)
	group by
		i_item_id
			,i_item_desc
			,i_category
			,i_class
			,i_current_price
	order by
		i_category
			,i_class
			,i_item_id
			,i_item_desc
			,revenueratio
	limit 100

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [41]:
%%sql
--q13

	select avg(ss_quantity)
		   ,avg(ss_ext_sales_price)
		   ,avg(ss_ext_wholesale_cost)
		   ,sum(ss_ext_wholesale_cost)
	 from dev.iceberg100.store_sales
		 ,dev.iceberg100.store
		 ,dev.iceberg100.customer_demographics
		 ,dev.iceberg100.household_demographics
		 ,dev.iceberg100.customer_address
		 ,dev.iceberg100.date_dim
	 where s_store_sk = ss_store_sk
	 and  ss_sold_date_sk = d_date_sk and d_year = 2001
	 and((ss_hdemo_sk=hd_demo_sk
	  and cd_demo_sk = ss_cdemo_sk
	  and cd_marital_status = 'U'
	  and cd_education_status = 'Unknown'
	  and ss_sales_price between 100.00 and 150.00
	  and hd_dep_count = 3
		 )or
		 (ss_hdemo_sk=hd_demo_sk
	  and cd_demo_sk = ss_cdemo_sk
	  and cd_marital_status = 'W'
	  and cd_education_status = '2 yr Degree'
	  and ss_sales_price between 50.00 and 100.00
	  and hd_dep_count = 1
		 ) or
		 (ss_hdemo_sk=hd_demo_sk
	  and cd_demo_sk = ss_cdemo_sk
	  and cd_marital_status = 'S'
	  and cd_education_status = 'College'
	  and ss_sales_price between 150.00 and 200.00
	  and hd_dep_count = 1
		 ))
	 and((ss_addr_sk = ca_address_sk
	  and ca_country = 'United States'
	  and (ca_state like 'WV' or ca_state like 'GA' or ca_state like 'TX')
	  and ss_net_profit between 100 and 200
		 ) or
		 (ss_addr_sk = ca_address_sk
	  and ca_country = 'United States'
	  and (ca_state like 'TN' or ca_state like 'KY' or ca_state like 'SC')
	  and ss_net_profit between 150 and 300
		 ) or
		 (ss_addr_sk = ca_address_sk
	  and ca_country = 'United States'
	  and (ca_state like 'OK' or ca_state like  'NE' or ca_state like  'CA')
	  and ss_net_profit between 50 and 250
		 ))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [63]:
%%sql
--q14

	with  cross_items as
	 (select i_item_sk ss_item_sk
	 from dev.iceberg100.item,
	 (select iss.i_brand_id brand_id
		 ,iss.i_class_id class_id
		 ,iss.i_category_id category_id
	 from dev.iceberg100.store_sales
		 ,dev.iceberg100.item iss
		 ,dev.iceberg100.date_dim d1
	 where ss_item_sk = iss.i_item_sk
	   and ss_sold_date_sk = d1.d_date_sk
	   and d1.d_year between 1998 AND 1998 + 2
	 intersect
	 select ics.i_brand_id
		 ,ics.i_class_id
		 ,ics.i_category_id
	 from dev.iceberg100.catalog_sales
		 ,dev.iceberg100.item ics
		 ,dev.iceberg100.date_dim d2
	 where cs_item_sk = ics.i_item_sk
	   and cs_sold_date_sk = d2.d_date_sk
	   and d2.d_year between 1998 AND 1998 + 2
	 intersect
	 select iws.i_brand_id
		 ,iws.i_class_id
		 ,iws.i_category_id
	 from dev.iceberg100.web_sales
		 ,dev.iceberg100.item iws
		 ,dev.iceberg100.date_dim d3
	 where ws_item_sk = iws.i_item_sk
	   and ws_sold_date_sk = d3.d_date_sk
	   and d3.d_year between 1998 AND 1998 + 2)
	 where i_brand_id = brand_id
		  and i_class_id = class_id
		  and i_category_id = category_id
	),
	 avg_sales as
	 (select avg(quantity*list_price) average_sales
	  from (select ss_quantity quantity
				 ,ss_list_price list_price
		   from dev.iceberg100.store_sales
			   ,dev.iceberg100.date_dim
		   where ss_sold_date_sk = d_date_sk
			 and d_year between 1998 and 1998 + 2
		   union all
		   select cs_quantity quantity
				 ,cs_list_price list_price
		   from dev.iceberg100.catalog_sales
			   ,dev.iceberg100.date_dim
		   where cs_sold_date_sk = d_date_sk
			 and d_year between 1998 and 1998 + 2
		   union all
		   select ws_quantity quantity
				 ,ws_list_price list_price
		   from dev.iceberg100.web_sales
			   ,dev.iceberg100.date_dim
		   where ws_sold_date_sk = d_date_sk
			 and d_year between 1998 and 1998 + 2) x)
	  select  channel, i_brand_id,i_class_id,i_category_id,sum(sales), sum(number_sales)
	 from(
		   select 'store' channel, i_brand_id,i_class_id
				 ,i_category_id,sum(ss_quantity*ss_list_price) sales
				 , count(*) number_sales
		   from dev.iceberg100.store_sales
			   ,dev.iceberg100.item
			   ,dev.iceberg100.date_dim
		   where ss_item_sk in (select ss_item_sk from cross_items)
			 and ss_item_sk = i_item_sk
			 and ss_sold_date_sk = d_date_sk
			 and d_year = 1998+2
			 and d_moy = 11
		   group by i_brand_id,i_class_id,i_category_id
		   having sum(ss_quantity*ss_list_price) > (select average_sales from avg_sales)
		   union all
		   select 'catalog' channel, i_brand_id,i_class_id,i_category_id, sum(cs_quantity*cs_list_price) sales, count(*) number_sales
		   from dev.iceberg100.catalog_sales
			   ,dev.iceberg100.item
			   ,dev.iceberg100.date_dim
		   where cs_item_sk in (select ss_item_sk from cross_items)
			 and cs_item_sk = i_item_sk
			 and cs_sold_date_sk = d_date_sk
			 and d_year = 1998+2
			 and d_moy = 11
		   group by i_brand_id,i_class_id,i_category_id
		   having sum(cs_quantity*cs_list_price) > (select average_sales from avg_sales)
		   union all
		   select 'web' channel, i_brand_id,i_class_id,i_category_id, sum(ws_quantity*ws_list_price) sales , count(*) number_sales
		   from dev.iceberg100.web_sales
			   ,dev.iceberg100.item
			   ,dev.iceberg100.date_dim
		   where ws_item_sk in (select ss_item_sk from cross_items)
			 and ws_item_sk = i_item_sk
			 and ws_sold_date_sk = d_date_sk
			 and d_year = 1998+2
			 and d_moy = 11
		   group by i_brand_id,i_class_id,i_category_id
		   having sum(ws_quantity*ws_list_price) > (select average_sales from avg_sales)
	 ) y
	 group by rollup (channel, i_brand_id,i_class_id,i_category_id)
	 order by channel,i_brand_id,i_class_id,i_category_id
	 limit 100

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [69]:
%%sql
--q15

	select  ca_zip
		   ,sum(cs_sales_price)
	 from dev.iceberg100.catalog_sales
		 ,dev.iceberg100.customer
		 ,dev.iceberg100.customer_address
		 ,dev.iceberg100.date_dim
	 where cs_bill_customer_sk = c_customer_sk
		and c_current_addr_sk = ca_address_sk
		and ( (substr(ca_zip,1,5) like '85669' or substr(ca_zip,1,5) like '86197' or substr(ca_zip,1,5) like '88274' or substr(ca_zip,1,5) like '83405'  or substr(ca_zip,1,5) like '86475'  or
									   substr(ca_zip,1,5) like '85392'  or substr(ca_zip,1,5) like '85460' or substr(ca_zip,1,5) like '80348' or substr(ca_zip,1,5) like '81792')
			  or (ca_state like 'CA' or ca_state like 'WA' or ca_state like 'GA')
			  or cs_sales_price > 500)
		and cs_sold_date_sk = d_date_sk
		and d_qoy = 1 and d_year = 2000
	 group by ca_zip
	 order by ca_zip
	 limit 100

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [70]:
%%sql

select * from dev.iceberg100.date_dim limit 10

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [72]:
%%sql
--q16

	select
	   count(distinct cs_order_number) as `order count`
	  ,sum(cs_ext_ship_cost) as `total shipping cost`
	  ,sum(cs_net_profit) as `total net profit`
	from
	   dev.iceberg100.catalog_sales cs1
	  ,dev.iceberg100.date_dim
	  ,dev.iceberg100.customer_address
	  ,dev.iceberg100.call_center
	where
		d_date between '2001-2-01' and
			   (cast('2001-2-01' as date) + INTERVAL 60 days)
	and cs1.cs_ship_date_sk = d_date_sk
	and cs1.cs_ship_addr_sk = ca_address_sk
	and ca_state = 'MS'
	and cs1.cs_call_center_sk = cc_call_center_sk
	and (cc_county LIKE 'Jackson County'
   OR cc_county LIKE 'Daviess County'
   OR cc_county LIKE 'Walker County'
   OR cc_county LIKE 'Dauphin County'
   OR cc_county LIKE 'Mobile County'
	)
	and exists (select *
				from dev.iceberg100.catalog_sales cs2
				where cs1.cs_order_number = cs2.cs_order_number
				  and cs1.cs_warehouse_sk <> cs2.cs_warehouse_sk)
	and not exists(select *
				   from dev.iceberg100.catalog_returns cr1
				   where cs1.cs_order_number = cr1.cr_order_number)
	order by count(distinct cs_order_number)
	limit 100

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [45]:
%%sql
--q17

	select  i_item_id
		   ,i_item_desc
		   ,s_state
		   ,count(ss_quantity) as store_sales_quantitycount
		   ,avg(ss_quantity) as store_sales_quantityave
		   ,stddev_samp(ss_quantity) as store_sales_quantitystdev
		   ,stddev_samp(ss_quantity)/avg(ss_quantity) as store_sales_quantitycov
		   ,count(sr_return_quantity) as store_returns_quantitycount
		   ,avg(sr_return_quantity) as store_returns_quantityave
		   ,stddev_samp(sr_return_quantity) as store_returns_quantitystdev
		   ,stddev_samp(sr_return_quantity)/avg(sr_return_quantity) as store_returns_quantitycov
		   ,count(cs_quantity) as catalog_sales_quantitycount ,avg(cs_quantity) as catalog_sales_quantityave
		   ,stddev_samp(cs_quantity) as catalog_sales_quantitystdev
		   ,stddev_samp(cs_quantity)/avg(cs_quantity) as catalog_sales_quantitycov
	 from dev.iceberg100.store_sales
		 ,dev.iceberg100.store_returns
		 ,dev.iceberg100.catalog_sales
		 ,dev.iceberg100.date_dim d1
		 ,dev.iceberg100.date_dim d2
		 ,dev.iceberg100.date_dim d3
		 ,dev.iceberg100.store
		 ,dev.iceberg100.item
	 where d1.d_quarter_name like '1999Q1'
	   and d1.d_date_sk = ss_sold_date_sk
	   and i_item_sk = ss_item_sk
	   and s_store_sk = ss_store_sk
	   and ss_customer_sk = sr_customer_sk
	   and ss_item_sk = sr_item_sk
	   and ss_ticket_number = sr_ticket_number
	   and sr_returned_date_sk = d2.d_date_sk
	   and (d2.d_quarter_name like '1999Q1' or d2.d_quarter_name like '1999Q2' or d2.d_quarter_name like '1999Q3')
	   and sr_customer_sk = cs_bill_customer_sk
	   and sr_item_sk = cs_item_sk
	   and cs_sold_date_sk = d3.d_date_sk
	   and (d3.d_quarter_name like '1999Q1' or d2.d_quarter_name like '1999Q2' or d2.d_quarter_name like '1999Q3')
	 group by i_item_id
			 ,i_item_desc
			 ,s_state
	 order by i_item_id
			 ,i_item_desc
			 ,s_state
	limit 100

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [46]:
%%sql
--q18

	select  i_item_id,
			ca_country,
			ca_state,
			ca_county,
			avg( cast(cs_quantity as decimal(12,2))) agg1,
			avg( cast(cs_list_price as decimal(12,2))) agg2,
			avg( cast(cs_coupon_amt as decimal(12,2))) agg3,
			avg( cast(cs_sales_price as decimal(12,2))) agg4,
			avg( cast(cs_net_profit as decimal(12,2))) agg5,
			avg( cast(c_birth_year as decimal(12,2))) agg6,
			avg( cast(cd1.cd_dep_count as decimal(12,2))) agg7
	 from dev.iceberg100.catalog_sales, dev.iceberg100.customer_demographics cd1,
		  dev.iceberg100.customer_demographics cd2, dev.iceberg100.customer, dev.iceberg100.customer_address, dev.iceberg100.date_dim, dev.iceberg100.item
	 where cs_sold_date_sk = d_date_sk and
		   cs_item_sk = i_item_sk and
		   cs_bill_cdemo_sk = cd1.cd_demo_sk and
		   cs_bill_customer_sk = c_customer_sk and
		   cd1.cd_gender = 'F' and
		   cd1.cd_education_status = 'Primary' and
		   c_current_cdemo_sk = cd2.cd_demo_sk and
		   c_current_addr_sk = ca_address_sk and
		   c_birth_month in (6,7,3,11,12,8) and
		   d_year = 1999 and
		   (ca_state like 'IL' or ca_state like 'WV' or ca_state like 'KS' or ca_state
					   like 'GA' or ca_state like 'LA' or ca_state like 'PA' or ca_state like 'TX')
	 group by rollup (i_item_id, ca_country, ca_state, ca_county)
	 order by ca_country,
			ca_state,
			ca_county,
		i_item_id
	 limit 100

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [47]:
%%sql
--q19

select  i_brand_id brand_id, i_brand brand, i_manufact_id, i_manufact,
 	sum(ss_ext_sales_price) ext_price
 from dev.iceberg100.date_dim, dev.iceberg100.store_sales, dev.iceberg100.item,dev.iceberg100.customer,dev.iceberg100.customer_address,dev.iceberg100.store
 where d_date_sk = ss_sold_date_sk
   and ss_item_sk = i_item_sk
   and i_manager_id=26
   and d_moy=12
   and d_year=2000
   and ss_customer_sk = c_customer_sk
   and c_current_addr_sk = ca_address_sk
   and substr(ca_zip,1,5) <> substr(s_zip,1,5)
   and ss_store_sk = s_store_sk
 group by i_brand
      ,i_brand_id
      ,i_manufact_id
      ,i_manufact
 order by ext_price desc
         ,i_brand
         ,i_brand_id
         ,i_manufact_id
         ,i_manufact
limit 100

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [48]:
%%sql
--q20

select  i_item_id
       ,i_item_desc
       ,i_category
       ,i_class
       ,i_current_price
       ,sum(cs_ext_sales_price) as itemrevenue
       ,sum(cs_ext_sales_price)*100/sum(sum(cs_ext_sales_price)) over
           (partition by i_class) as revenueratio
 from	dev.iceberg100.catalog_sales
     ,dev.iceberg100.item
     ,dev.iceberg100.date_dim
 where cs_item_sk = i_item_sk
   and (i_category like 'Books' or i_category like 'Home' or i_category like 'Jewelry')
   and cs_sold_date_sk = d_date_sk
 and d_date between cast('1998-05-08' as date)
 				and (cast('1998-05-08' as date) + INTERVAL 30 days)
 group by i_item_id
         ,i_item_desc
         ,i_category
         ,i_class
         ,i_current_price
 order by i_category
         ,i_class
         ,i_item_id
         ,i_item_desc
         ,revenueratio
limit 100

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [49]:
%%sql
--q21

select  *
 from(select w_warehouse_name
            ,i_item_id
            ,sum(case when (cast(d_date as date) < cast ('2000-05-22' as date))
	                then inv_quantity_on_hand
                      else 0 end) as inv_before
            ,sum(case when (cast(d_date as date) >= cast ('2000-05-22' as date))
                      then inv_quantity_on_hand
                      else 0 end) as inv_after
   from dev.iceberg100.inventory
       ,dev.iceberg100.warehouse
       ,dev.iceberg100.item
       ,dev.iceberg100.date_dim
   where i_current_price between 0.99 and 1.49
     and i_item_sk          = inv_item_sk
     and inv_warehouse_sk   = w_warehouse_sk
     and inv_date_sk    = d_date_sk
     and d_date between (cast ('2000-05-22' as date) - INTERVAL 30 days)
                    and (cast ('2000-05-22' as date) + INTERVAL 30 days)
   group by w_warehouse_name, i_item_id) x
 where (case when inv_before > 0
             then inv_after / inv_before
             else null
             end) between 2.0/3.0 and 3.0/2.0
 order by w_warehouse_name
         ,i_item_id
 limit 100

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [50]:
%%sql
--q22

select  i_product_name
             ,i_brand
             ,i_class
             ,i_category
             ,avg(inv_quantity_on_hand) qoh
       from dev.iceberg100.inventory
           ,dev.iceberg100.date_dim
           ,dev.iceberg100.item
       where inv_date_sk=d_date_sk
              and inv_item_sk=i_item_sk
              and d_month_seq between 1199 and 1199 + 11
       group by rollup(i_product_name
                       ,i_brand
                       ,i_class
                       ,i_category)
order by qoh, i_product_name, i_brand, i_class, i_category
limit 100

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [51]:
%%sql
--q23

with frequent_ss_items as
 (select substr(i_item_desc,1,30) itemdesc,i_item_sk item_sk,d_date solddate,count(*) cnt
  from dev.iceberg100.store_sales
      ,dev.iceberg100.date_dim
      ,dev.iceberg100.item
  where ss_sold_date_sk = d_date_sk
    and ss_item_sk = i_item_sk
    and d_year in (2000,2000+1,2000+2,2000+3)
  group by substr(i_item_desc,1,30),i_item_sk,d_date
  having count(*) >4),
 max_store_sales as
 (select max(csales) tpcds_cmax
  from (select c_customer_sk,sum(ss_quantity*ss_sales_price) csales
        from dev.iceberg100.store_sales
            ,dev.iceberg100.customer
            ,dev.iceberg100.date_dim
        where ss_customer_sk = c_customer_sk
         and ss_sold_date_sk = d_date_sk
         and d_year in (2000,2000+1,2000+2,2000+3)
        group by c_customer_sk)),
 best_ss_customer as
 (select c_customer_sk,sum(ss_quantity*ss_sales_price) ssales
  from dev.iceberg100.store_sales
      ,dev.iceberg100.customer
  where ss_customer_sk = c_customer_sk
  group by c_customer_sk
  having sum(ss_quantity*ss_sales_price) > (95/100.0) * (select
  *
from
 max_store_sales))
  select  sum(sales)
 from (select cs_quantity*cs_list_price sales
       from dev.iceberg100.catalog_sales
           ,dev.iceberg100.date_dim
       where d_year = 2000
         and d_moy = 5
         and cs_sold_date_sk = d_date_sk
         and cs_item_sk in (select item_sk from frequent_ss_items)
         and cs_bill_customer_sk in (select c_customer_sk from best_ss_customer)
      union all
      select ws_quantity*ws_list_price sales
       from dev.iceberg100.web_sales
           ,dev.iceberg100.date_dim
       where d_year = 2000
         and d_moy = 5
         and ws_sold_date_sk = d_date_sk
         and ws_item_sk in (select item_sk from frequent_ss_items)
         and ws_bill_customer_sk in (select c_customer_sk from best_ss_customer))
 limit 100

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [52]:
%%sql
--q24

with ssales as
(select c_last_name
      ,c_first_name
      ,s_store_name
      ,ca_state
      ,s_state
      ,i_color
      ,i_current_price
      ,i_manager_id
      ,i_units
      ,i_size
      ,sum(ss_net_paid_inc_tax) netpaid
from dev.iceberg100.store_sales
    ,dev.iceberg100.store_returns
    ,dev.iceberg100.store
    ,dev.iceberg100.item
    ,dev.iceberg100.customer
    ,dev.iceberg100.customer_address
where ss_ticket_number = sr_ticket_number
  and ss_item_sk = sr_item_sk
  and ss_customer_sk = c_customer_sk
  and ss_item_sk = i_item_sk
  and ss_store_sk = s_store_sk
  and c_current_addr_sk = ca_address_sk
  and c_birth_country <> upper(ca_country)
  and s_zip = ca_zip
and s_market_id=10
group by c_last_name
        ,c_first_name
        ,s_store_name
        ,ca_state
        ,s_state
        ,i_color
        ,i_current_price
        ,i_manager_id
        ,i_units
        ,i_size)
select c_last_name
      ,c_first_name
      ,s_store_name
      ,sum(netpaid) paid
from ssales
where i_color = 'navy'
group by c_last_name
        ,c_first_name
        ,s_store_name
having sum(netpaid) > (select 0.05*avg(netpaid)
                                 from ssales)
order by c_last_name
        ,c_first_name
        ,s_store_name

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [53]:
%%sql
--q25

select
 i_item_id
 ,i_item_desc
 ,s_store_id
 ,s_store_name
 ,sum(ss_net_profit) as store_sales_profit
 ,sum(sr_net_loss) as store_returns_loss
 ,sum(cs_net_profit) as catalog_sales_profit
 from
 dev.iceberg100.store_sales
 ,dev.iceberg100.store_returns
 ,dev.iceberg100.catalog_sales
 ,dev.iceberg100.date_dim d1
 ,dev.iceberg100.date_dim d2
 ,dev.iceberg100.date_dim d3
 ,dev.iceberg100.store
 ,dev.iceberg100.item
 where
 d1.d_moy = 4
 and d1.d_year = 2002
 and d1.d_date_sk = ss_sold_date_sk
 and i_item_sk = ss_item_sk
 and s_store_sk = ss_store_sk
 and ss_customer_sk = sr_customer_sk
 and ss_item_sk = sr_item_sk
 and ss_ticket_number = sr_ticket_number
 and sr_returned_date_sk = d2.d_date_sk
 and d2.d_moy               between 4 and  10
 and d2.d_year              = 2002
 and sr_customer_sk = cs_bill_customer_sk
 and sr_item_sk = cs_item_sk
 and cs_sold_date_sk = d3.d_date_sk
 and d3.d_moy               between 4 and  10
 and d3.d_year              = 2002
 group by
 i_item_id
 ,i_item_desc
 ,s_store_id
 ,s_store_name
 order by
 i_item_id
 ,i_item_desc
 ,s_store_id
 ,s_store_name
 limit 100

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [54]:
%%sql
--q26

select  i_item_id,
        avg(cs_quantity) agg1,
        avg(cs_list_price) agg2,
        avg(cs_coupon_amt) agg3,
        avg(cs_sales_price) agg4
 from dev.iceberg100.catalog_sales, dev.iceberg100.customer_demographics, dev.iceberg100.date_dim, dev.iceberg100.item, dev.iceberg100.promotion
 where cs_sold_date_sk = d_date_sk and
       cs_item_sk = i_item_sk and
       cs_bill_cdemo_sk = cd_demo_sk and
       cs_promo_sk = p_promo_sk and
       cd_gender = 'F' and
       cd_marital_status = 'M' and
       cd_education_status = '2 yr Degree' and
       (p_channel_email = 'N' or p_channel_event = 'N') and
       d_year = 2002
 group by i_item_id
 order by i_item_id
 limit 100

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [55]:
%%sql
--q27

select  i_item_id,
        s_state, grouping(s_state) g_state,
        avg(ss_quantity) agg1,
        avg(ss_list_price) agg2,
        avg(ss_coupon_amt) agg3,
        avg(ss_sales_price) agg4
 from dev.iceberg100.store_sales, dev.iceberg100.customer_demographics, dev.iceberg100.date_dim, dev.iceberg100.store, dev.iceberg100.item
 where ss_sold_date_sk = d_date_sk and
       ss_item_sk = i_item_sk and
       ss_store_sk = s_store_sk and
       ss_cdemo_sk = cd_demo_sk and
       cd_gender = 'F' and
       cd_marital_status = 'S' and
       cd_education_status = 'Advanced Degree' and
       d_year = 2000 and
       (s_state like 'WA' or s_state like 'LA' or s_state like 'LA' or s_state like 'TX' or s_state like 'AL' or s_state like 'PA')
 group by rollup (i_item_id, s_state)
 order by i_item_id
         ,s_state
 limit 100

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [56]:
%%sql
--q28

select  *
from (select avg(ss_list_price) B1_LP
            ,count(ss_list_price) B1_CNT
            ,count(distinct ss_list_price) B1_CNTD
      from dev.iceberg100.store_sales
      where ss_quantity between 0 and 5
        and (ss_list_price between 189 and 189+10
             or ss_coupon_amt between 4483 and 4483+1000
             or ss_wholesale_cost between 24 and 24+20)) B1,
     (select avg(ss_list_price) B2_LP
            ,count(ss_list_price) B2_CNT
            ,count(distinct ss_list_price) B2_CNTD
      from dev.iceberg100.store_sales
      where ss_quantity between 6 and 10
        and (ss_list_price between 71 and 71+10
          or ss_coupon_amt between 14775 and 14775+1000
          or ss_wholesale_cost between 38 and 38+20)) B2,
     (select avg(ss_list_price) B3_LP
            ,count(ss_list_price) B3_CNT
            ,count(distinct ss_list_price) B3_CNTD
      from dev.iceberg100.store_sales
      where ss_quantity between 11 and 15
        and (ss_list_price between 183 and 183+10
          or ss_coupon_amt between 13456 and 13456+1000
          or ss_wholesale_cost between 31 and 31+20)) B3,
     (select avg(ss_list_price) B4_LP
            ,count(ss_list_price) B4_CNT
            ,count(distinct ss_list_price) B4_CNTD
      from dev.iceberg100.store_sales
      where ss_quantity between 16 and 20
        and (ss_list_price between 135 and 135+10
          or ss_coupon_amt between 4905 and 4905+1000
          or ss_wholesale_cost between 27 and 27+20)) B4,
     (select avg(ss_list_price) B5_LP
            ,count(ss_list_price) B5_CNT
            ,count(distinct ss_list_price) B5_CNTD
      from dev.iceberg100.store_sales
      where ss_quantity between 21 and 25
        and (ss_list_price between 180 and 180+10
          or ss_coupon_amt between 17430 and 17430+1000
          or ss_wholesale_cost between 57 and 57+20)) B5,
     (select avg(ss_list_price) B6_LP
            ,count(ss_list_price) B6_CNT
            ,count(distinct ss_list_price) B6_CNTD
      from dev.iceberg100.store_sales
      where ss_quantity between 26 and 30
        and (ss_list_price between 49 and 49+10
          or ss_coupon_amt between 2950 and 2950+1000
          or ss_wholesale_cost between 52 and 52+20)) B6
limit 100

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [57]:
%%sql
--q29

select
     i_item_id
    ,i_item_desc
    ,s_store_id
    ,s_store_name
    ,stddev_samp(ss_quantity)        as store_sales_quantity
    ,stddev_samp(sr_return_quantity) as store_returns_quantity
    ,stddev_samp(cs_quantity)        as catalog_sales_quantity
 from
    dev.iceberg100.store_sales
   ,dev.iceberg100.store_returns
   ,dev.iceberg100.catalog_sales
   ,dev.iceberg100.date_dim             d1
   ,dev.iceberg100.date_dim             d2
   ,dev.iceberg100.date_dim             d3
   ,dev.iceberg100.store
   ,dev.iceberg100.item
 where
     d1.d_moy               = 4
 and d1.d_year              = 1998
 and d1.d_date_sk           = ss_sold_date_sk
 and i_item_sk              = ss_item_sk
 and s_store_sk             = ss_store_sk
 and ss_customer_sk         = sr_customer_sk
 and ss_item_sk             = sr_item_sk
 and ss_ticket_number       = sr_ticket_number
 and sr_returned_date_sk    = d2.d_date_sk
 and d2.d_moy               between 4 and  4 + 3
 and d2.d_year              = 1998
 and sr_customer_sk         = cs_bill_customer_sk
 and sr_item_sk             = cs_item_sk
 and cs_sold_date_sk        = d3.d_date_sk
 and d3.d_year              in (1998,1998+1,1998+2)
 group by
    i_item_id
   ,i_item_desc
   ,s_store_id
   ,s_store_name
 order by
    i_item_id
   ,i_item_desc
   ,s_store_id
   ,s_store_name
 limit 100

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [68]:
%%sql
--q30

with customer_total_return as
 (select wr_returning_customer_sk as ctr_customer_sk
        ,ca_state as ctr_state,
 	sum(wr_return_amt) as ctr_total_return
 from dev.iceberg100.web_returns
     ,dev.iceberg100.date_dim
     ,dev.iceberg100.customer_address
 where wr_returned_date_sk = d_date_sk
   and d_year =2000
   and wr_returning_addr_sk = ca_address_sk
 group by wr_returning_customer_sk
         ,ca_state)
  select  c_customer_id,c_salutation,c_first_name,c_last_name,c_preferred_cust_flag
       ,c_birth_day,c_birth_month,c_birth_year,c_birth_country,c_login,c_email_address
       ,dev.iceberg100.customer_address.c_last_review_date,ctr_total_return
 from customer_total_return ctr1
     ,dev.iceberg100.customer_address
     ,dev.iceberg100.customer
 where ctr1.ctr_total_return > (select avg(ctr_total_return)*1.2
 			  from customer_total_return ctr2
                  	  where ctr1.ctr_state = ctr2.ctr_state)
       and ca_address_sk = c_current_addr_sk
       and ca_state = 'GA'
       and ctr1.ctr_customer_sk = c_customer_sk
 order by c_customer_id,c_salutation,c_first_name,c_last_name,c_preferred_cust_flag
                  ,c_birth_day,c_birth_month,c_birth_year,c_birth_country,c_login,c_email_address
                  ,dev.iceberg100.customer_address.c_last_review_date,ctr_total_return
limit 100

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
[UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `dev`.`iceberg100`.`customer_address`.`c_last_review_date` cannot be resolved. Did you mean one of the following? [`dev`.`iceberg100`.`customer_address`.`ca_street_name`, `dev`.`iceberg100`.`customer_address`.`ca_street_type`, `dev`.`iceberg100`.`customer`.`c_last_review_date_sk`, `dev`.`iceberg100`.`customer_address`.`ca_state`, `dev`.`iceberg100`.`customer_address`.`ca_street_number`].; line 17 pos 8;
'WithCTE
:- CTERelationDef 27, false
:  +- SubqueryAlias customer_total_return
:     +- Aggregate [wr_returning_customer_sk#23795, ca_state#23848], [wr_returning_customer_sk#23795 AS ctr_customer_sk#23785, ca_state#23848 AS ctr_state#23786, sum(wr_return_amt#23803) AS ctr_total_return#23787]
:        +- Filter (((wr_returned_date_sk#23788 = d_date_sk#23812) AND (d_year#23818 = 2000)) AND (wr_returning_addr_sk#23798 = ca_address_sk#23840))
:           +- Join Inner
:              :- Joi

In [59]:
%%sql
--q31

with ss as
 (select ca_county,d_qoy, d_year,sum(ss_ext_sales_price) as store_sales
 from dev.iceberg100.store_sales,dev.iceberg100.date_dim,dev.iceberg100.customer_address
 where ss_sold_date_sk = d_date_sk
  and ss_addr_sk=ca_address_sk
 group by ca_county,d_qoy, d_year),
 ws as
 (select ca_county,d_qoy, d_year,sum(ws_ext_sales_price) as web_sales
 from dev.iceberg100.web_sales,dev.iceberg100.date_dim,dev.iceberg100.customer_address
 where ws_sold_date_sk = d_date_sk
  and ws_bill_addr_sk=ca_address_sk
 group by ca_county,d_qoy, d_year)
 select
        ss1.ca_county
       ,ss1.d_year
       ,ws2.web_sales/ws1.web_sales web_q1_q2_increase
       ,ss2.store_sales/ss1.store_sales store_q1_q2_increase
       ,ws3.web_sales/ws2.web_sales web_q2_q3_increase
       ,ss3.store_sales/ss2.store_sales store_q2_q3_increase
 from
        ss ss1
       ,ss ss2
       ,ss ss3
       ,ws ws1
       ,ws ws2
       ,ws ws3
 where
    ss1.d_qoy = 1
    and ss1.d_year = 1998
    and ss1.ca_county = ss2.ca_county
    and ss2.d_qoy = 2
    and ss2.d_year = 1998
 and ss2.ca_county = ss3.ca_county
    and ss3.d_qoy = 3
    and ss3.d_year = 1998
    and ss1.ca_county = ws1.ca_county
    and ws1.d_qoy = 1
    and ws1.d_year = 1998
    and ws1.ca_county = ws2.ca_county
    and ws2.d_qoy = 2
    and ws2.d_year = 1998
    and ws1.ca_county = ws3.ca_county
    and ws3.d_qoy = 3
    and ws3.d_year =1998
    and case when ws1.web_sales > 0 then ws2.web_sales/ws1.web_sales else null end
       > case when ss1.store_sales > 0 then ss2.store_sales/ss1.store_sales else null end
    and case when ws2.web_sales > 0 then ws3.web_sales/ws2.web_sales else null end
       > case when ss2.store_sales > 0 then ss3.store_sales/ss2.store_sales else null end
 order by ss1.ca_county

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [60]:
%%sql
--q32

select  sum(cs_ext_discount_amt)  as `excess discount amount`
from
   dev.iceberg100.catalog_sales
   ,dev.iceberg100.item
   ,dev.iceberg100.date_dim
where
i_manufact_id = 948
and i_item_sk = cs_item_sk
and d_date between '1998-02-03' and
        (cast('1998-02-03' as date) + INTERVAL 90 days)
and d_date_sk = cs_sold_date_sk
and cs_ext_discount_amt
     > (
         select
            1.3 * avg(cs_ext_discount_amt)
         from
            dev.iceberg100.catalog_sales
           ,dev.iceberg100.date_dim
         where
              cs_item_sk = i_item_sk
          and d_date between '1998-02-03' and
                             (cast('1998-02-03' as date) + INTERVAL 90 days)
          and d_date_sk = cs_sold_date_sk
      )
limit 100

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [61]:
%%sql
--q33

with ss as (
 select
          i_manufact_id,sum(ss_ext_sales_price) total_sales
 from
 	dev.iceberg100.store_sales,
 	dev.iceberg100.date_dim,
         dev.iceberg100.customer_address,
         dev.iceberg100.item
 where
         i_manufact_id in (select
  i_manufact_id
from
 dev.iceberg100.item
where i_category like 'Electronics')
 and     ss_item_sk              = i_item_sk
 and     ss_sold_date_sk         = d_date_sk
 and     d_year                  = 1999
 and     d_moy                   = 2
 and     ss_addr_sk              = ca_address_sk
 and     ca_gmt_offset           = -6
 group by i_manufact_id),
 cs as (
 select
          i_manufact_id,sum(cs_ext_sales_price) total_sales
 from
 	dev.iceberg100.catalog_sales,
 	dev.iceberg100.date_dim,
         dev.iceberg100.customer_address,
         dev.iceberg100.item
 where
         i_manufact_id               in (select
  i_manufact_id
from
 dev.iceberg100.item
where i_category like 'Electronics')
 and     cs_item_sk              = i_item_sk
 and     cs_sold_date_sk         = d_date_sk
 and     d_year                  = 1999
 and     d_moy                   = 2
 and     cs_bill_addr_sk         = ca_address_sk
 and     ca_gmt_offset           = -6
 group by i_manufact_id),
 ws as (
 select
          i_manufact_id,sum(ws_ext_sales_price) total_sales
 from
 	dev.iceberg100.web_sales,
 	dev.iceberg100.date_dim,
         dev.iceberg100.customer_address,
         dev.iceberg100.item
 where
         i_manufact_id               in (select
  i_manufact_id
from
 dev.iceberg100.item
where i_category like 'Electronics')
 and     ws_item_sk              = i_item_sk
 and     ws_sold_date_sk         = d_date_sk
 and     d_year                  = 1999
 and     d_moy                   = 2
 and     ws_bill_addr_sk         = ca_address_sk
 and     ca_gmt_offset           = -6
 group by i_manufact_id)
  select  i_manufact_id ,sum(total_sales) total_sales
 from  (select * from ss
        union all
        select * from cs
        union all
        select * from ws) tmp1
 group by i_manufact_id
 order by total_sales
limit 100

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [62]:
%%sql
--#q34

select c_last_name
       ,c_first_name
       ,c_salutation
       ,c_preferred_cust_flag
       ,ss_ticket_number
       ,cnt from
   (select ss_ticket_number
          ,ss_customer_sk
          ,count(*) cnt
    from dev.iceberg100.dev.iceberg100.store_sales,dev.iceberg100.date_dim,dev.iceberg100.store,dev.iceberg100.household_demographics
    where dev.iceberg100.store_sales.ss_sold_date_sk = dev.iceberg100.date_dim.d_date_sk
    and dev.iceberg100.store_sales.ss_store_sk = dev.iceberg100.store.s_store_sk
    and dev.iceberg100.store_sales.ss_hdemo_sk = dev.iceberg100.household_demographics.hd_demo_sk
    and (dev.iceberg100.date_dim.d_dom between 1 and 3 or dev.iceberg100.date_dim.d_dom between 25 and 28)
    and (dev.iceberg100.household_demographics.hd_buy_potential = '>10000' or
         dev.iceberg100.household_demographics.hd_buy_potential = '5001-10000')
    and dev.iceberg100.household_demographics.hd_vehicle_count > 0
    and (case when dev.iceberg100.household_demographics.hd_vehicle_count > 0
	then dev.iceberg100.household_demographics.hd_dep_count/ dev.iceberg100.household_demographics.hd_vehicle_count
	else null
	end)  > 1.2
    and dev.iceberg100.date_dim.d_year in (1999,1999+1,1999+2)
    and (dev.iceberg100.store.s_county like 'Jefferson Davis Parish' or dev.iceberg100.store.s_county like 'Levy County' or dev.iceberg100.store.s_county like 'Coal County' or dev.iceberg100.store.s_county like 'Oglethorpe County' or dev.iceberg100.store.s_county like 
                           'Mobile County' or dev.iceberg100.store.s_county like 'Gage County' or dev.iceberg100.store.s_county like 'Richland County' or dev.iceberg100.store.s_county like 'Gogebic County')
    group by ss_ticket_number,ss_customer_sk) dn,dev.iceberg100.customer
    where ss_customer_sk = c_customer_sk
      and cnt between 15 and 20
    order by c_last_name,c_first_name,c_salutation,c_preferred_cust_flag desc, ss_ticket_number

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
invalid syntax (<stdin>, line 1)
  File "<stdin>", line 1
    select c_last_name
                     ^
SyntaxError: invalid syntax



In [ ]:
%%sql
--#q35

select
  ca_state,
  cd_gender,
  cd_marital_status,
  cd_dep_count,
  count(*) cnt1,
  stddev_samp(cd_dep_count),
  stddev_samp(cd_dep_count),
  min(cd_dep_count),
  cd_dep_employed_count,
  count(*) cnt2,
  stddev_samp(cd_dep_employed_count),
  stddev_samp(cd_dep_employed_count),
  min(cd_dep_employed_count),
  cd_dep_college_count,
  count(*) cnt3,
  stddev_samp(cd_dep_college_count),
  stddev_samp(cd_dep_college_count),
  min(cd_dep_college_count)
 from
  dev.iceberg100.customer c,dev.iceberg100.customer_address ca,dev.iceberg100.customer_demographics
 where
  c.c_current_addr_sk = ca.ca_address_sk and
  cd_demo_sk = c.c_current_cdemo_sk and
  exists (select *
          from dev.iceberg100.store_sales,dev.iceberg100.date_dim
          where c.c_customer_sk = ss_customer_sk and
                ss_sold_date_sk = d_date_sk and
                d_year = 2002 and
                d_qoy < 4) and
   (exists (select *
            from dev.iceberg100.web_sales,dev.iceberg100.date_dim
            where c.c_customer_sk = ws_bill_customer_sk and
                  ws_sold_date_sk = d_date_sk and
                  d_year = 2002 and
                  d_qoy < 4) or
    exists (select *
            from dev.iceberg100.catalog_sales,dev.iceberg100.date_dim
            where c.c_customer_sk = cs_ship_customer_sk and
                  cs_sold_date_sk = d_date_sk and
                  d_year = 2002 and
                  d_qoy < 4))
 group by ca_state,
          cd_gender,
          cd_marital_status,
          cd_dep_count,
          cd_dep_employed_count,
          cd_dep_college_count
 order by ca_state,
          cd_gender,
          cd_marital_status,
          cd_dep_count,
          cd_dep_employed_count,
          cd_dep_college_count
 limit 100

In [ ]:
%%sql
--#q36


select
    sum(ss_net_profit)/sum(ss_ext_sales_price) as gross_margin
   ,i_category
   ,i_class
   ,grouping(i_category)+grouping(i_class) as lochierarchy
   ,rank() over (
 	partition by grouping(i_category)+grouping(i_class),
 	case when grouping(i_class) = 0 then i_category end
 	order by sum(ss_net_profit)/sum(ss_ext_sales_price) asc) as rank_within_parent
 from
    dev.iceberg100.store_sales
   ,dev.iceberg100.date_dim       d1
   ,dev.iceberg100.item
   ,dev.iceberg100.store
 where
    d1.d_year = 1998
 and d1.d_date_sk = ss_sold_date_sk
 and i_item_sk  = ss_item_sk
 and s_store_sk  = ss_store_sk
 and 
 (s_state LIKE 'OH%' OR
        s_state LIKE 'WV' OR
        s_state LIKE 'PA' OR
        s_state LIKE 'TN' OR
        s_state LIKE 'MN' OR
        s_state LIKE 'MO' OR
        s_state LIKE 'NM' OR
        s_state LIKE 'MI')
 group by rollup(i_category,i_class)
 order by
   lochierarchy desc
  ,case when lochierarchy = 0 then i_category end
  ,rank_within_parent
  limit 100

In [ ]:
%%sql
--#q37

select  i_item_id
       ,i_item_desc
       ,i_current_price
 from dev.iceberg100.item, dev.iceberg100.inventory, dev.iceberg100.date_dim, dev.iceberg100.catalog_sales
 where i_current_price between 35 and 35 + 30
 and inv_item_sk = i_item_sk
 and d_date_sk=inv_date_sk
 and d_date between cast('2001-01-20' as date) and (cast('2001-01-20' as date) + interval 60 days)
 and i_manufact_id in (928,715,942,861)
 and inv_quantity_on_hand between 100 and 500
 and cs_item_sk = i_item_sk
 group by i_item_id,i_item_desc,i_current_price
 order by i_item_id
 limit 100""",
    "q38": f"""
select  count(*) from (
    select distinct c_last_name, c_first_name, d_date
    from dev.iceberg100.store_sales, dev.iceberg100.date_dim, dev.iceberg100.customer
          where store_sales.ss_sold_date_sk = dev.iceberg100.date_dim.d_date_sk
      and store_sales.ss_customer_sk = dev.iceberg100.customer.c_customer_sk
      and d_month_seq between 1222 and 1222 + 11
  intersect
    select distinct c_last_name, c_first_name, d_date
    from dev.iceberg100.catalog_sales, dev.iceberg100.date_dim, dev.iceberg100.customer
          where dev.iceberg100.catalog_sales.cs_sold_date_sk = dev.iceberg100.date_dim.d_date_sk
      and dev.iceberg100.catalog_sales.cs_bill_customer_sk = dev.iceberg100.customer.c_customer_sk
      and d_month_seq between 1222 and 1222 + 11
  intersect
    select distinct c_last_name, c_first_name, d_date
    from dev.iceberg100.web_sales, dev.iceberg100.date_dim, dev.iceberg100.customer
          where dev.iceberg100.web_sales.ws_sold_date_sk = dev.iceberg100.date_dim.d_date_sk
      and dev.iceberg100.web_sales.ws_bill_customer_sk = dev.iceberg100.customer.c_customer_sk
      and d_month_seq between 1222 and 1222 + 11
) hot_cust
limit 100"""

In [ ]:
%%sql
--#q39

with inv as
(select w_warehouse_name,w_warehouse_sk,i_item_sk,d_moy
       ,stdev,mean, case mean when 0 then null else stdev/mean end cov
 from(select w_warehouse_name,w_warehouse_sk,i_item_sk,d_moy
            ,stddev_samp(inv_quantity_on_hand) stdev,avg(inv_quantity_on_hand) mean
      from dev.iceberg100.inventory
          ,dev.iceberg100.item
          ,dev.iceberg100.warehouse
          ,dev.iceberg100.date_dim
      where inv_item_sk = i_item_sk
        and inv_warehouse_sk = w_warehouse_sk
        and inv_date_sk = d_date_sk
        and d_year =1998
      group by w_warehouse_name,w_warehouse_sk,i_item_sk,d_moy) foo
 where case mean when 0 then 0 else stdev/mean end > 1)
select inv1.w_warehouse_sk,inv1.i_item_sk,inv1.d_moy,inv1.mean, inv1.cov
        ,inv2.w_warehouse_sk,inv2.i_item_sk,inv2.d_moy,inv2.mean, inv2.cov
from inv inv1,inv inv2
where inv1.i_item_sk = inv2.i_item_sk
  and inv1.w_warehouse_sk =  inv2.w_warehouse_sk
  and inv1.d_moy=4
  and inv2.d_moy=4+1
order by inv1.w_warehouse_sk,inv1.i_item_sk,inv1.d_moy,inv1.mean,inv1.cov
        ,inv2.d_moy,inv2.mean, inv2.cov

In [ ]:
%%sql
--#q40

select
   w_state
  ,i_item_id
  ,sum(case when (cast(d_date as date) < cast ('1999-02-02' as date))
 		then cs_sales_price - coalesce(cr_refunded_cash,0) else 0 end) as sales_before
  ,sum(case when (cast(d_date as date) >= cast ('1999-02-02' as date))
 		then cs_sales_price - coalesce(cr_refunded_cash,0) else 0 end) as sales_after
 from
   dev.iceberg100.catalog_sales left outer join dev.iceberg100.catalog_returns on
       (cs_order_number = cr_order_number
        and cs_item_sk = cr_item_sk)
  ,dev.iceberg100.warehouse
  ,dev.iceberg100.item
  ,dev.iceberg100.date_dim
 where
     i_current_price between 0.99 and 1.49
 and i_item_sk          = cs_item_sk
 and cs_warehouse_sk    = w_warehouse_sk
 and cs_sold_date_sk    = d_date_sk
 and d_date between (cast ('1999-02-02' as date) - INTERVAL 30 days)
                and (cast ('1999-02-02' as date) + INTERVAL 30 days)
 group by
    w_state,i_item_id
 order by w_state,i_item_id
limit 100

In [ ]:
%%sql
-#q41

select  distinct(i_product_name)
 from dev.iceberg100.item i1
 where i_manufact_id between 732 and 732+40
   and (select count(*) as item_cnt
        from dev.iceberg100.item
        where (i_manufact = i1.i_manufact and
        ((i_category = 'Women' and
        (i_color = 'beige' or i_color = 'spring') and
        (i_units = 'Tsp' or i_units = 'Ton') and
        (i_size = 'petite' or i_size = 'extra large')
        ) or
        (i_category = 'Women' and
        (i_color = 'white' or i_color = 'pale') and
        (i_units = 'Box' or i_units = 'Dram') and
        (i_size = 'large' or i_size = 'economy')
        ) or
        (i_category = 'Men' and
        (i_color = 'midnight' or i_color = 'frosted') and
        (i_units = 'Bunch' or i_units = 'Carton') and
        (i_size = 'small' or i_size = 'N/A')
        ) or
        (i_category = 'Men' and
        (i_color = 'azure' or i_color = 'goldenrod') and
        (i_units = 'Pallet' or i_units = 'Gross') and
        (i_size = 'petite' or i_size = 'extra large')
        ))) or
       (i_manufact = i1.i_manufact and
        ((i_category = 'Women' and
        (i_color = 'brown' or i_color = 'hot') and
        (i_units = 'Tbl' or i_units = 'Cup') and
        (i_size = 'petite' or i_size = 'extra large')
        ) or
        (i_category = 'Women' and
        (i_color = 'powder' or i_color = 'honeydew') and
        (i_units = 'Bundle' or i_units = 'Unknown') and
        (i_size = 'large' or i_size = 'economy')
        ) or
        (i_category = 'Men' and
        (i_color = 'antique' or i_color = 'purple') and
        (i_units = 'N/A' or i_units = 'Dozen') and
        (i_size = 'small' or i_size = 'N/A')
        ) or
        (i_category = 'Men' and
        (i_color = 'lavender' or i_color = 'tomato') and
        (i_units = 'Lb' or i_units = 'Oz') and
        (i_size = 'petite' or i_size = 'extra large')
        )))) > 0
 order by i_product_name
 limit 100

In [ ]:
%%sql
--#q42

select  dt.d_year
 	,dev.iceberg100.item.i_category_id
 	,dev.iceberg100.item.i_category
 	,sum(ss_ext_sales_price)
 from 	dev.iceberg100.date_dim dt
 	,dev.iceberg100.store_sales
 	,dev.iceberg100.item
 where dt.d_date_sk = dev.iceberg100.store_sales.ss_sold_date_sk
 	and dev.iceberg100.store_sales.ss_item_sk = dev.iceberg100.item.i_item_sk
 	and dev.iceberg100.item.i_manager_id = 1
 	and dt.d_moy=11
 	and dt.d_year=2002
 group by 	dt.d_year
 		,dev.iceberg100.item.i_category_id
 		,dev.iceberg100.item.i_category
 order by       sum(ss_ext_sales_price) desc,dt.d_year
 		,dev.iceberg100.item.i_category_id
 		,dev.iceberg100.item.i_category
limit 100

In [ ]:
%%sql
--#q43

select  s_store_name, s_store_id,
        sum(case when (d_day_name='Sunday') then ss_sales_price else null end) sun_sales,
        sum(case when (d_day_name='Monday') then ss_sales_price else null end) mon_sales,
        sum(case when (d_day_name='Tuesday') then ss_sales_price else  null end) tue_sales,
        sum(case when (d_day_name='Wednesday') then ss_sales_price else null end) wed_sales,
        sum(case when (d_day_name='Thursday') then ss_sales_price else null end) thu_sales,
        sum(case when (d_day_name='Friday') then ss_sales_price else null end) fri_sales,
        sum(case when (d_day_name='Saturday') then ss_sales_price else null end) sat_sales
 from dev.iceberg100.date_dim, dev.iceberg100.store_sales, dev.iceberg100.store
 where d_date_sk = ss_sold_date_sk and
       s_store_sk = ss_store_sk and
       s_gmt_offset = -6 and
       d_year = 1999
 group by s_store_name, s_store_id
 order by s_store_name, s_store_id,sun_sales,mon_sales,tue_sales,wed_sales,thu_sales,fri_sales,sat_sales
 limit 100

In [ ]:
%%sql
--#q44

select  asceding.rnk, i1.i_product_name best_performing, i2.i_product_name worst_performing
from(select *
     from (select item_sk,rank() over (order by rank_col asc) rnk
           from (select ss_item_sk item_sk,avg(ss_net_profit) rank_col
                 from dev.iceberg100.store_sales ss1
                 where ss_store_sk = 321
                 group by ss_item_sk
                 having avg(ss_net_profit) > 0.9*(select avg(ss_net_profit) rank_col
                                                  from dev.iceberg100.store_sales
                                                  where ss_store_sk = 321
                                                    and ss_addr_sk is null
                                                  group by ss_store_sk))V1)V11
     where rnk  < 11) asceding,
    (select *
     from (select item_sk,rank() over (order by rank_col desc) rnk
           from (select ss_item_sk item_sk,avg(ss_net_profit) rank_col
                 from dev.iceberg100.store_sales ss1
                 where ss_store_sk = 321
                 group by ss_item_sk
                 having avg(ss_net_profit) > 0.9*(select avg(ss_net_profit) rank_col
                                                  from dev.iceberg100.store_sales
                                                  where ss_store_sk = 321
                                                    and ss_addr_sk is null
                                                  group by ss_store_sk))V2)V21
     where rnk  < 11) descending,
dev.iceberg100.item i1,
dev.iceberg100.item i2
where asceding.rnk = descending.rnk
  and i1.i_item_sk=asceding.item_sk
  and i2.i_item_sk=descending.item_sk
order by asceding.rnk
limit 100

In [ ]:
%%sql
--#q45

select  ca_zip, ca_county, sum(ws_sales_price)
 from dev.iceberg100.web_sales, dev.iceberg100.customer, dev.iceberg100.customer_address, dev.iceberg100.date_dim, dev.iceberg100.item
 where ws_bill_customer_sk = c_customer_sk
 	and c_current_addr_sk = ca_address_sk
 	and ws_item_sk = i_item_sk
 	and ( 
        (SUBSTR(ca_zip, 1, 5) LIKE '85669' OR
        SUBSTR(ca_zip, 1, 5) LIKE '86197' OR
        SUBSTR(ca_zip, 1, 5) LIKE '88274' OR
        SUBSTR(ca_zip, 1, 5) LIKE '83405' OR
        SUBSTR(ca_zip, 1, 5) LIKE '86475' OR
        SUBSTR(ca_zip, 1, 5) LIKE '85392' OR
        SUBSTR(ca_zip, 1, 5) LIKE '85460' OR
        SUBSTR(ca_zip, 1, 5) LIKE '80348' OR
        SUBSTR(ca_zip, 1, 5) LIKE '81792')
 	      or
 	      i_item_id in (select i_item_id
                             from dev.iceberg100.item
                             where i_item_sk in (2, 3, 5, 7, 11, 13, 17, 19, 23, 29)
                             )
 	    )
 	and ws_sold_date_sk = d_date_sk
 	and d_qoy = 2 and d_year = 1999
 group by ca_zip, ca_county
 order by ca_zip, ca_county
 limit 100

In [ ]:
%%sql
--#q46

select  c_last_name
       ,c_first_name
       ,ca_city
       ,bought_city
       ,ss_ticket_number
       ,amt,profit
 from
   (select ss_ticket_number
          ,ss_customer_sk
          ,ca_city bought_city
          ,sum(ss_coupon_amt) amt
          ,sum(ss_net_profit) profit
    from dev.iceberg100.store_sales,dev.iceberg100.date_dim,dev.iceberg100.store,dev.iceberg100.household_demographics,dev.iceberg100.customer_address
    where dev.iceberg100.store_sales.ss_sold_date_sk = date_dim.d_date_sk
    and dev.iceberg100.store_sales.ss_store_sk = dev.iceberg100.store.s_store_sk
    and dev.iceberg100.store_sales.ss_hdemo_sk = dev.iceberg100.household_demographics.hd_demo_sk
    and dev.iceberg100.store_sales.ss_addr_sk = dev.iceberg100.customer_address.ca_address_sk
    and (dev.iceberg100.household_demographics.hd_dep_count = 2 or
         dev.iceberg100.household_demographics.hd_vehicle_count= 2)
    and dev.iceberg100.date_dim.d_dow in (6,0)
    and dev.iceberg100.date_dim.d_year in (1998,1998+1,1998+2)
    and 
    (dev.iceberg100.store.s_city LIKE 'Antioch' OR
        dev.iceberg100.store.s_city LIKE 'Mount Vernon' OR
        dev.iceberg100.store.s_city LIKE 'Jamestown' OR
        dev.iceberg100.store.s_city LIKE 'Wilson' OR
        dev.iceberg100.store.s_city LIKE 'Farmington')
    group by ss_ticket_number,ss_customer_sk,ss_addr_sk,ca_city) dn,dev.iceberg100.customer,dev.iceberg100.customer_address current_addr
    where ss_customer_sk = c_customer_sk
      and dev.iceberg100.customer.c_current_addr_sk = current_addr.ca_address_sk
      and current_addr.ca_city <> bought_city
  order by c_last_name
          ,c_first_name
          ,ca_city
          ,bought_city
          ,ss_ticket_number
  limit 100

In [ ]:
%%sql
--#q47

with v1 as(
 select i_category, i_brand,
        s_store_name, s_company_name,
        d_year, d_moy,
        sum(ss_sales_price) sum_sales,
        avg(sum(ss_sales_price)) over
          (partition by i_category, i_brand,
                     s_store_name, s_company_name, d_year)
          avg_monthly_sales,
        rank() over
          (partition by i_category, i_brand,
                     s_store_name, s_company_name
           order by d_year, d_moy) rn
 from dev.iceberg100.item, dev.iceberg100.store_sales, dev.iceberg100.date_dim, dev.iceberg100.store
 where ss_item_sk = i_item_sk and
       ss_sold_date_sk = d_date_sk and
       ss_store_sk = s_store_sk and
       (
         d_year = 2001 or
         ( d_year = 2001-1 and d_moy =12) or
         ( d_year = 2001+1 and d_moy =1)
       )
 group by i_category, i_brand,
          s_store_name, s_company_name,
          d_year, d_moy),
 v2 as(
 select v1.s_company_name
        ,v1.d_year, v1.d_moy
        ,v1.avg_monthly_sales
        ,v1.sum_sales, v1_lag.sum_sales psum, v1_lead.sum_sales nsum
 from v1, v1 v1_lag, v1 v1_lead
 where v1.i_category = v1_lag.i_category and
       v1.i_category = v1_lead.i_category and
       v1.i_brand = v1_lag.i_brand and
       v1.i_brand = v1_lead.i_brand and
       v1.s_store_name = v1_lag.s_store_name and
       v1.s_store_name = v1_lead.s_store_name and
       v1.s_company_name = v1_lag.s_company_name and
       v1.s_company_name = v1_lead.s_company_name and
       v1.rn = v1_lag.rn + 1 and
       v1.rn = v1_lead.rn - 1)
  select  *
 from v2
 where  d_year = 2001 and
        avg_monthly_sales > 0 and
        case when avg_monthly_sales > 0 then abs(sum_sales - avg_monthly_sales) / avg_monthly_sales else null end > 0.1
 order by sum_sales - avg_monthly_sales, avg_monthly_sales
 limit 100

In [ ]:
%%sql
--#q48

select sum (ss_quantity)
 from dev.iceberg100.store_sales, dev.iceberg100.store, dev.iceberg100.customer_demographics, dev.iceberg100.customer_address, dev.iceberg100.date_dim
 where s_store_sk = ss_store_sk
 and  ss_sold_date_sk = d_date_sk and d_year = 1999
 and
 (
  (
   cd_demo_sk = ss_cdemo_sk
   and
   cd_marital_status = 'D'
   and
   cd_education_status = 'College'
   and
   ss_sales_price between 100.00 and 150.00
   )
 or
  (
  cd_demo_sk = ss_cdemo_sk
   and
   cd_marital_status = 'W'
   and
   cd_education_status = 'Secondary'
   and
   ss_sales_price between 50.00 and 100.00
  )
 or
 (
  cd_demo_sk = ss_cdemo_sk
  and
   cd_marital_status = 'M'
   and
   cd_education_status = '2 yr Degree'
   and
   ss_sales_price between 150.00 and 200.00
 )
 )
 and
 (
  (
  ss_addr_sk = ca_address_sk
  and
  dev.iceberg100.customer_address.ca_country = 'United States'
  and
   (dev.iceberg100.customer_address.ca_state LIKE 'NE'
   OR dev.iceberg100.customer_address.ca_state LIKE 'IA'
   OR dev.iceberg100.customer_address.ca_state LIKE 'NY')
  and dev.iceberg100.store_sales.ss_net_profit between 0 and 2000
  )
 or
  (ss_addr_sk = ca_address_sk
  and
  dev.iceberg100.customer_address.ca_country = 'United States'
  and
  (dev.iceberg100.customer_address.ca_state LIKE 'IN' OR
        dev.iceberg100.customer_address.ca_state LIKE 'TN' OR
        dev.iceberg100.customer_address.ca_state LIKE 'OH')
  and dev.iceberg100.store_sales.ss_net_profit between 150 and 3000
  )
 or
  (ss_addr_sk = ca_address_sk
  and
  dev.iceberg100.customer_address.ca_country = 'United States'
  and
  (dev.iceberg100.customer_address.ca_state LIKE 'KS' OR
        dev.iceberg100.customer_address.ca_state LIKE 'CA' OR
        dev.iceberg100.customer_address.ca_state LIKE 'CO')
  and dev.iceberg100.store_sales.ss_net_profit between 50 and 25000
  )
 )

In [ ]:
%%sql
--#q49

select  channel, item, return_ratio, return_rank, currency_rank from
 (select
 'web' as channel
 ,web.item
 ,web.return_ratio
 ,web.return_rank
 ,web.currency_rank
 from (
 	select
 	 item
 	,return_ratio
 	,currency_ratio
 	,rank() over (order by return_ratio) as return_rank
 	,rank() over (order by currency_ratio) as currency_rank
 	from
 	(	select ws.ws_item_sk as item
 		,(cast(sum(coalesce(wr.wr_return_quantity,0)) as decimal(15,4))/
 		cast(sum(coalesce(ws.ws_quantity,0)) as decimal(15,4) )) as return_ratio
 		,(cast(sum(coalesce(wr.wr_return_amt,0)) as decimal(15,4))/
 		cast(sum(coalesce(ws.ws_net_paid,0)) as decimal(15,4) )) as currency_ratio
 		from
 		 dev.iceberg100.web_sales ws left outer join dev.iceberg100.web_returns wr
 			on (ws.ws_order_number = wr.wr_order_number and
 			ws.ws_item_sk = wr.wr_item_sk)
                 ,dev.iceberg100.date_dim
 		where
 			wr.wr_return_amt > 10000
 			and ws.ws_net_profit > 1
                         and ws.ws_net_paid > 0
                         and ws.ws_quantity > 0
                         and ws_sold_date_sk = d_date_sk
                         and d_year = 2000
                         and d_moy = 11
 		group by ws.ws_item_sk
 	) in_web
 ) web
 where
 (
 web.return_rank <= 10
 or
 web.currency_rank <= 10
 )
 union
 select
 'catalog' as channel
 ,catalog.item
 ,catalog.return_ratio
 ,catalog.return_rank
 ,catalog.currency_rank
 from (
 	select
 	 item
 	,return_ratio
 	,currency_ratio
 	,rank() over (order by return_ratio) as return_rank
 	,rank() over (order by currency_ratio) as currency_rank
 	from
 	(	select
 		cs.cs_item_sk as item
 		,(cast(sum(coalesce(cr.cr_return_quantity,0)) as decimal(15,4))/
 		cast(sum(coalesce(cs.cs_quantity,0)) as decimal(15,4) )) as return_ratio
 		,(cast(sum(coalesce(cr.cr_return_amount,0)) as decimal(15,4))/
 		cast(sum(coalesce(cs.cs_net_paid,0)) as decimal(15,4) )) as currency_ratio
 		from
 		dev.iceberg100.catalog_sales cs left outer join dev.iceberg100.catalog_returns cr
 			on (cs.cs_order_number = cr.cr_order_number and
 			cs.cs_item_sk = cr.cr_item_sk)
                ,dev.iceberg100.date_dim
 		where
 			cr.cr_return_amount > 10000
 			and cs.cs_net_profit > 1
                         and cs.cs_net_paid > 0
                         and cs.cs_quantity > 0
                         and cs_sold_date_sk = d_date_sk
                         and d_year = 2000
                         and d_moy = 11
                 group by cs.cs_item_sk
 	) in_cat
 ) catalog
 where
 (
 catalog.return_rank <= 10
 or
 catalog.currency_rank <=10
 )
 union
 select
 'store' as channel
 ,store.item
 ,store.return_ratio
 ,store.return_rank
 ,store.currency_rank
 from (
 	select
 	 item
 	,return_ratio
 	,currency_ratio
 	,rank() over (order by return_ratio) as return_rank
 	,rank() over (order by currency_ratio) as currency_rank
 	from
 	(	select sts.ss_item_sk as item
 		,(cast(sum(coalesce(sr.sr_return_quantity,0)) as decimal(15,4))/cast(sum(coalesce(sts.ss_quantity,0)) as decimal(15,4) )) as return_ratio
 		,(cast(sum(coalesce(sr.sr_return_amt,0)) as decimal(15,4))/cast(sum(coalesce(sts.ss_net_paid,0)) as decimal(15,4) )) as currency_ratio
 		from
 		dev.iceberg100.store_sales sts left outer join dev.iceberg100.store_returns sr
 			on (sts.ss_ticket_number = sr.sr_ticket_number and sts.ss_item_sk = sr.sr_item_sk)
                ,dev.iceberg100.date_dim
 		where
 			sr.sr_return_amt > 10000
 			and sts.ss_net_profit > 1
                         and sts.ss_net_paid > 0
                         and sts.ss_quantity > 0
                         and ss_sold_date_sk = d_date_sk
                         and d_year = 2000
                         and d_moy = 11
 		group by sts.ss_item_sk
 	) in_store
 ) store
 where  (
 store.return_rank <= 10
 or
 store.currency_rank <= 10
 )
 )
 order by 1,4,5,2
 limit 100

In [ ]:
%%sql
--#q50

select
   s_store_name
  ,s_company_id
  ,s_street_number
  ,s_street_name
  ,s_street_type
  ,s_suite_number
  ,s_city
  ,s_county
  ,s_state
  ,s_zip
  ,sum(case when (sr_returned_date_sk - ss_sold_date_sk <= 30 ) then 1 else 0 end)  as `30 days`
  ,sum(case when (sr_returned_date_sk - ss_sold_date_sk > 30) and
                 (sr_returned_date_sk - ss_sold_date_sk <= 60) then 1 else 0 end )  as `31-60 days`
  ,sum(case when (sr_returned_date_sk - ss_sold_date_sk > 60) and
                 (sr_returned_date_sk - ss_sold_date_sk <= 90) then 1 else 0 end)  as `61-90 days`
  ,sum(case when (sr_returned_date_sk - ss_sold_date_sk > 90) and
                 (sr_returned_date_sk - ss_sold_date_sk <= 120) then 1 else 0 end)  as `91-120 days`
  ,sum(case when (sr_returned_date_sk - ss_sold_date_sk  > 120) then 1 else 0 end)  as `>120 days`
from
   dev.iceberg100.store_sales
  ,dev.iceberg100.store_returns
  ,dev.iceberg100.store
  ,dev.iceberg100.date_dim d1
  ,dev.iceberg100.date_dim d2
where
    d2.d_year = 1999
and d2.d_moy  = 9
and ss_ticket_number = sr_ticket_number
and ss_item_sk = sr_item_sk
and ss_sold_date_sk   = d1.d_date_sk
and sr_returned_date_sk   = d2.d_date_sk
and ss_customer_sk = sr_customer_sk
and ss_store_sk = s_store_sk
group by
   s_store_name
  ,s_company_id
  ,s_street_number
  ,s_street_name
  ,s_street_type
  ,s_suite_number
  ,s_city
  ,s_county
  ,s_state
  ,s_zip
order by s_store_name
        ,s_company_id
        ,s_street_number
        ,s_street_name
        ,s_street_type
        ,s_suite_number
        ,s_city
        ,s_county
        ,s_state
        ,s_zip
limit 100

In [ ]:
%%sql
--#q51

WITH web_v1 as (
select
  ws_item_sk item_sk, d_date,
  sum(sum(ws_sales_price))
      over (partition by ws_item_sk order by d_date rows between unbounded preceding and current row) cume_sales
from dev.iceberg100.web_sales
    ,dev.iceberg100.date_dim
where ws_sold_date_sk=d_date_sk
  and d_month_seq between 1176 and 1176+11
  and ws_item_sk is not NULL
group by ws_item_sk, d_date),
store_v1 as (
select
  ss_item_sk item_sk, d_date,
  sum(sum(ss_sales_price))
      over (partition by ss_item_sk order by d_date rows between unbounded preceding and current row) cume_sales
from dev.iceberg100.store_sales
    ,dev.iceberg100.date_dim
where ss_sold_date_sk=d_date_sk
  and d_month_seq between 1176 and 1176+11
  and ss_item_sk is not NULL
group by ss_item_sk, d_date)
 select  *
from (select item_sk
     ,d_date
     ,dev.iceberg100.web_sales
     ,dev.iceberg100.store_sales
     ,max(dev.iceberg100.web_sales)
         over (partition by item_sk order by d_date rows between unbounded preceding and current row) web_cumulative
     ,max(store_sales)
         over (partition by item_sk order by d_date rows between unbounded preceding and current row) store_cumulative
     from (select case when web.item_sk is not null then web.item_sk else store.item_sk end item_sk
                 ,case when web.d_date is not null then web.d_date else store.d_date end d_date
                 ,web.cume_sales dev.iceberg100.web_sales
                 ,store.cume_sales store_sales
           from web_v1 web full outer join store_v1 store on (web.item_sk = store.item_sk
                                                          and web.d_date = store.d_date)
          )x )y
where web_cumulative > store_cumulative
order by item_sk
        ,d_date
limit 100

In [ ]:
%%sql
--#q52

select  dt.d_year
 	,dev.iceberg100.item.i_brand_id brand_id
 	,item.i_brand brand
 	,sum(ss_ext_sales_price) ext_price
 from dev.iceberg100.date_dim dt
     ,dev.iceberg100.store_sales
     ,dev.iceberg100.item
 where dt.d_date_sk = store_sales.ss_sold_date_sk
    and store_sales.ss_item_sk = dev.iceberg100.item.i_item_sk
    and dev.iceberg100.item.i_manager_id = 1
    and dt.d_moy=11
    and dt.d_year=2001
 group by dt.d_year
 	,dev.iceberg100.item.i_brand
 	,dev.iceberg100.item.i_brand_id
 order by dt.d_year
 	,ext_price desc
 	,brand_id
limit 100

In [ ]:
%%sql
--#q53

select  * from
(select i_manufact_id,
sum(ss_sales_price) sum_sales,
avg(sum(ss_sales_price)) over (partition by i_manufact_id) avg_quarterly_sales
from dev.iceberg100.item, dev.iceberg100.store_sales, dev.iceberg100.date_dim, dev.iceberg100.store
where ss_item_sk = i_item_sk and
ss_sold_date_sk = d_date_sk and
ss_store_sk = s_store_sk and
d_month_seq in (1218,1218+1,1218+2,1218+3,1218+4,1218+5,1218+6,1218+7,1218+8,1218+9,1218+10,1218+11) and
((i_category in ('Books','Children','Electronics') and
i_class in ('personal','portable','reference','self-help') and
i_brand in ('scholaramalgamalg #14','scholaramalgamalg #7',
		'exportiunivamalg #9','scholaramalgamalg #9'))
or(i_category in ('Women','Music','Men') and
i_class in ('accessories','classical','fragrances','pants') and
i_brand in ('amalgimporto #1','edu packscholar #1','exportiimporto #1',
		'importoamalg #1')))
group by i_manufact_id, d_qoy ) tmp1
where case when avg_quarterly_sales > 0
	then abs (sum_sales - avg_quarterly_sales)/ avg_quarterly_sales
	else null end > 0.1
order by avg_quarterly_sales,
	 sum_sales,
	 i_manufact_id
limit 100

In [ ]:
%%sql
--#q54

with my_customers as (
 select distinct c_customer_sk
        , c_current_addr_sk
 from
        ( select cs_sold_date_sk sold_date_sk,
                 cs_bill_customer_sk customer_sk,
                 cs_item_sk item_sk
          from   dev.iceberg100.catalog_sales
          union all
          select ws_sold_date_sk sold_date_sk,
                 ws_bill_customer_sk customer_sk,
                 ws_item_sk item_sk
          from   dev.iceberg100.web_sales
         ) cs_or_ws_sales,
         dev.iceberg100.item,
         dev.iceberg100.date_dim,
         dev.iceberg100.customer
 where   sold_date_sk = d_date_sk
         and item_sk = i_item_sk
         and i_category = 'Music'
         and i_class = 'country'
         and c_customer_sk = cs_or_ws_sales.customer_sk
         and d_moy = 7
         and d_year = 2001
 )
 , my_revenue as (
 select c_customer_sk,
        sum(ss_ext_sales_price) as revenue
 from   my_customers,
        dev.iceberg100.store_sales,
        dev.iceberg100.customer_address,
        dev.iceberg100.store,
        dev.iceberg100.date_dim
 where  c_current_addr_sk = ca_address_sk
        and ca_county = s_county
        and ca_state = s_state
        and ss_sold_date_sk = d_date_sk
        and c_customer_sk = ss_customer_sk
        and d_month_seq between (select distinct d_month_seq+1
                                 from   dev.iceberg100.date_dim where d_year = 2001 and d_moy = 7)
                           and  (select distinct d_month_seq+3
                                 from   dev.iceberg100.date_dim where d_year = 2001 and d_moy = 7)
 group by c_customer_sk
 )
 , segments as
 (select cast((revenue/50) as int) as segment
  from   my_revenue
 )
  select  segment, count(*) as num_customers, segment*50 as segment_base
 from segments
 group by segment
 order by segment, num_customers
 limit 100

In [ ]:
%%sql
--#q55

select  i_brand_id brand_id, i_brand brand,
 	sum(ss_ext_sales_price) ext_price
 from dev.iceberg100.date_dim, dev.iceberg100.store_sales, dev.iceberg100.item
 where d_date_sk = ss_sold_date_sk
 	and ss_item_sk = i_item_sk
 	and i_manager_id=87
 	and d_moy=11
 	and d_year=2001
 group by i_brand, i_brand_id
 order by ext_price desc, i_brand_id
limit 100

In [ ]:
%%sql
--#q56

with ss as (
 select i_item_id,sum(ss_ext_sales_price) total_sales
 from
 	dev.iceberg100.store_sales,
 	dev.iceberg100.date_dim,
         dev.iceberg100.customer_address,
         dev.iceberg100.item
 where i_item_id in (select
     i_item_id
from dev.iceberg100.item
where i_color in ('tan','lace','gainsboro'))
 and     ss_item_sk              = i_item_sk
 and     ss_sold_date_sk         = d_date_sk
 and     d_year                  = 1998
 and     d_moy                   = 3
 and     ss_addr_sk              = ca_address_sk
 and     ca_gmt_offset           = -5
 group by i_item_id),
 cs as (
 select i_item_id,sum(cs_ext_sales_price) total_sales
 from
 	dev.iceberg100.catalog_sales,
 	dev.iceberg100.date_dim,
         dev.iceberg100.customer_address,
         dev.iceberg100.item
 where
         i_item_id               in (select
  i_item_id
from dev.iceberg100.item
where i_color in ('tan','lace','gainsboro'))
 and     cs_item_sk              = i_item_sk
 and     cs_sold_date_sk         = d_date_sk
 and     d_year                  = 1998
 and     d_moy                   = 3
 and     cs_bill_addr_sk         = ca_address_sk
 and     ca_gmt_offset           = -5
 group by i_item_id),
 ws as (
 select i_item_id,sum(ws_ext_sales_price) total_sales
 from
 	dev.iceberg100.web_sales,
 	dev.iceberg100.date_dim,
         dev.iceberg100.customer_address,
         dev.iceberg100.item
 where
         i_item_id               in (select
  i_item_id
from dev.iceberg100.item
where i_color in ('tan','lace','gainsboro'))
 and     ws_item_sk              = i_item_sk
 and     ws_sold_date_sk         = d_date_sk
 and     d_year                  = 1998
 and     d_moy                   = 3
 and     ws_bill_addr_sk         = ca_address_sk
 and     ca_gmt_offset           = -5
 group by i_item_id)
  select  i_item_id ,sum(total_sales) total_sales
 from  (select * from ss
        union all
        select * from cs
        union all
        select * from ws) tmp1
 group by i_item_id
 order by total_sales,
          i_item_id
 limit 100

In [ ]:
%%sql
--#q57

with v1 as(
 select i_category, i_brand,
        cc_name,
        d_year, d_moy,
        sum(cs_sales_price) sum_sales,
        avg(sum(cs_sales_price)) over
          (partition by i_category, i_brand,
                     cc_name, d_year)
          avg_monthly_sales,
        rank() over
          (partition by i_category, i_brand,
                     cc_name
           order by d_year, d_moy) rn
 from dev.iceberg100.item, dev.iceberg100.catalog_sales, dev.iceberg100.date_dim, dev.iceberg100.call_center
 where cs_item_sk = i_item_sk and
       cs_sold_date_sk = d_date_sk and
       cc_call_center_sk= cs_call_center_sk and
       (
         d_year = 2001 or
         ( d_year = 2001-1 and d_moy =12) or
         ( d_year = 2001+1 and d_moy =1)
       )
 group by i_category, i_brand,
          cc_name , d_year, d_moy),
 v2 as(
 select v1.i_category, v1.i_brand, v1.cc_name
        ,v1.d_year, v1.d_moy
        ,v1.avg_monthly_sales
        ,v1.sum_sales, v1_lag.sum_sales psum, v1_lead.sum_sales nsum
 from v1, v1 v1_lag, v1 v1_lead
 where v1.i_category = v1_lag.i_category and
       v1.i_category = v1_lead.i_category and
       v1.i_brand = v1_lag.i_brand and
       v1.i_brand = v1_lead.i_brand and
       v1. cc_name = v1_lag. cc_name and
       v1. cc_name = v1_lead. cc_name and
       v1.rn = v1_lag.rn + 1 and
       v1.rn = v1_lead.rn - 1)
  select  *
 from v2
 where  d_year = 2001 and
        avg_monthly_sales > 0 and
        case when avg_monthly_sales > 0 then abs(sum_sales - avg_monthly_sales) / avg_monthly_sales else null end > 0.1
 order by sum_sales - avg_monthly_sales, avg_monthly_sales
 limit 100

In [ ]:
%%sql
--#q58

with ss_items as
 (select i_item_id item_id
        ,sum(ss_ext_sales_price) ss_item_rev
 from dev.iceberg100.store_sales
     ,dev.iceberg100.item
     ,dev.iceberg100.date_dim
 where ss_item_sk = i_item_sk
   and d_date in (select d_date
                  from dev.iceberg100.date_dim
                  where d_week_seq = (select d_week_seq
                                      from dev.iceberg100.date_dim
                                      where d_date = '2000-03-26'))
   and ss_sold_date_sk   = d_date_sk
 group by i_item_id),
 cs_items as
 (select i_item_id item_id
        ,sum(cs_ext_sales_price) cs_item_rev
  from dev.iceberg100.catalog_sales
      ,dev.iceberg100.item
      ,dev.iceberg100.date_dim
 where cs_item_sk = i_item_sk
  and  d_date in (select d_date
                  from dev.iceberg100.date_dim
                  where d_week_seq = (select d_week_seq
                                      from dev.iceberg100.date_dim
                                      where d_date = '2000-03-26'))
  and  cs_sold_date_sk = d_date_sk
 group by i_item_id),
 ws_items as
 (select i_item_id item_id
        ,sum(ws_ext_sales_price) ws_item_rev
  from dev.iceberg100.web_sales
      ,dev.iceberg100.item
      ,dev.iceberg100.date_dim
 where ws_item_sk = i_item_sk
  and  d_date in (select d_date
                  from dev.iceberg100.date_dim
                  where d_week_seq =(select d_week_seq
                                     from dev.iceberg100.date_dim
                                     where d_date = '2000-03-26'))
  and ws_sold_date_sk   = d_date_sk
 group by i_item_id)
  select  ss_items.item_id
       ,ss_item_rev
       ,ss_item_rev/((ss_item_rev+cs_item_rev+ws_item_rev)/3) * 100 ss_dev
       ,cs_item_rev
       ,cs_item_rev/((ss_item_rev+cs_item_rev+ws_item_rev)/3) * 100 cs_dev
       ,ws_item_rev
       ,ws_item_rev/((ss_item_rev+cs_item_rev+ws_item_rev)/3) * 100 ws_dev
       ,(ss_item_rev+cs_item_rev+ws_item_rev)/3 average
 from ss_items,cs_items,ws_items
 where ss_items.item_id=cs_items.item_id
   and ss_items.item_id=ws_items.item_id
   and ss_item_rev between 0.9 * cs_item_rev and 1.1 * cs_item_rev
   and ss_item_rev between 0.9 * ws_item_rev and 1.1 * ws_item_rev
   and cs_item_rev between 0.9 * ss_item_rev and 1.1 * ss_item_rev
   and cs_item_rev between 0.9 * ws_item_rev and 1.1 * ws_item_rev
   and ws_item_rev between 0.9 * ss_item_rev and 1.1 * ss_item_rev
   and ws_item_rev between 0.9 * cs_item_rev and 1.1 * cs_item_rev
 order by item_id
         ,ss_item_rev
 limit 100

In [ ]:
%%sql
--#q59

with wss as
 (select d_week_seq,
        ss_store_sk,
        sum(case when (d_day_name='Sunday') then ss_sales_price else null end) sun_sales,
        sum(case when (d_day_name='Monday') then ss_sales_price else null end) mon_sales,
        sum(case when (d_day_name='Tuesday') then ss_sales_price else  null end) tue_sales,
        sum(case when (d_day_name='Wednesday') then ss_sales_price else null end) wed_sales,
        sum(case when (d_day_name='Thursday') then ss_sales_price else null end) thu_sales,
        sum(case when (d_day_name='Friday') then ss_sales_price else null end) fri_sales,
        sum(case when (d_day_name='Saturday') then ss_sales_price else null end) sat_sales
 from dev.iceberg100.store_sales,dev.iceberg100.date_dim
 where d_date_sk = ss_sold_date_sk
 group by d_week_seq,ss_store_sk
 )
  select  s_store_name1,s_store_id1,d_week_seq1
       ,sun_sales1/sun_sales2,mon_sales1/mon_sales2
       ,tue_sales1/tue_sales2,wed_sales1/wed_sales2,thu_sales1/thu_sales2
       ,fri_sales1/fri_sales2,sat_sales1/sat_sales2
 from
 (select s_store_name s_store_name1,wss.d_week_seq d_week_seq1
        ,s_store_id s_store_id1,sun_sales sun_sales1
        ,mon_sales mon_sales1,tue_sales tue_sales1
        ,wed_sales wed_sales1,thu_sales thu_sales1
        ,fri_sales fri_sales1,sat_sales sat_sales1
  from wss,dev.iceberg100.store,dev.iceberg100.date_dim d
  where d.d_week_seq = wss.d_week_seq and
        ss_store_sk = s_store_sk and
        d_month_seq between 1199 and 1199 + 11) y,
 (select s_store_name s_store_name2,wss.d_week_seq d_week_seq2
        ,s_store_id s_store_id2,sun_sales sun_sales2
        ,mon_sales mon_sales2,tue_sales tue_sales2
        ,wed_sales wed_sales2,thu_sales thu_sales2
        ,fri_sales fri_sales2,sat_sales sat_sales2
  from wss,dev.iceberg100.store,dev.iceberg100.date_dim d
  where d.d_week_seq = wss.d_week_seq and
        ss_store_sk = s_store_sk and
        d_month_seq between 1199+ 12 and 1199 + 23) x
 where s_store_id1=s_store_id2
   and d_week_seq1=d_week_seq2-52
 order by s_store_name1,s_store_id1,d_week_seq1
limit 100

In [ ]:
%%sql
--#q60

with ss as (
 select
          i_item_id,sum(ss_ext_sales_price) total_sales
 from
 	dev.iceberg100.store_sales,
 	dev.iceberg100.date_dim,
         dev.iceberg100.customer_address,
         dev.iceberg100.item
 where
         i_item_id in (select
  i_item_id
from
 dev.iceberg100.item
where i_category in ('Men'))
 and     ss_item_sk              = i_item_sk
 and     ss_sold_date_sk         = d_date_sk
 and     d_year                  = 2000
 and     d_moy                   = 9
 and     ss_addr_sk              = ca_address_sk
 and     ca_gmt_offset           = -5
 group by i_item_id),
 cs as (
 select
          i_item_id,sum(cs_ext_sales_price) total_sales
 from
 	dev.iceberg100.catalog_sales,
 	dev.iceberg100.date_dim,
         dev.iceberg100.customer_address,
         dev.iceberg100.item
 where
         i_item_id               in (select
  i_item_id
from
 dev.iceberg100.item
where i_category in ('Men'))
 and     cs_item_sk              = i_item_sk
 and     cs_sold_date_sk         = d_date_sk
 and     d_year                  = 2000
 and     d_moy                   = 9
 and     cs_bill_addr_sk         = ca_address_sk
 and     ca_gmt_offset           = -5
 group by i_item_id),
 ws as (
 select
          i_item_id,sum(ws_ext_sales_price) total_sales
 from
 	dev.iceberg100.web_sales,
 	dev.iceberg100.date_dim,
         dev.iceberg100.customer_address,
         dev.iceberg100.item
 where
         i_item_id               in (select
  i_item_id
from
 dev.iceberg100.item
where i_category in ('Men'))
 and     ws_item_sk              = i_item_sk
 and     ws_sold_date_sk         = d_date_sk
 and     d_year                  = 2000
 and     d_moy                   = 9
 and     ws_bill_addr_sk         = ca_address_sk
 and     ca_gmt_offset           = -5
 group by i_item_id)
  select
  i_item_id
,sum(total_sales) total_sales
 from  (select * from ss
        union all
        select * from cs
        union all
        select * from ws) tmp1
 group by i_item_id
 order by i_item_id
      ,total_sales
 limit 100

In [ ]:
%%sql
--#q61

select  promotions,total,cast(promotions as decimal(15,4))/cast(total as decimal(15,4))*100
from
  (select sum(ss_ext_sales_price) promotions
   from  dev.iceberg100.store_sales
        ,dev.iceberg100.store
        ,dev.iceberg100.promotion
        ,dev.iceberg100.date_dim
        ,dev.iceberg100.customer
        ,dev.iceberg100.customer_address
        ,dev.iceberg100.item
   where ss_sold_date_sk = d_date_sk
   and   ss_store_sk = s_store_sk
   and   ss_promo_sk = p_promo_sk
   and   ss_customer_sk= c_customer_sk
   and   ca_address_sk = c_current_addr_sk
   and   ss_item_sk = i_item_sk
   and   ca_gmt_offset = -7
   and   i_category = 'Electronics'
   and   (p_channel_dmail = 'Y' or p_channel_email = 'Y' or p_channel_tv = 'Y')
   and   s_gmt_offset = -7
   and   d_year = 2001
   and   d_moy  = 11) promotional_sales,
  (select sum(ss_ext_sales_price) total
   from  dev.iceberg100.store_sales
        ,dev.iceberg100.store
        ,dev.iceberg100.date_dim
        ,dev.iceberg100.customer
        ,dev.iceberg100.customer_address
        ,dev.iceberg100.item
   where ss_sold_date_sk = d_date_sk
   and   ss_store_sk = s_store_sk
   and   ss_customer_sk= c_customer_sk
   and   ca_address_sk = c_current_addr_sk
   and   ss_item_sk = i_item_sk
   and   ca_gmt_offset = -7
   and   i_category = 'Electronics'
   and   s_gmt_offset = -7
   and   d_year = 2001
   and   d_moy  = 11) all_sales
order by promotions, total
limit 100

In [ ]:
%%sql
--#q62

select
   substr(w_warehouse_name,1,20)
  ,sm_type
  ,web_name
  ,sum(case when (ws_ship_date_sk - ws_sold_date_sk <= 30 ) then 1 else 0 end)  as `30 days`
  ,sum(case when (ws_ship_date_sk - ws_sold_date_sk > 30) and
                 (ws_ship_date_sk - ws_sold_date_sk <= 60) then 1 else 0 end )  as `31-60 days`
  ,sum(case when (ws_ship_date_sk - ws_sold_date_sk > 60) and
                 (ws_ship_date_sk - ws_sold_date_sk <= 90) then 1 else 0 end)  as `61-90 days`
  ,sum(case when (ws_ship_date_sk - ws_sold_date_sk > 90) and
                 (ws_ship_date_sk - ws_sold_date_sk <= 120) then 1 else 0 end)  as `91-120 days`
  ,sum(case when (ws_ship_date_sk - ws_sold_date_sk  > 120) then 1 else 0 end)  as `>120 days`
from
   dev.iceberg100.web_sales
  ,dev.iceberg100.warehouse
  ,dev.iceberg100.ship_mode
  ,dev.iceberg100.web_site
  ,dev.iceberg100.date_dim
where
    d_month_seq between 1194 and 1194 + 11
and ws_ship_date_sk   = d_date_sk
and ws_warehouse_sk   = w_warehouse_sk
and ws_ship_mode_sk   = sm_ship_mode_sk
and ws_web_site_sk    = web_site_sk
group by
   substr(w_warehouse_name,1,20)
  ,sm_type
  ,web_name
order by substr(w_warehouse_name,1,20)
        ,sm_type
       ,web_name
limit 100

In [ ]:
%%sql
--#q63

select  *
from (select i_manager_id
             ,sum(ss_sales_price) sum_sales
             ,avg(sum(ss_sales_price)) over (partition by i_manager_id) avg_monthly_sales
      from dev.iceberg100.item
          ,dev.iceberg100.store_sales
          ,dev.iceberg100.date_dim
          ,dev.iceberg100.store
      where ss_item_sk = i_item_sk
        and ss_sold_date_sk = d_date_sk
        and ss_store_sk = s_store_sk
        and d_month_seq in (1205,1205+1,1205+2,1205+3,1205+4,1205+5,1205+6,1205+7,1205+8,1205+9,1205+10,1205+11)
        and ((    i_category in ('Books','Children','Electronics')
              and i_class in ('personal','portable','reference','self-help')
              and i_brand in ('scholaramalgamalg #14','scholaramalgamalg #7',
		                  'exportiunivamalg #9','scholaramalgamalg #9'))
           or(    i_category in ('Women','Music','Men')
              and i_class in ('accessories','classical','fragrances','pants')
              and i_brand in ('amalgimporto #1','edu packscholar #1','exportiimporto #1',
		                 'importoamalg #1')))
group by i_manager_id, d_moy) tmp1
where case when avg_monthly_sales > 0 then abs (sum_sales - avg_monthly_sales) / avg_monthly_sales else null end > 0.1
order by i_manager_id
        ,avg_monthly_sales
        ,sum_sales
limit 100

In [ ]:
%%sql
--#q64

with cs_ui as
 (select cs_item_sk
        ,sum(cs_ext_list_price) as sale,sum(cr_refunded_cash+cr_reversed_charge+cr_store_credit) as refund
  from dev.iceberg100.catalog_sales
      ,dev.iceberg100.catalog_returns
  where cs_item_sk = cr_item_sk
    and cs_order_number = cr_order_number
  group by cs_item_sk
  having sum(cs_ext_list_price)>2*sum(cr_refunded_cash+cr_reversed_charge+cr_store_credit)),
cross_sales as
 (select i_product_name product_name
     ,i_item_sk item_sk
     ,s_store_name store_name
     ,s_zip store_zip
     ,ad1.ca_street_number b_street_number
     ,ad1.ca_street_name b_street_name
     ,ad1.ca_city b_city
     ,ad1.ca_zip b_zip
     ,ad2.ca_street_number c_street_number
     ,ad2.ca_street_name c_street_name
     ,ad2.ca_city c_city
     ,ad2.ca_zip c_zip
     ,d1.d_year as syear
     ,d2.d_year as fsyear
     ,d3.d_year s2year
     ,count(*) cnt
     ,sum(ss_wholesale_cost) s1
     ,sum(ss_list_price) s2
     ,sum(ss_coupon_amt) s3
  FROM   dev.iceberg100.store_sales
        ,dev.iceberg100.store_returns
        ,cs_ui
        ,dev.iceberg100.date_dim d1
        ,dev.iceberg100.date_dim d2
        ,dev.iceberg100.date_dim d3
        ,dev.iceberg100.store
        ,dev.iceberg100.customer
        ,dev.iceberg100.customer_demographics cd1
        ,dev.iceberg100.customer_demographics cd2
        ,dev.iceberg100.promotion
        ,dev.iceberg100.household_demographics hd1
        ,dev.iceberg100.household_demographics hd2
        ,dev.iceberg100.customer_address ad1
        ,dev.iceberg100.customer_address ad2
        ,dev.iceberg100.income_band ib1
        ,dev.iceberg100.income_band ib2
        ,dev.iceberg100.item
  WHERE  ss_store_sk = s_store_sk AND
         ss_sold_date_sk = d1.d_date_sk AND
         ss_customer_sk = c_customer_sk AND
         ss_cdemo_sk= cd1.cd_demo_sk AND
         ss_hdemo_sk = hd1.hd_demo_sk AND
         ss_addr_sk = ad1.ca_address_sk and
         ss_item_sk = i_item_sk and
         ss_item_sk = sr_item_sk and
         ss_ticket_number = sr_ticket_number and
         ss_item_sk = cs_ui.cs_item_sk and
         c_current_cdemo_sk = cd2.cd_demo_sk AND
         c_current_hdemo_sk = hd2.hd_demo_sk AND
         c_current_addr_sk = ad2.ca_address_sk and
         c_first_sales_date_sk = d2.d_date_sk and
         c_first_shipto_date_sk = d3.d_date_sk and
         ss_promo_sk = p_promo_sk and
         hd1.hd_income_band_sk = ib1.ib_income_band_sk and
         hd2.hd_income_band_sk = ib2.ib_income_band_sk and
         cd1.cd_marital_status <> cd2.cd_marital_status and
         i_color in ('peach','misty','drab','chocolate','almond','saddle') and
         i_current_price between 75 and 75 + 10 and
         i_current_price between 75 + 1 and 75 + 15
group by i_product_name
       ,i_item_sk
       ,s_store_name
       ,s_zip
       ,ad1.ca_street_number
       ,ad1.ca_street_name
       ,ad1.ca_city
       ,ad1.ca_zip
       ,ad2.ca_street_number
       ,ad2.ca_street_name
       ,ad2.ca_city
       ,ad2.ca_zip
       ,d1.d_year
       ,d2.d_year
       ,d3.d_year
)
select cs1.product_name
     ,cs1.store_name
     ,cs1.store_zip
     ,cs1.b_street_number
     ,cs1.b_street_name
     ,cs1.b_city
     ,cs1.b_zip
     ,cs1.c_street_number
     ,cs1.c_street_name
     ,cs1.c_city
     ,cs1.c_zip
     ,cs1.syear
     ,cs1.cnt
     ,cs1.s1 as s11
     ,cs1.s2 as s21
     ,cs1.s3 as s31
     ,cs2.s1 as s12
     ,cs2.s2 as s22
     ,cs2.s3 as s32
     ,cs2.syear
     ,cs2.cnt
from cross_sales cs1,cross_sales cs2
where cs1.item_sk=cs2.item_sk and
     cs1.syear = 2000 and
     cs2.syear = 2000 + 1 and
     cs2.cnt <= cs1.cnt and
     cs1.store_name = cs2.store_name and
     cs1.store_zip = cs2.store_zip
order by cs1.product_name
       ,cs1.store_name
       ,cs2.cnt
       ,cs1.s1
       ,cs2.s1

In [ ]:
%%sql
--#q65

select
	s_store_name,
	i_item_desc,
	sc.revenue,
	i_current_price,
	i_wholesale_cost,
	i_brand
 from dev.iceberg100.store, dev.iceberg100.item,
     (select ss_store_sk, avg(revenue) as ave
 	from
 	    (select  ss_store_sk, ss_item_sk,
 		     sum(ss_sales_price) as revenue
 		from dev.iceberg100.store_sales, dev.iceberg100.date_dim
 		where ss_sold_date_sk = d_date_sk and d_month_seq between 1208 and 1208+11
 		group by ss_store_sk, ss_item_sk) sa
 	group by ss_store_sk) sb,
     (select  ss_store_sk, ss_item_sk, sum(ss_sales_price) as revenue
 	from dev.iceberg100.store_sales, dev.iceberg100.date_dim
 	where ss_sold_date_sk = d_date_sk and d_month_seq between 1208 and 1208+11
 	group by ss_store_sk, ss_item_sk) sc
 where sb.ss_store_sk = sc.ss_store_sk and
       sc.revenue <= 0.1 * sb.ave and
       s_store_sk = sc.ss_store_sk and
       i_item_sk = sc.ss_item_sk
 order by s_store_name, i_item_desc
limit 100

In [73]:
%%sql
--#q66

select
         w_warehouse_name
 	,w_warehouse_sq_ft
 	,w_city
 	,w_county
 	,w_state
 	,w_country
        ,ship_carriers
        ,year
 	,sum(jan_sales) as jan_sales
 	,sum(feb_sales) as feb_sales
 	,sum(mar_sales) as mar_sales
 	,sum(apr_sales) as apr_sales
 	,sum(may_sales) as may_sales
 	,sum(jun_sales) as jun_sales
 	,sum(jul_sales) as jul_sales
 	,sum(aug_sales) as aug_sales
 	,sum(sep_sales) as sep_sales
 	,sum(oct_sales) as oct_sales
 	,sum(nov_sales) as nov_sales
 	,sum(dec_sales) as dec_sales
 	,sum(jan_sales/w_warehouse_sq_ft) as jan_sales_per_sq_foot
 	,sum(feb_sales/w_warehouse_sq_ft) as feb_sales_per_sq_foot
 	,sum(mar_sales/w_warehouse_sq_ft) as mar_sales_per_sq_foot
 	,sum(apr_sales/w_warehouse_sq_ft) as apr_sales_per_sq_foot
 	,sum(may_sales/w_warehouse_sq_ft) as may_sales_per_sq_foot
 	,sum(jun_sales/w_warehouse_sq_ft) as jun_sales_per_sq_foot
 	,sum(jul_sales/w_warehouse_sq_ft) as jul_sales_per_sq_foot
 	,sum(aug_sales/w_warehouse_sq_ft) as aug_sales_per_sq_foot
 	,sum(sep_sales/w_warehouse_sq_ft) as sep_sales_per_sq_foot
 	,sum(oct_sales/w_warehouse_sq_ft) as oct_sales_per_sq_foot
 	,sum(nov_sales/w_warehouse_sq_ft) as nov_sales_per_sq_foot
 	,sum(dec_sales/w_warehouse_sq_ft) as dec_sales_per_sq_foot
 	,sum(jan_net) as jan_net
 	,sum(feb_net) as feb_net
 	,sum(mar_net) as mar_net
 	,sum(apr_net) as apr_net
 	,sum(may_net) as may_net
 	,sum(jun_net) as jun_net
 	,sum(jul_net) as jul_net
 	,sum(aug_net) as aug_net
 	,sum(sep_net) as sep_net
 	,sum(oct_net) as oct_net
 	,sum(nov_net) as nov_net
 	,sum(dec_net) as dec_net
 from (
     select
 	w_warehouse_name
 	,w_warehouse_sq_ft
 	,w_city
 	,w_county
 	,w_state
 	,w_country
 	,'HARMSTORF' || ',' || 'USPS' as ship_carriers
       ,d_year as year
 	,sum(case when d_moy = 1
 		then ws_sales_price* ws_quantity else 0 end) as jan_sales
 	,sum(case when d_moy = 2
 		then ws_sales_price* ws_quantity else 0 end) as feb_sales
 	,sum(case when d_moy = 3
 		then ws_sales_price* ws_quantity else 0 end) as mar_sales
 	,sum(case when d_moy = 4
 		then ws_sales_price* ws_quantity else 0 end) as apr_sales
 	,sum(case when d_moy = 5
 		then ws_sales_price* ws_quantity else 0 end) as may_sales
 	,sum(case when d_moy = 6
 		then ws_sales_price* ws_quantity else 0 end) as jun_sales
 	,sum(case when d_moy = 7
 		then ws_sales_price* ws_quantity else 0 end) as jul_sales
 	,sum(case when d_moy = 8
 		then ws_sales_price* ws_quantity else 0 end) as aug_sales
 	,sum(case when d_moy = 9
 		then ws_sales_price* ws_quantity else 0 end) as sep_sales
 	,sum(case when d_moy = 10
 		then ws_sales_price* ws_quantity else 0 end) as oct_sales
 	,sum(case when d_moy = 11
 		then ws_sales_price* ws_quantity else 0 end) as nov_sales
 	,sum(case when d_moy = 12
 		then ws_sales_price* ws_quantity else 0 end) as dec_sales
 	,sum(case when d_moy = 1
 		then ws_net_paid_inc_tax * ws_quantity else 0 end) as jan_net
 	,sum(case when d_moy = 2
 		then ws_net_paid_inc_tax * ws_quantity else 0 end) as feb_net
 	,sum(case when d_moy = 3
 		then ws_net_paid_inc_tax * ws_quantity else 0 end) as mar_net
 	,sum(case when d_moy = 4
 		then ws_net_paid_inc_tax * ws_quantity else 0 end) as apr_net
 	,sum(case when d_moy = 5
 		then ws_net_paid_inc_tax * ws_quantity else 0 end) as may_net
 	,sum(case when d_moy = 6
 		then ws_net_paid_inc_tax * ws_quantity else 0 end) as jun_net
 	,sum(case when d_moy = 7
 		then ws_net_paid_inc_tax * ws_quantity else 0 end) as jul_net
 	,sum(case when d_moy = 8
 		then ws_net_paid_inc_tax * ws_quantity else 0 end) as aug_net
 	,sum(case when d_moy = 9
 		then ws_net_paid_inc_tax * ws_quantity else 0 end) as sep_net
 	,sum(case when d_moy = 10
 		then ws_net_paid_inc_tax * ws_quantity else 0 end) as oct_net
 	,sum(case when d_moy = 11
 		then ws_net_paid_inc_tax * ws_quantity else 0 end) as nov_net
 	,sum(case when d_moy = 12
 		then ws_net_paid_inc_tax * ws_quantity else 0 end) as dec_net
     from
          dev.iceberg100.web_sales
         ,dev.iceberg100.warehouse
         ,dev.iceberg100.date_dim
         ,dev.iceberg100.time_dim
 	  ,dev.iceberg100.ship_mode
     where
            ws_warehouse_sk =  w_warehouse_sk
        and ws_sold_date_sk = d_date_sk
        and ws_sold_time_sk = t_time_sk
 	and ws_ship_mode_sk = sm_ship_mode_sk
        and d_year = 2002
 	and t_time between 24285 and 24285+28800
 	and sm_carrier in ('HARMSTORF','USPS')
     group by
        w_warehouse_name
 	,w_warehouse_sq_ft
 	,w_city
 	,w_county
 	,w_state
 	,w_country
       ,d_year
 union all
     select
 	w_warehouse_name
 	,w_warehouse_sq_ft
 	,w_city
 	,w_county
 	,w_state
 	,w_country
 	,'HARMSTORF' || ',' || 'USPS' as ship_carriers
       ,d_year as year
 	,sum(case when d_moy = 1
 		then cs_ext_list_price* cs_quantity else 0 end) as jan_sales
 	,sum(case when d_moy = 2
 		then cs_ext_list_price* cs_quantity else 0 end) as feb_sales
 	,sum(case when d_moy = 3
 		then cs_ext_list_price* cs_quantity else 0 end) as mar_sales
 	,sum(case when d_moy = 4
 		then cs_ext_list_price* cs_quantity else 0 end) as apr_sales
 	,sum(case when d_moy = 5
 		then cs_ext_list_price* cs_quantity else 0 end) as may_sales
 	,sum(case when d_moy = 6
 		then cs_ext_list_price* cs_quantity else 0 end) as jun_sales
 	,sum(case when d_moy = 7
 		then cs_ext_list_price* cs_quantity else 0 end) as jul_sales
 	,sum(case when d_moy = 8
 		then cs_ext_list_price* cs_quantity else 0 end) as aug_sales
 	,sum(case when d_moy = 9
 		then cs_ext_list_price* cs_quantity else 0 end) as sep_sales
 	,sum(case when d_moy = 10
 		then cs_ext_list_price* cs_quantity else 0 end) as oct_sales
 	,sum(case when d_moy = 11
 		then cs_ext_list_price* cs_quantity else 0 end) as nov_sales
 	,sum(case when d_moy = 12
 		then cs_ext_list_price* cs_quantity else 0 end) as dec_sales
 	,sum(case when d_moy = 1
 		then cs_net_paid * cs_quantity else 0 end) as jan_net
 	,sum(case when d_moy = 2
 		then cs_net_paid * cs_quantity else 0 end) as feb_net
 	,sum(case when d_moy = 3
 		then cs_net_paid * cs_quantity else 0 end) as mar_net
 	,sum(case when d_moy = 4
 		then cs_net_paid * cs_quantity else 0 end) as apr_net
 	,sum(case when d_moy = 5
 		then cs_net_paid * cs_quantity else 0 end) as may_net
 	,sum(case when d_moy = 6
 		then cs_net_paid * cs_quantity else 0 end) as jun_net
 	,sum(case when d_moy = 7
 		then cs_net_paid * cs_quantity else 0 end) as jul_net
 	,sum(case when d_moy = 8
 		then cs_net_paid * cs_quantity else 0 end) as aug_net
 	,sum(case when d_moy = 9
 		then cs_net_paid * cs_quantity else 0 end) as sep_net
 	,sum(case when d_moy = 10
 		then cs_net_paid * cs_quantity else 0 end) as oct_net
 	,sum(case when d_moy = 11
 		then cs_net_paid * cs_quantity else 0 end) as nov_net
 	,sum(case when d_moy = 12
 		then cs_net_paid * cs_quantity else 0 end) as dec_net
     from
          dev.iceberg100.catalog_sales
         ,dev.iceberg100.warehouse
         ,dev.iceberg100.date_dim
         ,dev.iceberg100.time_dim
 	 ,dev.iceberg100.ship_mode
     where
            cs_warehouse_sk =  w_warehouse_sk
        and cs_sold_date_sk = d_date_sk
        and cs_sold_time_sk = t_time_sk
 	and cs_ship_mode_sk = sm_ship_mode_sk
        and d_year = 2002
 	and t_time between 24285 AND 24285+28800
 	and sm_carrier in ('HARMSTORF','USPS')
     group by
        w_warehouse_name
 	,w_warehouse_sq_ft
 	,w_city
 	,w_county
 	,w_state
 	,w_country
       ,d_year
 ) x
 group by
        w_warehouse_name
 	,w_warehouse_sq_ft
 	,w_city
 	,w_county
 	,w_state
 	,w_country
 	,ship_carriers
       ,year
 order by w_warehouse_name
 limit 100

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [ ]:
%%sql
--#q67

select  *
from (select i_category
            ,i_class
            ,i_brand
            ,i_product_name
            ,d_year
            ,d_qoy
            ,d_moy
            ,s_store_id
            ,sumsales
            ,rank() over (partition by i_category order by sumsales desc) rk
      from (select i_category
                  ,i_class
                  ,i_brand
                  ,i_product_name
                  ,d_year
                  ,d_qoy
                  ,d_moy
                  ,s_store_id
                  ,sum(coalesce(ss_sales_price*ss_quantity,0)) sumsales
            from dev.iceberg100.store_sales
                ,dev.iceberg100.date_dim
                ,dev.iceberg100.store
                ,dev.iceberg100.item
       where  ss_sold_date_sk=d_date_sk
          and ss_item_sk=i_item_sk
          and ss_store_sk = s_store_sk
          and d_month_seq between 1196 and 1196+11
       group by  rollup(i_category, i_class, i_brand, i_product_name, d_year, d_qoy, d_moy,s_store_id))dw1) dw2
where rk <= 100
order by i_category
        ,i_class
        ,i_brand
        ,i_product_name
        ,d_year
        ,d_qoy
        ,d_moy
        ,s_store_id
        ,sumsales
        ,rk
limit 100

In [ ]:
%%sql
--#q68

select  c_last_name
       ,c_first_name
       ,ca_city
       ,bought_city
       ,ss_ticket_number
       ,extended_price
       ,extended_tax
       ,list_price
 from (select ss_ticket_number
             ,ss_customer_sk
             ,ca_city bought_city
             ,sum(ss_ext_sales_price) extended_price
             ,sum(ss_ext_list_price) list_price
             ,sum(ss_ext_tax) extended_tax
       from dev.iceberg100.store_sales
           ,dev.iceberg100.date_dim
           ,dev.iceberg100.store
           ,dev.iceberg100.household_demographics
           ,dev.iceberg100.customer_address
       where dev.iceberg100.store_sales.ss_sold_date_sk = dev.iceberg100.date_dim.d_date_sk
         and dev.iceberg100.store_sales.ss_store_sk = dev.iceberg100.store.s_store_sk
        and dev.iceberg100.store_sales.ss_hdemo_sk = dev.iceberg100.household_demographics.hd_demo_sk
        and dev.iceberg100.store_sales.ss_addr_sk = dev.iceberg100.customer_address.ca_address_sk
        and dev.iceberg100.date_dim.d_dom between 1 and 2
        and (dev.iceberg100.household_demographics.hd_dep_count = 1 or
             dev.iceberg100.household_demographics.hd_vehicle_count= -1)
        and dev.iceberg100.date_dim.d_year in (1998,1998+1,1998+2)
        and dev.iceberg100.store.s_city in ('Bethel','Summit')
       group by ss_ticket_number
               ,ss_customer_sk
               ,ss_addr_sk,ca_city) dn
      ,dev.iceberg100.customer
      ,dev.iceberg100.customer_address current_addr
 where ss_customer_sk = c_customer_sk
   and dev.iceberg100.customer.c_current_addr_sk = current_addr.ca_address_sk
   and current_addr.ca_city <> bought_city
 order by c_last_name
         ,ss_ticket_number
 limit 100

In [ ]:
%%sql
--#q69

select
  cd_gender,
  cd_marital_status,
  cd_education_status,
  count(*) cnt1,
  cd_purchase_estimate,
  count(*) cnt2,
  cd_credit_rating,
  count(*) cnt3
 from
  dev.iceberg100.customer c,dev.iceberg100.customer_address ca,dev.iceberg100.customer_demographics
 where
  c.c_current_addr_sk = ca.ca_address_sk and
  ca_state in ('OK','GA','VA') and
  cd_demo_sk = c.c_current_cdemo_sk and
  exists (select *
          from dev.iceberg100.store_sales,dev.iceberg100.date_dim
          where c.c_customer_sk = ss_customer_sk and
                ss_sold_date_sk = d_date_sk and
                d_year = 2004 and
                d_moy between 4 and 4+2) and
   (not exists (select *
            from dev.iceberg100.web_sales,dev.iceberg100.date_dim
            where c.c_customer_sk = ws_bill_customer_sk and
                  ws_sold_date_sk = d_date_sk and
                  d_year = 2004 and
                  d_moy between 4 and 4+2) and
    not exists (select *
            from dev.iceberg100.catalog_sales,dev.iceberg100.date_dim
            where c.c_customer_sk = cs_ship_customer_sk and
                  cs_sold_date_sk = d_date_sk and
                  d_year = 2004 and
                  d_moy between 4 and 4+2))
 group by cd_gender,
          cd_marital_status,
          cd_education_status,
          cd_purchase_estimate,
          cd_credit_rating
 order by cd_gender,
          cd_marital_status,
          cd_education_status,
          cd_purchase_estimate,
          cd_credit_rating
 limit 100

In [ ]:
%%sql
--#q70

select
    sum(ss_net_profit) as total_sum
   ,s_state
   ,s_county
   ,grouping(s_state)+grouping(s_county) as lochierarchy
   ,rank() over (
 	partition by grouping(s_state)+grouping(s_county),
 	case when grouping(s_county) = 0 then s_state end
 	order by sum(ss_net_profit) desc) as rank_within_parent
 from
    dev.iceberg100.store_sales
   ,dev.iceberg100.date_dim       d1
   ,dev.iceberg100.store
 where
    d1.d_month_seq between 1197 and 1197+11
 and d1.d_date_sk = ss_sold_date_sk
 and s_store_sk  = ss_store_sk
 and s_state in
             ( select s_state
               from  (select s_state as s_state,
 			    rank() over ( partition by s_state order by sum(ss_net_profit) desc) as ranking
                      from   dev.iceberg100.store_sales, dev.iceberg100.store, dev.iceberg100.date_dim
                      where  d_month_seq between 1197 and 1197+11
 			    and d_date_sk = ss_sold_date_sk
 			    and s_store_sk  = ss_store_sk
                      group by s_state
                     ) tmp1
               where ranking <= 5
             )
 group by rollup(s_state,s_county)
 order by
   lochierarchy desc
  ,case when lochierarchy = 0 then s_state end
  ,rank_within_parent
 limit 100

In [ ]:
%%sql
--#q71

select i_brand_id brand_id, i_brand brand,t_hour,t_minute,
 	sum(ext_price) ext_price
 from dev.iceberg100.item, (select ws_ext_sales_price as ext_price,
                        ws_sold_date_sk as sold_date_sk,
                        ws_item_sk as sold_item_sk,
                        ws_sold_time_sk as time_sk
                 from dev.iceberg100.web_sales,dev.iceberg100.date_dim
                 where d_date_sk = ws_sold_date_sk
                   and d_moy=12
                   and d_year=1999
                 union all
                 select cs_ext_sales_price as ext_price,
                        cs_sold_date_sk as sold_date_sk,
                        cs_item_sk as sold_item_sk,
                        cs_sold_time_sk as time_sk
                 from dev.iceberg100.catalog_sales,dev.iceberg100.date_dim
                 where d_date_sk = cs_sold_date_sk
                   and d_moy=12
                   and d_year=1999
                 union all
                 select ss_ext_sales_price as ext_price,
                        ss_sold_date_sk as sold_date_sk,
                        ss_item_sk as sold_item_sk,
                        ss_sold_time_sk as time_sk
                 from dev.iceberg100.store_sales,dev.iceberg100.date_dim
                 where d_date_sk = ss_sold_date_sk
                   and d_moy=12
                   and d_year=1999
                 ) tmp,dev.iceberg100.time_dim
 where
   sold_item_sk = i_item_sk
   and i_manager_id=1
   and time_sk = t_time_sk
   and (t_meal_time = 'breakfast' or t_meal_time = 'dinner')
 group by i_brand, i_brand_id,t_hour,t_minute
 order by ext_price desc, i_brand_id

In [ ]:
%%sql
--#q72

select  i_item_desc
      ,w_warehouse_name
      ,d1.d_week_seq
      ,sum(case when p_promo_sk is null then 1 else 0 end) no_promo
      ,sum(case when p_promo_sk is not null then 1 else 0 end) promo
      ,count(*) total_cnt
from dev.iceberg100.catalog_sales
join dev.iceberg100.inventory on (cs_item_sk = inv_item_sk)
join dev.iceberg100.warehouse on (w_warehouse_sk=inv_warehouse_sk)
join dev.iceberg100.item on (i_item_sk = cs_item_sk)
join dev.iceberg100.customer_demographics on (cs_bill_cdemo_sk = cd_demo_sk)
join dev.iceberg100.household_demographics on (cs_bill_hdemo_sk = hd_demo_sk)
join dev.iceberg100.date_dim d1 on (cs_sold_date_sk = d1.d_date_sk)
join dev.iceberg100.date_dim d2 on (inv_date_sk = d2.d_date_sk)
join dev.iceberg100.date_dim d3 on (cs_ship_date_sk = d3.d_date_sk)
left outer join dev.iceberg100.promotion on (cs_promo_sk=p_promo_sk)
left outer join dev.iceberg100.catalog_returns on (cr_item_sk = cs_item_sk and cr_order_number = cs_order_number)
where d1.d_week_seq = d2.d_week_seq
  and inv_quantity_on_hand < cs_quantity
  and d3.d_date > d1.d_date + interval 5 days
  and hd_buy_potential = '>10000'
  and d1.d_year = 2002
  and cd_marital_status = 'D'
group by i_item_desc,w_warehouse_name,d1.d_week_seq
order by total_cnt desc, i_item_desc, w_warehouse_name, d_week_seq
limit 100

In [ ]:
%%sql
--#q73

select c_last_name
       ,c_first_name
       ,c_salutation
       ,c_preferred_cust_flag
       ,ss_ticket_number
       ,cnt from
   (select ss_ticket_number
          ,ss_customer_sk
          ,count(*) cnt
    from dev.iceberg100.store_sales,dev.iceberg100.date_dim,dev.iceberg100.store,dev.iceberg100.household_demographics
    where dev.iceberg100.store_sales.ss_sold_date_sk = dev.iceberg100.date_dim.d_date_sk
    and dev.iceberg100.store_sales.ss_store_sk = dev.iceberg100.store.s_store_sk
    and dev.iceberg100.store_sales.ss_hdemo_sk = dev.iceberg100.household_demographics.hd_demo_sk
    and dev.iceberg100.date_dim.d_dom between 1 and 2
    and (dev.iceberg100.household_demographics.hd_buy_potential = '501-1000' or
         dev.iceberg100.household_demographics.hd_buy_potential = 'Unknown')
    and dev.iceberg100.household_demographics.hd_vehicle_count > 0
    and case when dev.iceberg100.household_demographics.hd_vehicle_count > 0 then
             dev.iceberg100.household_demographics.hd_dep_count/ dev.iceberg100.household_demographics.hd_vehicle_count else null end > 1
    and dev.iceberg100.date_dim.d_year in (1999,1999+1,1999+2)
    and dev.iceberg100.store.s_county in ('Franklin Parish','Ziebach County','Luce County','Williamson County')
    group by ss_ticket_number,ss_customer_sk) dj,dev.iceberg100.customer
    where ss_customer_sk = c_customer_sk
      and cnt between 1 and 5
    order by cnt desc, c_last_name asc

In [ ]:
%%sql
--#q74

with year_total as (
 select c_customer_id customer_id
       ,c_first_name customer_first_name
       ,c_last_name customer_last_name
       ,d_year as year
       ,max(ss_net_paid) year_total
       ,'s' sale_type
 from dev.iceberg100.customer
     ,dev.iceberg100.store_sales
     ,dev.iceberg100.date_dim
 where c_customer_sk = ss_customer_sk
   and ss_sold_date_sk = d_date_sk
   and d_year in (2001,2001+1)
 group by c_customer_id
         ,c_first_name
         ,c_last_name
         ,d_year
 union all
 select c_customer_id customer_id
       ,c_first_name customer_first_name
       ,c_last_name customer_last_name
       ,d_year as year
       ,max(ws_net_paid) year_total
       ,'w' sale_type
 from dev.iceberg100.customer
     ,dev.iceberg100.web_sales
     ,dev.iceberg100.date_dim
 where c_customer_sk = ws_bill_customer_sk
   and ws_sold_date_sk = d_date_sk
   and d_year in (2001,2001+1)
 group by c_customer_id
         ,c_first_name
         ,c_last_name
         ,d_year
         )
  select
        t_s_secyear.customer_id, t_s_secyear.customer_first_name, t_s_secyear.customer_last_name
 from year_total t_s_firstyear
     ,year_total t_s_secyear
     ,year_total t_w_firstyear
     ,year_total t_w_secyear
 where t_s_secyear.customer_id = t_s_firstyear.customer_id
         and t_s_firstyear.customer_id = t_w_secyear.customer_id
         and t_s_firstyear.customer_id = t_w_firstyear.customer_id
         and t_s_firstyear.sale_type = 's'
         and t_w_firstyear.sale_type = 'w'
         and t_s_secyear.sale_type = 's'
         and t_w_secyear.sale_type = 'w'
         and t_s_firstyear.year = 2001
         and t_s_secyear.year = 2001+1
         and t_w_firstyear.year = 2001
         and t_w_secyear.year = 2001+1
         and t_s_firstyear.year_total > 0
         and t_w_firstyear.year_total > 0
         and case when t_w_firstyear.year_total > 0 then t_w_secyear.year_total / t_w_firstyear.year_total else null end
           > case when t_s_firstyear.year_total > 0 then t_s_secyear.year_total / t_s_firstyear.year_total else null end
 order by 3,1,2
limit 100

In [ ]:
%%sql
--#q75

WITH all_sales AS (
 SELECT d_year
       ,i_brand_id
       ,i_class_id
       ,i_category_id
       ,i_manufact_id
       ,SUM(sales_cnt) AS sales_cnt
       ,SUM(sales_amt) AS sales_amt
 FROM (SELECT d_year
             ,i_brand_id
             ,i_class_id
             ,i_category_id
             ,i_manufact_id
             ,cs_quantity - COALESCE(cr_return_quantity,0) AS sales_cnt
             ,cs_ext_sales_price - COALESCE(cr_return_amount,0.0) AS sales_amt
       FROM dev.iceberg100.catalog_sales JOIN dev.iceberg100.item ON i_item_sk=cs_item_sk
                          JOIN dev.iceberg100.date_dim ON d_date_sk=cs_sold_date_sk
                          LEFT JOIN dev.iceberg100.catalog_returns ON (cs_order_number=cr_order_number
                                                    AND cs_item_sk=cr_item_sk)
       WHERE i_category='Sports'
       UNION
       SELECT d_year
             ,i_brand_id
             ,i_class_id
             ,i_category_id
             ,i_manufact_id
             ,ss_quantity - COALESCE(sr_return_quantity,0) AS sales_cnt
             ,ss_ext_sales_price - COALESCE(sr_return_amt,0.0) AS sales_amt
       FROM dev.iceberg100.store_sales JOIN dev.iceberg100.item ON i_item_sk=ss_item_sk
                        JOIN dev.iceberg100.date_dim ON d_date_sk=ss_sold_date_sk
                        LEFT JOIN dev.iceberg100.store_returns ON (ss_ticket_number=sr_ticket_number
                                                AND ss_item_sk=sr_item_sk)
       WHERE i_category='Sports'
       UNION
       SELECT d_year
             ,i_brand_id
             ,i_class_id
             ,i_category_id
             ,i_manufact_id
             ,ws_quantity - COALESCE(wr_return_quantity,0) AS sales_cnt
             ,ws_ext_sales_price - COALESCE(wr_return_amt,0.0) AS sales_amt
       FROM dev.iceberg100.web_sales JOIN dev.iceberg100.item ON i_item_sk=ws_item_sk
                      JOIN dev.iceberg100.date_dim ON d_date_sk=ws_sold_date_sk
                      LEFT JOIN dev.iceberg100.web_returns ON (ws_order_number=wr_order_number
                                            AND ws_item_sk=wr_item_sk)
       WHERE i_category='Sports') sales_detail
 GROUP BY d_year, i_brand_id, i_class_id, i_category_id, i_manufact_id)
 SELECT  prev_yr.d_year AS prev_year
                          ,curr_yr.d_year AS year
                          ,curr_yr.i_brand_id
                          ,curr_yr.i_class_id
                          ,curr_yr.i_category_id
                          ,curr_yr.i_manufact_id
                          ,prev_yr.sales_cnt AS prev_yr_cnt
                          ,curr_yr.sales_cnt AS curr_yr_cnt
                          ,curr_yr.sales_cnt-prev_yr.sales_cnt AS sales_cnt_diff
                          ,curr_yr.sales_amt-prev_yr.sales_amt AS sales_amt_diff
 FROM all_sales curr_yr, all_sales prev_yr
 WHERE curr_yr.i_brand_id=prev_yr.i_brand_id
   AND curr_yr.i_class_id=prev_yr.i_class_id
   AND curr_yr.i_category_id=prev_yr.i_category_id
   AND curr_yr.i_manufact_id=prev_yr.i_manufact_id
   AND curr_yr.d_year=2001
   AND prev_yr.d_year=2001-1
   AND CAST(curr_yr.sales_cnt AS DECIMAL(17,2))/CAST(prev_yr.sales_cnt AS DECIMAL(17,2))<0.9
 ORDER BY sales_cnt_diff,sales_amt_diff
 limit 100

In [ ]:
%%sql
--#q76

select  channel, col_name, d_year, d_qoy, i_category, COUNT(*) sales_cnt, SUM(ext_sales_price) sales_amt FROM (
        SELECT 'store' as channel, 'ss_cdemo_sk' col_name, d_year, d_qoy, i_category, ss_ext_sales_price ext_sales_price
         FROM dev.iceberg100.store_sales, dev.iceberg100.item, dev.iceberg100.date_dim
         WHERE ss_cdemo_sk IS NULL
           AND ss_sold_date_sk=d_date_sk
           AND ss_item_sk=i_item_sk
        UNION ALL
        SELECT 'web' as channel, 'ws_ship_hdemo_sk' col_name, d_year, d_qoy, i_category, ws_ext_sales_price ext_sales_price
         FROM dev.iceberg100.web_sales, dev.iceberg100.item, dev.iceberg100.date_dim
         WHERE ws_ship_hdemo_sk IS NULL
           AND ws_sold_date_sk=d_date_sk
           AND ws_item_sk=i_item_sk
        UNION ALL
        SELECT 'catalog' as channel, 'cs_ship_customer_sk' col_name, d_year, d_qoy, i_category, cs_ext_sales_price ext_sales_price
         FROM dev.iceberg100.catalog_sales, dev.iceberg100.item, dev.iceberg100.date_dim
         WHERE cs_ship_customer_sk IS NULL
           AND cs_sold_date_sk=d_date_sk
           AND cs_item_sk=i_item_sk) foo
GROUP BY channel, col_name, d_year, d_qoy, i_category
ORDER BY channel, col_name, d_year, d_qoy, i_category
limit 100

In [ ]:
%%sql
--#q77

with ss as
 (select s_store_sk,
         sum(ss_ext_sales_price) as sales,
         sum(ss_net_profit) as profit
 from dev.iceberg100.store_sales,
      dev.iceberg100.date_dim,
      dev.iceberg100.store
 where ss_sold_date_sk = d_date_sk
       and d_date between cast('2001-08-27' as date)
                  and (cast('2001-08-27' as date) +  INTERVAL 30 days)
       and ss_store_sk = s_store_sk
 group by s_store_sk)
 ,
 sr as
 (select s_store_sk,
         sum(sr_return_amt) as returns,
         sum(sr_net_loss) as profit_loss
 from dev.iceberg100.store_returns,
      dev.iceberg100.date_dim,
      dev.iceberg100.store
 where sr_returned_date_sk = d_date_sk
       and d_date between cast('2001-08-27' as date)
                  and (cast('2001-08-27' as date) +  INTERVAL 30 days)
       and sr_store_sk = s_store_sk
 group by s_store_sk),
 cs as
 (select cs_call_center_sk,
        sum(cs_ext_sales_price) as sales,
        sum(cs_net_profit) as profit
 from dev.iceberg100.catalog_sales,
      dev.iceberg100.date_dim
 where cs_sold_date_sk = d_date_sk
       and d_date between cast('2001-08-27' as date)
                  and (cast('2001-08-27' as date) +  INTERVAL 30 days)
 group by cs_call_center_sk
 ),
 cr as
 (select cr_call_center_sk,
         sum(cr_return_amount) as returns,
         sum(cr_net_loss) as profit_loss
 from dev.iceberg100.catalog_returns,
      dev.iceberg100.date_dim
 where cr_returned_date_sk = d_date_sk
       and d_date between cast('2001-08-27' as date)
                  and (cast('2001-08-27' as date) +  INTERVAL 30 days)
 group by cr_call_center_sk
 ),
 ws as
 ( select wp_web_page_sk,
        sum(ws_ext_sales_price) as sales,
        sum(ws_net_profit) as profit
 from dev.iceberg100.web_sales,
      dev.iceberg100.date_dim,
      dev.iceberg100.web_page
 where ws_sold_date_sk = d_date_sk
       and d_date between cast('2001-08-27' as date)
                  and (cast('2001-08-27' as date) +  INTERVAL 30 days)
       and ws_web_page_sk = wp_web_page_sk
 group by wp_web_page_sk),
 wr as
 (select wp_web_page_sk,
        sum(wr_return_amt) as returns,
        sum(wr_net_loss) as profit_loss
 from dev.iceberg100.web_returns,
      dev.iceberg100.date_dim,
      dev.iceberg100.web_page
 where wr_returned_date_sk = d_date_sk
       and d_date between cast('2001-08-27' as date)
                  and (cast('2001-08-27' as date) +  INTERVAL 30 days)
       and wr_web_page_sk = wp_web_page_sk
 group by wp_web_page_sk)
  select  channel
        , id
        , sum(sales) as sales
        , sum(returns) as returns
        , sum(profit) as profit
 from
 (select 'store channel' as channel
        , ss.s_store_sk as id
        , sales
        , coalesce(returns, 0) as returns
        , (profit - coalesce(profit_loss,0)) as profit
 from   ss left join sr
        on  ss.s_store_sk = sr.s_store_sk
 union all
 select 'catalog channel' as channel
        , cs_call_center_sk as id
        , sales
        , returns
        , (profit - profit_loss) as profit
 from  cs
       , cr
 union all
 select 'web channel' as channel
        , ws.wp_web_page_sk as id
        , sales
        , coalesce(returns, 0) returns
        , (profit - coalesce(profit_loss,0)) as profit
 from   ws left join wr
        on  ws.wp_web_page_sk = wr.wp_web_page_sk
 ) x
 group by rollup (channel, id)
 order by channel
         ,id
 limit 100

In [ ]:
%%sql
--#q78

with ws as
  (select d_year AS ws_sold_year, ws_item_sk,
    ws_bill_customer_sk ws_customer_sk,
    sum(ws_quantity) ws_qty,
    sum(ws_wholesale_cost) ws_wc,
    sum(ws_sales_price) ws_sp
   from dev.iceberg100.web_sales
   left join dev.iceberg100.web_returns on wr_order_number=ws_order_number and ws_item_sk=wr_item_sk
   join dev.iceberg100.date_dim on ws_sold_date_sk = d_date_sk
   where wr_order_number is null
   group by d_year, ws_item_sk, ws_bill_customer_sk
   ),
cs as
  (select d_year AS cs_sold_year, cs_item_sk,
    cs_bill_customer_sk cs_customer_sk,
    sum(cs_quantity) cs_qty,
    sum(cs_wholesale_cost) cs_wc,
    sum(cs_sales_price) cs_sp
   from dev.iceberg100.catalog_sales
   left join dev.iceberg100.catalog_returns on cr_order_number=cs_order_number and cs_item_sk=cr_item_sk
   join dev.iceberg100.date_dim on cs_sold_date_sk = d_date_sk
   where cr_order_number is null
   group by d_year, cs_item_sk, cs_bill_customer_sk
   ),
ss as
  (select d_year AS ss_sold_year, ss_item_sk,
    ss_customer_sk,
    sum(ss_quantity) ss_qty,
    sum(ss_wholesale_cost) ss_wc,
    sum(ss_sales_price) ss_sp
   from dev.iceberg100.store_sales
   left join dev.iceberg100.store_returns on sr_ticket_number=ss_ticket_number and ss_item_sk=sr_item_sk
   join dev.iceberg100.date_dim on ss_sold_date_sk = d_date_sk
   where sr_ticket_number is null
   group by d_year, ss_item_sk, ss_customer_sk
   )
 select
ss_sold_year, ss_item_sk, ss_customer_sk,
round(ss_qty/(coalesce(ws_qty,0)+coalesce(cs_qty,0)),2) ratio,
ss_qty store_qty, ss_wc store_wholesale_cost, ss_sp store_sales_price,
coalesce(ws_qty,0)+coalesce(cs_qty,0) other_chan_qty,
coalesce(ws_wc,0)+coalesce(cs_wc,0) other_chan_wholesale_cost,
coalesce(ws_sp,0)+coalesce(cs_sp,0) other_chan_sales_price
from ss
left join ws on (ws_sold_year=ss_sold_year and ws_item_sk=ss_item_sk and ws_customer_sk=ss_customer_sk)
left join cs on (cs_sold_year=ss_sold_year and cs_item_sk=ss_item_sk and cs_customer_sk=ss_customer_sk)
where (coalesce(ws_qty,0)>0 or coalesce(cs_qty, 0)>0) and ss_sold_year=2002
order by
  ss_sold_year, ss_item_sk, ss_customer_sk,
  ss_qty desc, ss_wc desc, ss_sp desc,
  other_chan_qty,
  other_chan_wholesale_cost,
  other_chan_sales_price,
  ratio
limit 100

In [ ]:
%%sql
--#q79

select
  c_last_name,c_first_name,substr(s_city,1,30),ss_ticket_number,amt,profit
  from
   (select ss_ticket_number
          ,ss_customer_sk
          ,dev.iceberg100.store.s_city
          ,sum(ss_coupon_amt) amt
          ,sum(ss_net_profit) profit
    from dev.iceberg100.store_sales,dev.iceberg100.date_dim,dev.iceberg100.store,dev.iceberg100.household_demographics
    where dev.iceberg100.store_sales.ss_sold_date_sk = dev.iceberg100.date_dim.d_date_sk
    and dev.iceberg100.store_sales.ss_store_sk = dev.iceberg100.store.s_store_sk
    and dev.iceberg100.store_sales.ss_hdemo_sk = dev.iceberg100.household_demographics.hd_demo_sk
    and (dev.iceberg100.household_demographics.hd_dep_count = 0 or dev.iceberg100.household_demographics.hd_vehicle_count > 0)
    and dev.iceberg100.date_dim.d_dow = 1
    and dev.iceberg100.date_dim.d_year in (2000,2000+1,2000+2)
    and dev.iceberg100.store.s_number_employees between 200 and 295
    group by ss_ticket_number,ss_customer_sk,ss_addr_sk,dev.iceberg100.store.s_city) ms,dev.iceberg100.customer
    where ss_customer_sk = c_customer_sk
 order by c_last_name,c_first_name,substr(s_city,1,30), profit
limit 100

In [ ]:
%%sql
--#q80

with ssr as
 (select  s_store_id as store_id,
          sum(ss_ext_sales_price) as sales,
          sum(coalesce(sr_return_amt, 0)) as returns,
          sum(ss_net_profit - coalesce(sr_net_loss, 0)) as profit
  from dev.iceberg100.store_sales left outer join dev.iceberg100.store_returns on
         (ss_item_sk = sr_item_sk and ss_ticket_number = sr_ticket_number),
     dev.iceberg100.date_dim,
     dev.iceberg100.store,
     dev.iceberg100.item,
     dev.iceberg100.promotion
 where ss_sold_date_sk = d_date_sk
       and d_date between cast('1999-08-12' as date)
                  and (cast('1999-08-12' as date) +  INTERVAL 60 days)
       and ss_store_sk = s_store_sk
       and ss_item_sk = i_item_sk
       and i_current_price > 50
       and ss_promo_sk = p_promo_sk
       and p_channel_tv = 'N'
 group by s_store_id)
 ,
 csr as
 (select  cp_catalog_page_id as catalog_page_id,
          sum(cs_ext_sales_price) as sales,
          sum(coalesce(cr_return_amount, 0)) as returns,
          sum(cs_net_profit - coalesce(cr_net_loss, 0)) as profit
  from dev.iceberg100.catalog_sales left outer join dev.iceberg100.catalog_returns on
         (cs_item_sk = cr_item_sk and cs_order_number = cr_order_number),
     dev.iceberg100.date_dim,
     dev.iceberg100.catalog_page,
     dev.iceberg100.item,
     dev.iceberg100.promotion
 where cs_sold_date_sk = d_date_sk
       and d_date between cast('1999-08-12' as date)
                  and (cast('1999-08-12' as date) +  INTERVAL 60 days)
        and cs_catalog_page_sk = cp_catalog_page_sk
       and cs_item_sk = i_item_sk
       and i_current_price > 50
       and cs_promo_sk = p_promo_sk
       and p_channel_tv = 'N'
group by cp_catalog_page_id)
 ,
 wsr as
 (select  web_site_id,
          sum(ws_ext_sales_price) as sales,
          sum(coalesce(wr_return_amt, 0)) as returns,
          sum(ws_net_profit - coalesce(wr_net_loss, 0)) as profit
  from dev.iceberg100.web_sales left outer join dev.iceberg100.web_returns on
         (ws_item_sk = wr_item_sk and ws_order_number = wr_order_number),
     dev.iceberg100.date_dim,
     dev.iceberg100.web_site,
     dev.iceberg100.item,
     dev.iceberg100.promotion
 where ws_sold_date_sk = d_date_sk
       and d_date between cast('1999-08-12' as date)
                  and (cast('1999-08-12' as date) +  INTERVAL 60 days)
        and ws_web_site_sk = web_site_sk
       and ws_item_sk = i_item_sk
       and i_current_price > 50
       and ws_promo_sk = p_promo_sk
       and p_channel_tv = 'N'
group by web_site_id)
  select  channel
        , id
        , sum(sales) as sales
        , sum(returns) as returns
        , sum(profit) as profit
 from
 (select 'store channel' as channel
        , 'store' || store_id as id
        , sales
        , returns
        , profit
 from   ssr
 union all
 select 'catalog channel' as channel
        , 'catalog_page' || catalog_page_id as id
        , sales
        , returns
        , profit
 from  csr
 union all
 select 'web channel' as channel
        , 'web_site' || web_site_id as id
        , sales
        , returns
        , profit
 from   wsr
 ) x
 group by rollup (channel, id)
 order by channel
         ,id
 limit 100

In [ ]:
%%sql
--#q81

with customer_total_return as
 (select cr_returning_customer_sk as ctr_customer_sk
        ,ca_state as ctr_state,
 	sum(cr_return_amt_inc_tax) as ctr_total_return
 from dev.iceberg100.catalog_returns
     ,dev.iceberg100.date_dim
     ,dev.iceberg100.customer_address
 where cr_returned_date_sk = d_date_sk
   and d_year =2001
   and cr_returning_addr_sk = ca_address_sk
 group by cr_returning_customer_sk
         ,ca_state )
  select  c_customer_id,c_salutation,c_first_name,c_last_name,ca_street_number,ca_street_name
                   ,ca_street_type,ca_suite_number,ca_city,ca_county,ca_state,ca_zip,ca_country,ca_gmt_offset
                  ,ca_location_type,ctr_total_return
 from customer_total_return ctr1
     ,dev.iceberg100.customer_address
     ,dev.iceberg100.customer
 where ctr1.ctr_total_return > (select avg(ctr_total_return)*1.2
 			  from customer_total_return ctr2
                  	  where ctr1.ctr_state = ctr2.ctr_state)
       and ca_address_sk = c_current_addr_sk
       and ca_state = 'NC'
       and ctr1.ctr_customer_sk = c_customer_sk
 order by c_customer_id,c_salutation,c_first_name,c_last_name,ca_street_number,ca_street_name
                   ,ca_street_type,ca_suite_number,ca_city,ca_county,ca_state,ca_zip,ca_country,ca_gmt_offset
                  ,ca_location_type,ctr_total_return
 limit 100

In [ ]:
%%sql
--#q82

select  i_item_id
       ,i_item_desc
       ,i_current_price
 from dev.iceberg100.item, dev.iceberg100.inventory, dev.iceberg100.date_dim, dev.iceberg100.store_sales
 where i_current_price between 82 and 82+30
 and inv_item_sk = i_item_sk
 and d_date_sk=inv_date_sk
 and d_date between cast('2002-03-10' as date) and (cast('2002-03-10' as date) +  INTERVAL 60 days)
 and i_manufact_id in (941,920,105,693)
 and inv_quantity_on_hand between 100 and 500
 and ss_item_sk = i_item_sk
 group by i_item_id,i_item_desc,i_current_price
 order by i_item_id
 limit 100

In [ ]:
%%sql
--#q83

with sr_items as
 (select i_item_id item_id,
        sum(sr_return_quantity) sr_item_qty
 from dev.iceberg100.store_returns,
      dev.iceberg100.item,
      dev.iceberg100.date_dim
 where sr_item_sk = i_item_sk
 and   d_date    in
	(select d_date
	from dev.iceberg100.date_dim
	where d_week_seq in
		(select d_week_seq
		from dev.iceberg100.date_dim
	  where d_date in ('1999-04-14','1999-09-28','1999-11-12')))
 and   sr_returned_date_sk   = d_date_sk
 group by i_item_id),
 cr_items as
 (select i_item_id item_id,
        sum(cr_return_quantity) cr_item_qty
 from dev.iceberg100.catalog_returns,
      dev.iceberg100.item,
      dev.iceberg100.date_dim
 where cr_item_sk = i_item_sk
 and   d_date    in
	(select d_date
	from dev.iceberg100.date_dim
	where d_week_seq in
		(select d_week_seq
		from dev.iceberg100.date_dim
	  where d_date in ('1999-04-14','1999-09-28','1999-11-12')))
 and   cr_returned_date_sk   = d_date_sk
 group by i_item_id),
 wr_items as
 (select i_item_id item_id,
        sum(wr_return_quantity) wr_item_qty
 from dev.iceberg100.web_returns,
      dev.iceberg100.item,
      dev.iceberg100.date_dim
 where wr_item_sk = i_item_sk
 and   d_date    in
	(select d_date
	from dev.iceberg100.date_dim
	where d_week_seq in
		(select d_week_seq
		from dev.iceberg100.date_dim
		where d_date in ('1999-04-14','1999-09-28','1999-11-12')))
 and   wr_returned_date_sk   = d_date_sk
 group by i_item_id)
  select  sr_items.item_id
       ,sr_item_qty
       ,sr_item_qty/(sr_item_qty+cr_item_qty+wr_item_qty)/3.0 * 100 sr_dev
       ,cr_item_qty
       ,cr_item_qty/(sr_item_qty+cr_item_qty+wr_item_qty)/3.0 * 100 cr_dev
       ,wr_item_qty
       ,wr_item_qty/(sr_item_qty+cr_item_qty+wr_item_qty)/3.0 * 100 wr_dev
       ,(sr_item_qty+cr_item_qty+wr_item_qty)/3.0 average
 from sr_items
     ,cr_items
     ,wr_items
 where sr_items.item_id=cr_items.item_id
   and sr_items.item_id=wr_items.item_id
 order by sr_items.item_id
         ,sr_item_qty
 limit 100

In [ ]:
%%sql
--#q84

select  c_customer_id as customer_id
       , coalesce(c_last_name,'') || ', ' || coalesce(c_first_name,'') as customername
 from dev.iceberg100.customer
     ,dev.iceberg100.customer_address
     ,dev.iceberg100.customer_demographics
     ,dev.iceberg100.household_demographics
     ,dev.iceberg100.income_band
     ,dev.iceberg100.store_returns
 where ca_city	        =  'Antioch'
   and c_current_addr_sk = ca_address_sk
   and ib_lower_bound   >=  55019
   and ib_upper_bound   <=  55019 + 50000
   and ib_income_band_sk = hd_income_band_sk
   and cd_demo_sk = c_current_cdemo_sk
   and hd_demo_sk = c_current_hdemo_sk
   and sr_cdemo_sk = cd_demo_sk
 order by c_customer_id
 limit 100

In [ ]:
%%sql
-#q85

select  substr(r_reason_desc,1,20)
       ,avg(ws_quantity)
       ,avg(wr_refunded_cash)
       ,avg(wr_fee)
 from dev.iceberg100.web_sales, dev.iceberg100.web_returns, dev.iceberg100.web_page, dev.iceberg100.customer_demographics cd1,
      dev.iceberg100.customer_demographics cd2, dev.iceberg100.customer_address, dev.iceberg100.date_dim, dev.iceberg100.reason
 where ws_web_page_sk = wp_web_page_sk
   and ws_item_sk = wr_item_sk
   and ws_order_number = wr_order_number
   and ws_sold_date_sk = d_date_sk and d_year = 2001
   and cd1.cd_demo_sk = wr_refunded_cdemo_sk
   and cd2.cd_demo_sk = wr_returning_cdemo_sk
   and ca_address_sk = wr_refunded_addr_sk
   and r_reason_sk = wr_reason_sk
   and
   (
    (
     cd1.cd_marital_status = 'S'
     and
     cd1.cd_marital_status = cd2.cd_marital_status
     and
     cd1.cd_education_status = '2 yr Degree'
     and
     cd1.cd_education_status = cd2.cd_education_status
     and
     ws_sales_price between 100.00 and 150.00
    )
   or
    (
     cd1.cd_marital_status = 'D'
     and
     cd1.cd_marital_status = cd2.cd_marital_status
     and
     cd1.cd_education_status = 'Advanced Degree'
     and
     cd1.cd_education_status = cd2.cd_education_status
     and
     ws_sales_price between 50.00 and 100.00
    )
   or
    (
     cd1.cd_marital_status = 'W'
     and
     cd1.cd_marital_status = cd2.cd_marital_status
     and
     cd1.cd_education_status = '4 yr Degree'
     and
     cd1.cd_education_status = cd2.cd_education_status
     and
     ws_sales_price between 150.00 and 200.00
    )
   )
   and
   (
    (
     ca_country = 'United States'
     and
     ca_state in ('OK', 'TX', 'MO')
     and ws_net_profit between 100 and 200
    )
    or
    (
     ca_country = 'United States'
     and
     ca_state in ('GA', 'KS', 'NC')
     and ws_net_profit between 150 and 300
    )
    or
    (
     ca_country = 'United States'
     and
     ca_state in ('VA', 'WI', 'WV')
     and ws_net_profit between 50 and 250
    )
   )
group by r_reason_desc
order by substr(r_reason_desc,1,20)
        ,avg(ws_quantity)
        ,avg(wr_refunded_cash)
        ,avg(wr_fee)
limit 100

In [ ]:
%%sql
--#q86

select
    sum(ws_net_paid) as total_sum
   ,i_category
   ,i_class
   ,grouping(i_category)+grouping(i_class) as lochierarchy
   ,rank() over (
 	partition by grouping(i_category)+grouping(i_class),
 	case when grouping(i_class) = 0 then i_category end
 	order by sum(ws_net_paid) desc) as rank_within_parent
 from
    dev.iceberg100.web_sales
   ,dev.iceberg100.date_dim       d1
   ,dev.iceberg100.item
 where
    d1.d_month_seq between 1215 and 1215+11
 and d1.d_date_sk = ws_sold_date_sk
 and i_item_sk  = ws_item_sk
 group by rollup(i_category,i_class)
 order by
   lochierarchy desc,
   case when lochierarchy = 0 then i_category end,
   rank_within_parent
 limit 100

In [ ]:
%%sql
--#q87

select count(*)
from ((select distinct c_last_name, c_first_name, d_date
       from dev.iceberg100.store_sales, dev.iceberg100.date_dim, dev.iceberg100.customer
       where store_sales.ss_sold_date_sk = dev.iceberg100.date_dim.d_date_sk
         and store_sales.ss_customer_sk = dev.iceberg100.customer.c_customer_sk
         and d_month_seq between 1221 and 1221+11)
       except
      (select distinct c_last_name, c_first_name, d_date
       from dev.iceberg100.catalog_sales, dev.iceberg100.date_dim, dev.iceberg100.customer
       where dev.iceberg100.catalog_sales.cs_sold_date_sk = dev.iceberg100.date_dim.d_date_sk
         and dev.iceberg100.catalog_sales.cs_bill_customer_sk = dev.iceberg100.customer.c_customer_sk
         and d_month_seq between 1221 and 1221+11)
       except
      (select distinct c_last_name, c_first_name, d_date
       from dev.iceberg100.web_sales, dev.iceberg100.date_dim, dev.iceberg100.customer
       where dev.iceberg100.web_sales.ws_sold_date_sk = dev.iceberg100.date_dim.d_date_sk
         and dev.iceberg100.web_sales.ws_bill_customer_sk = dev.iceberg100.customer.c_customer_sk
         and d_month_seq between 1221 and 1221+11)
) cool_cust

In [ ]:
%%sql
--#q88

select  *
from
 (select count(*) h8_30_to_9
 from dev.iceberg100.store_sales, dev.iceberg100.household_demographics , dev.iceberg100.time_dim, dev.iceberg100.store
 where ss_sold_time_sk = dev.iceberg100.time_dim.t_time_sk
     and ss_hdemo_sk = dev.iceberg100.household_demographics.hd_demo_sk
     and ss_store_sk = s_store_sk
     and dev.iceberg100.time_dim.t_hour = 8
     and dev.iceberg100.time_dim.t_minute >= 30
     and ((dev.iceberg100.household_demographics.hd_dep_count = 1 and dev.iceberg100.household_demographics.hd_vehicle_count<=1+2) or
          (dev.iceberg100.household_demographics.hd_dep_count = 2 and dev.iceberg100.household_demographics.hd_vehicle_count<=2+2) or
          (dev.iceberg100.household_demographics.hd_dep_count = 0 and dev.iceberg100.household_demographics.hd_vehicle_count<=0+2))
     and dev.iceberg100.store.s_store_name = 'ese') s1,
 (select count(*) h9_to_9_30
 from dev.iceberg100.store_sales, dev.iceberg100.household_demographics , dev.iceberg100.time_dim, dev.iceberg100.store
 where ss_sold_time_sk = dev.iceberg100.time_dim.t_time_sk
     and ss_hdemo_sk = dev.iceberg100.household_demographics.hd_demo_sk
     and ss_store_sk = s_store_sk
     and dev.iceberg100.time_dim.t_hour = 9
     and dev.iceberg100.time_dim.t_minute < 30
     and ((dev.iceberg100.household_demographics.hd_dep_count = 1 and dev.iceberg100.household_demographics.hd_vehicle_count<=1+2) or
          (dev.iceberg100.household_demographics.hd_dep_count = 2 and dev.iceberg100.household_demographics.hd_vehicle_count<=2+2) or
          (dev.iceberg100.household_demographics.hd_dep_count = 0 and dev.iceberg100.household_demographics.hd_vehicle_count<=0+2))
     and dev.iceberg100.store.s_store_name = 'ese') s2,
 (select count(*) h9_30_to_10
 from dev.iceberg100.store_sales, dev.iceberg100.household_demographics , dev.iceberg100.time_dim, dev.iceberg100.store
 where ss_sold_time_sk = dev.iceberg100.time_dim.t_time_sk
     and ss_hdemo_sk = dev.iceberg100.household_demographics.hd_demo_sk
     and ss_store_sk = s_store_sk
     and dev.iceberg100.time_dim.t_hour = 9
     and dev.iceberg100.time_dim.t_minute >= 30
     and ((dev.iceberg100.household_demographics.hd_dep_count = 1 and dev.iceberg100.household_demographics.hd_vehicle_count<=1+2) or
          (dev.iceberg100.household_demographics.hd_dep_count = 2 and dev.iceberg100.household_demographics.hd_vehicle_count<=2+2) or
          (dev.iceberg100.household_demographics.hd_dep_count = 0 and dev.iceberg100.household_demographics.hd_vehicle_count<=0+2))
     and dev.iceberg100.store.s_store_name = 'ese') s3,
 (select count(*) h10_to_10_30
 from dev.iceberg100.store_sales, dev.iceberg100.household_demographics , dev.iceberg100.time_dim, dev.iceberg100.store
 where ss_sold_time_sk = dev.iceberg100.time_dim.t_time_sk
     and ss_hdemo_sk = dev.iceberg100.household_demographics.hd_demo_sk
     and ss_store_sk = s_store_sk
     and dev.iceberg100.time_dim.t_hour = 10
     and dev.iceberg100.time_dim.t_minute < 30
     and ((dev.iceberg100.household_demographics.hd_dep_count = 1 and dev.iceberg100.household_demographics.hd_vehicle_count<=1+2) or
          (dev.iceberg100.household_demographics.hd_dep_count = 2 and dev.iceberg100.household_demographics.hd_vehicle_count<=2+2) or
          (dev.iceberg100.household_demographics.hd_dep_count = 0 and dev.iceberg100.household_demographics.hd_vehicle_count<=0+2))
     and dev.iceberg100.store.s_store_name = 'ese') s4,
 (select count(*) h10_30_to_11
 from dev.iceberg100.store_sales, dev.iceberg100.household_demographics , dev.iceberg100.time_dim, dev.iceberg100.store
 where ss_sold_time_sk = dev.iceberg100.time_dim.t_time_sk
     and ss_hdemo_sk = dev.iceberg100.household_demographics.hd_demo_sk
     and ss_store_sk = s_store_sk
     and dev.iceberg100.time_dim.t_hour = 10
     and dev.iceberg100.time_dim.t_minute >= 30
     and ((dev.iceberg100.household_demographics.hd_dep_count = 1 and dev.iceberg100.household_demographics.hd_vehicle_count<=1+2) or
          (dev.iceberg100.household_demographics.hd_dep_count = 2 and dev.iceberg100.household_demographics.hd_vehicle_count<=2+2) or
          (dev.iceberg100.household_demographics.hd_dep_count = 0 and dev.iceberg100.household_demographics.hd_vehicle_count<=0+2))
     and dev.iceberg100.store.s_store_name = 'ese') s5,
 (select count(*) h11_to_11_30
 from dev.iceberg100.store_sales, dev.iceberg100.household_demographics , dev.iceberg100.time_dim, dev.iceberg100.store
 where ss_sold_time_sk = dev.iceberg100.time_dim.t_time_sk
     and ss_hdemo_sk = dev.iceberg100.household_demographics.hd_demo_sk
     and ss_store_sk = s_store_sk
     and dev.iceberg100.time_dim.t_hour = 11
     and dev.iceberg100.time_dim.t_minute < 30
     and ((dev.iceberg100.household_demographics.hd_dep_count = 1 and dev.iceberg100.household_demographics.hd_vehicle_count<=1+2) or
          (dev.iceberg100.household_demographics.hd_dep_count = 2 and dev.iceberg100.household_demographics.hd_vehicle_count<=2+2) or
          (dev.iceberg100.household_demographics.hd_dep_count = 0 and dev.iceberg100.household_demographics.hd_vehicle_count<=0+2))
     and dev.iceberg100.store.s_store_name = 'ese') s6,
 (select count(*) h11_30_to_12
 from dev.iceberg100.store_sales, dev.iceberg100.household_demographics , dev.iceberg100.time_dim, dev.iceberg100.store
 where ss_sold_time_sk = dev.iceberg100.time_dim.t_time_sk
     and ss_hdemo_sk = dev.iceberg100.household_demographics.hd_demo_sk
     and ss_store_sk = s_store_sk
     and dev.iceberg100.time_dim.t_hour = 11
     and dev.iceberg100.time_dim.t_minute >= 30
     and ((dev.iceberg100.household_demographics.hd_dep_count = 1 and dev.iceberg100.household_demographics.hd_vehicle_count<=1+2) or
          (dev.iceberg100.household_demographics.hd_dep_count = 2 and dev.iceberg100.household_demographics.hd_vehicle_count<=2+2) or
          (dev.iceberg100.household_demographics.hd_dep_count = 0 and dev.iceberg100.household_demographics.hd_vehicle_count<=0+2))
     and dev.iceberg100.store.s_store_name = 'ese') s7,
 (select count(*) h12_to_12_30
 from dev.iceberg100.store_sales, dev.iceberg100.household_demographics , dev.iceberg100.time_dim, dev.iceberg100.store
 where ss_sold_time_sk = dev.iceberg100.time_dim.t_time_sk
     and ss_hdemo_sk = dev.iceberg100.household_demographics.hd_demo_sk
     and ss_store_sk = s_store_sk
     and dev.iceberg100.time_dim.t_hour = 12
     and dev.iceberg100.time_dim.t_minute < 30
     and ((dev.iceberg100.household_demographics.hd_dep_count = 1 and dev.iceberg100.household_demographics.hd_vehicle_count<=1+2) or
          (dev.iceberg100.household_demographics.hd_dep_count = 2 and dev.iceberg100.household_demographics.hd_vehicle_count<=2+2) or
          (dev.iceberg100.household_demographics.hd_dep_count = 0 and dev.iceberg100.household_demographics.hd_vehicle_count<=0+2))
     and dev.iceberg100.store.s_store_name = 'ese') s8

In [ ]:
%%sql
--#q89

select  *
from(
select i_category, i_class, i_brand,
       s_store_name, s_company_name,
       d_moy,
       sum(ss_sales_price) sum_sales,
       avg(sum(ss_sales_price)) over
         (partition by i_category, i_brand, s_store_name, s_company_name)
         avg_monthly_sales
from dev.iceberg100.item, dev.iceberg100.store_sales, dev.iceberg100.date_dim, dev.iceberg100.store
where ss_item_sk = i_item_sk and
      ss_sold_date_sk = d_date_sk and
      ss_store_sk = s_store_sk and
      d_year in (2001) and
        ((i_category in ('Women','Music','Home') and
          i_class in ('fragrances','pop','bedding')
         )
      or (i_category in ('Books','Men','Children') and
          i_class in ('home repair','sports-apparel','infants')
        ))
group by i_category, i_class, i_brand,
         s_store_name, s_company_name, d_moy) tmp1
where case when (avg_monthly_sales <> 0) then (abs(sum_sales - avg_monthly_sales) / avg_monthly_sales) else null end > 0.1
order by sum_sales - avg_monthly_sales, s_store_name
limit 100

In [14]:
%%sql
--#q90

select  cast(amc as decimal(15,4))/cast(pmc as decimal(15,4)) am_pm_ratio
 from ( select count(*) amc
       from dev.iceberg100.web_sales, dev.iceberg100.household_demographics , dev.iceberg100.time_dim, dev.iceberg100.web_page
       where ws_sold_time_sk = dev.iceberg100.time_dim.t_time_sk
         and ws_ship_hdemo_sk = dev.iceberg100.household_demographics.hd_demo_sk
         and ws_web_page_sk = dev.iceberg100.web_page.wp_web_page_sk
         and dev.iceberg100.time_dim.t_hour between 8 and 8+1
         and dev.iceberg100.household_demographics.hd_dep_count = 4
         and dev.iceberg100.web_page.wp_char_count between 5000 and 5200) at,
      ( select count(*) pmc
       from dev.iceberg100.web_sales, dev.iceberg100.household_demographics , dev.iceberg100.time_dim, dev.iceberg100.web_page
       where ws_sold_time_sk = dev.iceberg100.time_dim.t_time_sk
         and ws_ship_hdemo_sk = dev.iceberg100.household_demographics.hd_demo_sk
         and ws_web_page_sk = dev.iceberg100.web_page.wp_web_page_sk
         and dev.iceberg100.time_dim.t_hour between 19 and 19+1
         and dev.iceberg100.household_demographics.hd_dep_count = 4
         and dev.iceberg100.web_page.wp_char_count between 5000 and 5200) pt
 order by am_pm_ratio
 limit 100

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [13]:
%%sql
--#q91

select
        cc_call_center_id Call_Center,
        cc_name Call_Center_Name,
        cc_manager Manager,
        sum(cr_net_loss) Returns_Loss
from
        dev.iceberg100.call_center,
        dev.iceberg100.catalog_returns,
        dev.iceberg100.date_dim,
        dev.iceberg100.customer,
        dev.iceberg100.customer_address,
        dev.iceberg100.customer_demographics,
        dev.iceberg100.household_demographics
where
        cr_call_center_sk       = cc_call_center_sk
and     cr_returned_date_sk     = d_date_sk
and     cr_returning_customer_sk= c_customer_sk
and     cd_demo_sk              = c_current_cdemo_sk
and     hd_demo_sk              = c_current_hdemo_sk
and     ca_address_sk           = c_current_addr_sk
and     d_year                  = 2002
and     d_moy                   = 11
and     ( (cd_marital_status       = 'M' and cd_education_status     = 'Unknown')
        or(cd_marital_status       = 'W' and cd_education_status     = 'Advanced Degree'))
and     hd_buy_potential like '5001-10000%'
and     ca_gmt_offset           = -6
group by cc_call_center_id,cc_name,cc_manager,cd_marital_status,cd_education_status
order by sum(cr_net_loss) desc

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [12]:
%%sql
--#q92

select
   sum(ws_ext_discount_amt)  as `Excess Discount Amount`
from
    dev.iceberg100.web_sales
   ,dev.iceberg100.item
   ,dev.iceberg100.date_dim
where
i_manufact_id = 561
and i_item_sk = ws_item_sk
and d_date between '2001-03-13' and
        (cast('2001-03-13' as date) + INTERVAL 90 days)
and d_date_sk = ws_sold_date_sk
and ws_ext_discount_amt
     > (
         SELECT
            1.3 * avg(ws_ext_discount_amt)
         FROM
            dev.iceberg100.web_sales
           ,dev.iceberg100.date_dim
         WHERE
              ws_item_sk = i_item_sk
          and d_date between '2001-03-13' and
                             (cast('2001-03-13' as date) + INTERVAL 90 days)
          and d_date_sk = ws_sold_date_sk
      )
order by sum(ws_ext_discount_amt)
limit 100

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [15]:
%%sql
--#q93

select  ss_customer_sk
            ,sum(act_sales) sumsales
      from (select ss_item_sk
                  ,ss_ticket_number
                  ,ss_customer_sk
                  ,case when sr_return_quantity is not null then (ss_quantity-sr_return_quantity)*ss_sales_price
                                                            else (ss_quantity*ss_sales_price) end act_sales
            from dev.iceberg100.store_sales left outer join dev.iceberg100.store_returns on (sr_item_sk = ss_item_sk
                                                               and sr_ticket_number = ss_ticket_number)
                ,dev.iceberg100.reason
            where sr_reason_sk = r_reason_sk
              and r_reason_desc = 'reason 64')
      group by ss_customer_sk
      order by sumsales, ss_customer_sk
limit 100

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [10]:
%%sql
--#q94

select
   count(distinct ws_order_number) as `order count`
  ,sum(ws_ext_ship_cost) as `total shipping cost`
  ,sum(ws_net_profit) as `total net profit`
from
   dev.iceberg100.web_sales ws1
  ,dev.iceberg100.date_dim
  ,dev.iceberg100.customer_address
  ,dev.iceberg100.web_site
where
    d_date between '2001-5-01' and
           (cast('2001-5-01' as date) + INTERVAL 60 days)
and ws1.ws_ship_date_sk = d_date_sk
and ws1.ws_ship_addr_sk = ca_address_sk
and ca_state = 'TX'
and ws1.ws_web_site_sk = web_site_sk
and web_company_name = 'pri'
and exists (select *
            from dev.iceberg100.web_sales ws2
            where ws1.ws_order_number = ws2.ws_order_number
              and ws1.ws_warehouse_sk <> ws2.ws_warehouse_sk)
and not exists(select *
               from dev.iceberg100.web_returns wr1
               where ws1.ws_order_number = wr1.wr_order_number)
order by count(distinct ws_order_number)
limit 100

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [9]:
%%sql
--#q95

with ws_wh as
(select ws1.ws_order_number,ws1.ws_warehouse_sk wh1,ws2.ws_warehouse_sk wh2
 from dev.iceberg100.web_sales ws1,dev.iceberg100.web_sales ws2
 where ws1.ws_order_number = ws2.ws_order_number
   and ws1.ws_warehouse_sk <> ws2.ws_warehouse_sk)
 select
   count(distinct ws_order_number) as `order count`
  ,sum(ws_ext_ship_cost) as `total shipping cost`
  ,sum(ws_net_profit) as `total net profit`
from
   dev.iceberg100.web_sales ws1
  ,dev.iceberg100.date_dim
  ,dev.iceberg100.customer_address
  ,dev.iceberg100.web_site
where
    d_date between '2000-3-01' and
           (cast('2000-3-01' as date) + INTERVAL 60 days)
and ws1.ws_ship_date_sk = d_date_sk
and ws1.ws_ship_addr_sk = ca_address_sk
and ca_state = 'TN'
and ws1.ws_web_site_sk = web_site_sk
and web_company_name = 'pri'
and ws1.ws_order_number in (select ws_order_number
                            from ws_wh)
and ws1.ws_order_number in (select wr_order_number
                            from dev.iceberg100.web_returns,ws_wh
                            where wr_order_number = ws_wh.ws_order_number)
order by count(distinct ws_order_number)
limit 100

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [ ]:
%%sql
--#q96

select  count(*)
from dev.iceberg100.store_sales
    ,dev.iceberg100.household_demographics
    ,dev.iceberg100.time_dim, dev.iceberg100.store
where ss_sold_time_sk = dev.iceberg100.time_dim.t_time_sk
    and ss_hdemo_sk = dev.iceberg100.household_demographics.hd_demo_sk
    and ss_store_sk = s_store_sk
    and dev.iceberg100.time_dim.t_hour = 16
    and dev.iceberg100.time_dim.t_minute >= 30
    and dev.iceberg100.household_demographics.hd_dep_count = 4
    and dev.iceberg100.store.s_store_name = 'ese'
order by count(*)
limit 100

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [21]:
%timeit for _ in range(1000): True

20.2 µs ± 3.35 µs per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [26]:
%%sql
--#q97

with ssci as (
select ss_customer_sk customer_sk
      ,ss_item_sk item_sk
from dev.iceberg100.store_sales,dev.iceberg100.date_dim
where ss_sold_date_sk = d_date_sk
  and d_month_seq between 1209 and 1209 + 11
group by ss_customer_sk
        ,ss_item_sk),
csci as(
 select cs_bill_customer_sk customer_sk
      ,cs_item_sk item_sk
from dev.iceberg100.catalog_sales,dev.iceberg100.date_dim
where cs_sold_date_sk = d_date_sk
  and d_month_seq between 1209 and 1209 + 11
group by cs_bill_customer_sk
        ,cs_item_sk)
 select  sum(case when ssci.customer_sk is not null and csci.customer_sk is null then 1 else 0 end) store_only
      ,sum(case when ssci.customer_sk is null and csci.customer_sk is not null then 1 else 0 end) catalog_only
      ,sum(case when ssci.customer_sk is not null and csci.customer_sk is not null then 1 else 0 end) store_and_catalog
from ssci full outer join csci on (ssci.customer_sk=csci.customer_sk
                               and ssci.item_sk = csci.item_sk)
limit 100

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [6]:
%%sql
--#q98

select i_item_id
      ,i_item_desc
      ,i_category
      ,i_class
      ,i_current_price
      ,sum(ss_ext_sales_price) as itemrevenue
      ,sum(ss_ext_sales_price)*100/sum(sum(ss_ext_sales_price)) over
          (partition by i_class) as revenueratio
from
	dev.iceberg100.store_sales
    	,dev.iceberg100.item
    	,dev.iceberg100.date_dim
where
	ss_item_sk = i_item_sk
  	and i_category in ('Jewelry', 'Home', 'Shoes')
  	and ss_sold_date_sk = d_date_sk
	and d_date between cast('2001-04-12' as date)
				and (cast('2001-04-12' as date) + interval 30 days)
group by
	i_item_id
        ,i_item_desc
        ,i_category
        ,i_class
        ,i_current_price
order by
	i_category
        ,i_class
        ,i_item_id
        ,i_item_desc
        ,revenueratio

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [5]:
%%sql
--#q99

select
   substr(w_warehouse_name,1,20)
  ,sm_type
  ,cc_name
  ,sum(case when (cs_ship_date_sk - cs_sold_date_sk <= 30 ) then 1 else 0 end)  as `30 days`
  ,sum(case when (cs_ship_date_sk - cs_sold_date_sk > 30) and
                 (cs_ship_date_sk - cs_sold_date_sk <= 60) then 1 else 0 end )  as `31-60 days`
  ,sum(case when (cs_ship_date_sk - cs_sold_date_sk > 60) and
                 (cs_ship_date_sk - cs_sold_date_sk <= 90) then 1 else 0 end)  as `61-90 days`
  ,sum(case when (cs_ship_date_sk - cs_sold_date_sk > 90) and
                 (cs_ship_date_sk - cs_sold_date_sk <= 120) then 1 else 0 end)  as `91-120 days`
  ,sum(case when (cs_ship_date_sk - cs_sold_date_sk  > 120) then 1 else 0 end)  as `>120 days`
from
   dev.iceberg100.catalog_sales
  ,dev.iceberg100.warehouse
  ,dev.iceberg100.ship_mode
  ,dev.iceberg100.call_center
  ,dev.iceberg100.date_dim
where
    d_month_seq between 1203 and 1203 + 11
and cs_ship_date_sk   = d_date_sk
and cs_warehouse_sk   = w_warehouse_sk
and cs_ship_mode_sk   = sm_ship_mode_sk
and cs_call_center_sk = cc_call_center_sk
group by
   substr(w_warehouse_name,1,20)
  ,sm_type
  ,cc_name
order by substr(w_warehouse_name,1,20)
        ,sm_type
        ,cc_name
limit 100

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()